# Source Code 400 with Vis and Zip

## 步骤 1：环境设置与库导入

这是我们的“军火库”，包含了所有执行任务所需的工具。我们将安装 zopfli 以获得顶级的DEFLATE压缩效果，并导入用于数据处理、可视化和文件操作的全套库。

In [ ]:
# ===================================================================
# 最终章 V14：一体化优化与分析引擎 (终极版)
# ===================================================================
print("🚀 终极优化与分析流程启动...")

# ===================================================================
# 步骤 1：环境设置与库导入
# ===================================================================
print("\n🔧 步骤 1/9: 环境设置与库导入")

import subprocess, sys, os, zipfile, shutil, json, copy, zlib, warnings, time, glob, re, traceback, importlib.util
try:
    import zopfli, plotly, nltk
    print("   - 核心库 zopfli, plotly, nltk 已安装。")
except ImportError:
    print("   - 正在安装核心库 zopfli, plotly, nltk...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "zopfli", "plotly", "nltk", "-q"])
import zopfli.zlib, numpy
from tqdm.notebook import tqdm
from collections import defaultdict, Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

print("✅ 步骤 1/9 完成")

## 步骤 2：定义核心功能函数

这里是引擎的核心部件。我们定义了三个关键函数：

1. check_solution: 一个健壮的验证器，用于确保代码的功能正确性。
2. decode_if_compressed: 一个解码器，用于处理从其他来源获取的、可能已经被压缩过的代码，确保我们在同一起跑线上比较长度。
3. zip_src_extreme: 我们的“极限压缩”函数，它会不择手段地尝试各种方法来缩减代码的最终字节数。

In [ ]:
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *
show_legend()

In [ ]:
# ===================================================================
# 步骤 2：定义核心功能函数 (集成所有顶尖策略)
# ===================================================================
print("\n⚙️ 步骤 2/9: 定义核心功能函数")

def decode_and_clean(code_bytes, task_num):
    if (b'#encoding:L1' in code_bytes or b'#coding:L1' in code_bytes) and b'zlib.decompress' in code_bytes:
        try:
            temp_py_path = f"/kaggle/working/temp_decompress_{task_num}.py"
            code_str = code_bytes.decode('latin-1').replace('exec', 'print', 1)
            with open(temp_py_path, 'w', encoding='latin-1') as f: f.write(code_str)
            result = subprocess.run([sys.executable, temp_py_path], capture_output=True, text=False, timeout=5)
            os.remove(temp_py_path)
            if result.returncode == 0 and result.stdout.startswith(b"b'") and result.stdout.endswith(b"'\n"):
                code_bytes = result.stdout[2:-2].decode('unicode_escape').encode('latin-1')
        except Exception:
            if 'temp_py_path' in locals() and os.path.exists(temp_py_path): os.remove(temp_py_path)
    s = code_bytes.decode('latin-1').replace('\r\n', '\n').replace('\r', '\n').replace('\n\n', '\n').strip()
    return s.encode('latin-1')

def zip_src_uber_extreme(src_code_bytes, substitutions={}):
    options = [src_code_bytes]
    code_variants_to_compress = {"original": src_code_bytes}
    if substitutions:
        substituted_code = src_code_bytes
        for old, new in substitutions.items(): substituted_code = substituted_code.replace(old, new)
        if substituted_code != src_code_bytes: code_variants_to_compress["substituted"] = substituted_code
    for variant_name, code_to_compress in code_variants_to_compress.items():
        current_substitutions = substitutions if variant_name == "substituted" else {}
        for compress_func in [zopfli.zlib.compress, lambda data: zlib.compress(data, 9)]:
            for trailing in [b"", b"\n", b" ", b"#"]:
                src = code_to_compress + trailing
                while (compressed := compress_func(src))[-1] == ord('"'): src += b"#"
                
                # --- SYNTAX ERROR FIX START ---
                def sanitize(b_in):
                    b_out = bytearray()
                    for b in b_in:
                        if b == 0:
                            b_out.extend(b'\\x00')
                        elif b == ord('\r'):
                            b_out.extend(b'\\r')
                        elif b == ord('\\'):
                            b_out.extend(b'\\\\')
                        else:
                            b_out.append(b)
                    return bytes(b_out)
                # --- SYNTAX ERROR FIX END ---

                compressed_sanitized = sanitize(compressed)
                delim = b'"'
                if b'"' in compressed_sanitized: delim = b"'"
                if b"'" in compressed_sanitized and b'"' in compressed_sanitized:
                    delim = b'"""' if b'"""' not in compressed_sanitized else b"'''"
                exec_core_parts = [f'zlib.decompress(bytes({delim.decode("latin-1")}{compressed_sanitized.decode("latin-1")}{delim.decode("latin-1")},"latin-1"))']
                if current_substitutions:
                    for old, new in current_substitutions.items(): exec_core_parts.append(f'.replace({repr(new)}, {repr(old)})')
                final_code = f'#encoding:L1\nimport zlib\nexec({"".join(exec_core_parts)})'.encode('latin-1')
                try:
                    exec(final_code, {}); options.append(final_code)
                except Exception: continue
    return min(options, key=len)
    
def simple_verify_program(task_num, examples):
    task_name, task_path = "task_with_imports", "/kaggle/working/task.py"
    spec = importlib.util.spec_from_file_location(task_name, task_path)
    if spec is None: return 0, 1, 0, 1
    module = sys.modules.get(task_name) or importlib.util.module_from_spec(spec)
    sys.modules[task_name] = module
    spec.loader.exec_module(module)
    if not hasattr(module, "p") or not callable(getattr(module, "p")): return 0, 1, 0, 1
    program = getattr(module, "p")
    def verify(example_subset):
        right, wrong = 0, 0
        for example in example_subset:
            try:
                if numpy.array_equal(np.array(program(copy.deepcopy(example["input"]))), np.array(example["output"])): right += 1
                else: wrong += 1
            except: wrong += 1
        return right, wrong
    agi_r, agi_w = verify(examples["train"] + examples["test"])
    gen_r, gen_w = verify(examples["arc-gen"])
    return agi_r, agi_w, gen_r, gen_w

print("✅ 步骤 2/9 完成")

# Respect, Credit and Recognition
Huge thanks to everyone for sharing and contributing in this competition!

This collection belongs to all of us—it’s built from your efforts too. We’re now at 400, enjoy!

I have included my own solutions and refinements on existing problems but the vast majority of the solutions come from aggregating together work from many others:

@cristianocalcagno → gcgc-solutions-for-all-400-tasks

@jazivxt → oh-barnacles

@jazivxt → code-compression

@markvoi → google-golf-code-98-solved-problems

@mbilichenko → golf-code-optimization-trials

@cheeseexports → big-zippa

Also, for all the other tasks solved, credit goes to the many contributors—even if I didn’t reference code from them directly: @adilshamim8 @adyanthm @boristown @bibanh @cheeseexports @daosyduyminh @dedquoc @fedimser @henrychibueze @jacekwl @jaejohn @jazivxt @jeroencottaar @katsuyanomura @kosirowada @krishnayadav456wrsty @kuntalmaity @limalkasadith @mcwema @mmoffitt @mpwolke @muhammaddanyalmalik @muhammadqasimshabbir @nina2025 @ottitsch @quannguyn12 @raviannaswamy @seshura @seshurajup @shevtsovapolina @taylorsamarel @zshashz

In [ ]:
show_examples(load_examples(1)['train'])

In [ ]:
%%writefile task001.py
p=lambda g:[[x&y for x in a for y in b]for a in g for b in g]

# [001] 007bbfb7.json
* image_repetition
* fractal_repetition

# [002] 00d62c1b.json
* loop_filling

In [ ]:
show_examples(load_examples(2)['train'])

In [ ]:
%%writefile task002.py
def p(a):
 def o(i,e):
  try:
   if a[i][e]<1:a[i][e]=1;o(i+1,e);o(i-1,e);o(i,e+1);o(i,e-1)
  except:0
 for i in range(len(a)):o(i,0);o(i,-1);o(0,i);o(-1,i)
 return[[(4,0,0,3)[o]for o in o]for o in a]

# [003] 017c7c7b.json
* recoloring
* pattern_expansion
* pattern_repetition
* image_expansion

In [ ]:
show_examples(load_examples(3)['train'])

In [ ]:
%%writefile task003.py
p=lambda g:[[c*2for c in r]for r in g+(g[:3],g[2:5])[g[1]!=g[4]]]

# [004] 025d127b.json
* pattern_modification

In [ ]:
show_examples(load_examples(4)['train'])

In [ ]:
%%writefile task004.py
def p(g,p=enumerate):
 a=[[0]*len(g[0])for _ in g]
 for q in{*sum(g,[])}-{0}:
  f=[(y,x)for y,r in p(g)for x,v in p(r)if v==q];k,l=map(max,zip(*f))
  for y,x in f:a[y][x+(y<k)*(x<l)]=q
 return a

# [005] 045e512c.json
* pattern_expansion
* direction_guessing

In [ ]:
show_examples(load_examples(5)['train'])

In [ ]:
%%writefile task005.py
def p(a):
	o,d=len(a),len(a[0]);e=[(m,s)for m in(-4,0,4)for s in(-4,0,4)if m|s];t=range;q=[d]*10;r=[o]*10;j=[-1]*10;b=[-1]*10
	for s in t(o):
		for h in t(d):
			m=a[s][h]
			if h<q[m]:q[m]=h
			if s<r[m]:r[m]=s
			if h>j[m]:j[m]=h
			if s>b[m]:b[m]=s
	for m in t(1,10):
		if j[m]-q[m]==2 and b[m]-r[m]==2:c,i=q[m],r[m];break
	r=[[0]*d for m in a]
	for x in t(3):r[i+x][c:c+3]=a[i+x][c:c+3]
	z=[(s,m)for m in t(3)for s in t(3)if a[i+m][c+s]]
	for(f,u)in e:
		h=c+f;s=i+u;m=0
		for k in t(3):
			for y in t(3):
				v,l=h+y,s+k
				if 0<=v<d and 0<=l<o and a[l][v]:m=a[l][v];break
			if m:break
		if not m:continue
		h=c;s=i
		while 1:
			h+=f;s+=u
			if not(-3<h<d and-3<s<o):break
			for(y,k)in z:
				v,l=h+y,s+k
				if 0<=v<d and 0<=l<o:r[l][v]=m
	return r

# [006] 0520fde7.json
* detect_wall
* separate_images
* pattern_intersection

In [ ]:
show_examples(load_examples(6)['train'])

In [ ]:
%%writefile task006.py
p=lambda j:[[x*y*2for x,y in zip(r,r[4:])]for r in j[:3]]

# [007] 05269061.json
* image_filling
* pattern_expansion
* diagonals

In [ ]:
show_examples(load_examples(7)['train'])

In [ ]:
%%writefile task007.py
def p(g):a,b=range(len(g)),range(len(g[0]));d={(i+j)%3:g[i][j]for i in a for j in b if g[i][j]};return[[d.get((i+j)%3,0)for j in b]for i in a]

In [ ]:
show_examples(load_examples(8)['train'])

In [ ]:
%%writefile task008.py
def p(s,u=enumerate):
 q,E=[(q,b)for(q,E)in u(s)for(b,d)in u(E)if d==2],[(q,b)for(q,E)in u(s)for(b,d)in u(E)if d==8]
 if not q or not E:return s
 k=lambda b:(min(q for(q,E)in b),max(q for(q,E)in b),min(q for(E,q)in b),max(q for(E,q)in b));l,n,a,r=k(q);e,i,w,k=k(E);b=d=0
 if r<w:d=w-r-1
 elif k<a:d=k-a+1
 elif n<e:b=e-n-1
 elif i<l:b=i-l+1
 e,n={*q},{*E};return[[8 if(q,E)in n else 2 if(q-b,E-d)in e else 0 for(E,k)in u(s[0])]for(q,E)in u(s)]

# [009] 06df4c85.json
* detect_grid
* connect_the_dots
* grid_coloring

In [ ]:
show_examples(load_examples(9)['train'])

In [ ]:
%%writefile task009.py
def p(m):
 l=range((k:=len(m)));h=m[2][2];r=eval(str((n:=[[r*(r!=h)for r in r]for r in m])));E=lambda t:(t.index(d),k-t[::-1].index(d))
 for d in l:
  for i in l:
   if d in(t:=n[i]):a,b=E(t);r[i][a:b]=[d]*(b-a)
   if d in(t:=[r[i]for r in n]):
    for j in range(*E(t)):r[j][i]=d
 return[[(h,r[i][j])[m[i][j]!=h]for j in l]for i in l]

# [010] 08ed6ac7.json
* measure_length
* order_numbers
* associate_colors_to_ranks
* recoloring

In [ ]:
show_examples(load_examples(10)['train'])

In [ ]:
%%writefile task010.py
p=lambda g:(a:={})or[[x if x-5else a.setdefault(i,len(a)+1) for i,x in enumerate(r)]for r in g]

# [011] 09629e4f.json
* detect_grid
* separate_images
* count_tiles
* take_minimum
* enlarge_image
* create_grid
* adapt_image_to_grid

In [ ]:
show_examples(load_examples(11)['train'])

In [ ]:
%%writefile task011.py
p=lambda g,A=range:(lambda c,e:[[5if x%4==3or y%4==3else g[c*4+x//4][e*4+y//4]for y in A(11)]for x in A(11)])(*next((c,e)for c in A(3)for e in A(3)if sum(g[c*4+w][e*4+l]==0for w in A(3)for l in A(3))==5))

# [012] 0962bcdd.json
* pattern_expansion

In [ ]:
show_examples(load_examples(12)['train'])

In [ ]:
%%writefile task012.py
def p(j,A=range(-2,3),c=enumerate,E=abs):k=[E[:]for E in j];[k[I+D].__setitem__(C+F,H if E(D)==E(F)else B[C-1])for(I,B)in c(j)for(C,H)in c(B)if H and B[C-1]*B[C+1]for D in A for F in A if E(D)==E(F)or not D*F];return k

# [013] 0a938d79.json
* direction_guessing
* draw_line_from_point
* pattern_expansion

In [ ]:
show_examples(load_examples(13)['train'])

In [ ]:
%%writefile task013.py
def p(a):
 n=l=s=d=h=0;i=[]
 if(n:=sum(a[0])or sum(a[-1])):a=[*zip(*a[::-1])]
 H=len(a[0])
 for A in range(len(a)):
  if a[A][0]+a[A][-1]and s<2:a[A]=[a[A][0]+a[A][-1]]*H;s+=1;i+=a[A][:1];h=A
  l+=0<s<2
 for A in range(h+l,len(a),l):a[A]=[i[d]]*H;d^=1
 if n:a=[*map(list,zip(*a))][::-1]
 return a

# [014] 0b148d64.json
* detect_grid
* separate_images
* find_the_intruder
* crop

In [ ]:
show_examples(load_examples(14)['train'])

In [ ]:
%%writefile task014.py
from collections import*
def p(e):
 A=[l for l in e for l in l];o=Counter(A).most_common(3);o=[o for o in o if o[0]>0][-1][0];e=[l for l in e if o in l];R=[]
 for l in e:
  for t in range(len(l)):
   if l[t]==o:R+=[t]
 return[l[min(R):max(R)+1]for l in e]

# [015] 0ca9ddb6.json
* pattern_expansion
* associate_patterns_to_colors

In [ ]:
show_examples(load_examples(15)['train'])

In [ ]:
%%writefile task015.py
def p(g,r=range(9)):
 s=[(i,j,v)for i in r for j in r if 0<(v:=g[i][j])<3]
 for i,j,v in s:
  for a in-1,0,1:
   for b in-1,0,1:
    if a|b and(a*b!=0)==(v>1):g[i+a][j+b]=7-3*(v>1)
 return g

# [016] 0d3d703e.json
* associate_colors_to_colors

In [ ]:
show_examples(load_examples(16)['train'])

In [ ]:
%%writefile task016.py
p=lambda j:[[int('0564312798'[x])for x in r]for r in j]

# [017] 0dfd9992.json
* image_filling
* pattern_expansion

In [ ]:
show_examples(load_examples(17)['train'])

In [ ]:
%%writefile task017.py
def p(j,u=enumerate):
	e=range;a=len(j);a=len(j[0]);k=lambda t,k:t==k or t*k<1;i=next((r for r in e(1,a)if all(k(a,e)for n in j for(a,e)in zip(n,n[r:]))),a);m=next((r for r in e(1,a)if all(k(a,e)for(r,n)in zip(j,j[r:])for(a,e)in zip(r,n))),a);o={}
	for(e,r)in u(j):
		for(n,a)in u(r):
			if a:o[e%m,n%i]=a
	for(e,r)in u(j):
		for(n,a)in u(r):
			if not a:r[n]=o[e%m,n%i]
	return j

# [018] 0e206a2e.json
* associate_patterns_to_patterns
* pattern_repetition
* pattern_rotation
* pattern_reflection
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(18)['train'])

In [ ]:
%%writefile task018.py
def p(d):
 G=len(d);H=len(d[0]); R=[];K=set();V={*K}
 for r in range(G):
  for c in range(H):
   if(v:=d[r][c])and(P:=(v,r,c))not in V:
    C=[P];V.add(P)
    for J in C:
     for a,b in[(0,1),(0,-1),(1,0),(-1,0)]:
      if 0<=(A:=J[1]+a)<G and 0<=(D:=J[2]+b)<H and(w:=d[A][D])and(N:=(w,A,D))not in V:V.add(N);C.append(N)
    if len(C)>3:R+=[C]
    else:K.update(C)
 for B in R:
  Z=list(zip(*B));Ro,Co=min(Z[1]),min(Z[2]);L=[(v,r-Ro,c-Co)for v,r,c in B]
  if C:=next((Ca for W in range(3,7)if(T:=9-W)and(Y:=[[(v,[W-1-c,c,T-1-r,c][i],[r,T-1-r,c,r][i])for v,r,c in L]for i in range(4)])if(C:=next((Ca for Cl in Y for A in range(-6,G)for D in range(-6,H)if(Ca:=[(v,r+A,c+D)for v,r,c in Cl])and all(0<=p[1]<G and 0<=p[2]<H for p in Ca)and sum(p in K for p in Ca)==3),0))),0):
   for v,r,c in C:d[r][c]=v
   for v,r,c in B:d[r][c]=0
 return d

# [019] 10fcaaa3.json
* pattern_expansion
* image_repetition

In [ ]:
show_examples(load_examples(19)['train'])

In [ ]:
%%writefile task019.py
s=len
r=range
def p(g):
 g=[i[:]+i[:]for i in g]+[i[:]+i[:]for i in g];I,F=s(g),s(g[0])
 for i in r(I):
  for a in r(F):
   t=g[i][a]
   if t>0and t!=8:
    for(t,e)in[[1,1],[-1,-1],[-1,1],[1,-1]]:
     if t+i>=0and e+a>=0and t+i<I and e+a<F:
      if g[t+i][e+a]==0:g[t+i][e+a]=8
 return g

# [020] 11852cab.json
* pattern_expansion

In [ ]:
show_examples(load_examples(20)['train'])

In [ ]:
%%writefile task020.py
def p(g,H=enumerate):
 t=l=9**9;b=r=-1
 for y,a in H(g):
  for x,v in H(a):
   if v:t=min(t,y);b=max(b,y);l=min(l,x);r=max(r,x)
 s=t+b;S=l+r
 for y in range(t,b+1):
  for x in range(l,r+1):
   Y=s-y;X=S-x;u=t+x-l;v=l+y-t;U=t+r-x;V=l+b-y
   P=((y,x),(y,X),(Y,x),(Y,X),(u,v),(U,v),(u,V),(U,V))
   c=max(g[i][j]for i,j in P)
   for i,j in P:g[i][j]=c
 return g


# [021] 1190e5a7.json
* detect_grid
* count_hor_lines
* count_ver_lines
* detect_background_color
* color_guessing
* create_image_from_info

In [ ]:
show_examples(load_examples(21)['train'])

In [ ]:
%%writefile task021.py
p=lambda g:[[(l:=g[0][0])]*-~sum(x!=l for x in g[0])]*-~sum(r[0]!=l for r in g)

# [022] 137eaa0f.json
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(22)['train'])

In [ ]:
%%writefile task022.py
h=len
l=range
def p(g):
 e=[[0,0,0]for n in l(3)];d,d=h(g),h(g[0])
 for n in l(d):
  for r in l(d):
   if g[n][r]==5:
    for A in l(-1,2):
     for D in l(-1,2):
      if n+A>=0and r+D>=0and g[n+A][r+D]!=0:e[1+A][1+D]=g[n+A][r+D]
 return e

# [023] 150deff5.json
* pattern_coloring
* pattern_deconstruction
* associate_colors_to_patterns

In [ ]:
show_examples(load_examples(23)['train'])

In [ ]:
%%writefile task023.py
def p(g,f=enumerate,L=range):
 s={(A,i)for(A,s)in f(g)for(i,l)in f(s)if l}
 while s:
  i={A:[]for A in s}
  for l in L(3):
   for d in L(11):
    for r in L(11):
     A={(d+[A//2,A,0][l],r+[A%2,0,A][l])for A in L((4,3,3)[l])}
     if A<=s:[i[s].append(A)for s in A]
  r=min(s,key=lambda A:len(i[A]));A=i[r][0];s-=A
  for(G,t)in A:g[G][t]=(len(A)-3)*6+2
 return g

# [024] 178fcbfb.json
* direction_guessing
* draw_line_from_point

In [ ]:
show_examples(load_examples(24)['train'])

In [ ]:
%%writefile task024.py
p=lambda g,E=enumerate:(Z:={i for R in g for i,x in E(R)if x==2})and 0 or[[1in R and 1or 3in R and 3or v or(i in Z)*2for i,v in E(R)]for R in g]

# [025] 1a07d186.json
* bring_patterns_close
* find_the_intruder

In [ ]:
show_examples(load_examples(25)['train'])

In [ ]:
%%writefile task025.py
d=len
r=range
def p(g):
 f=[0]
 for i in range(8):
  g=list(map(list,zip(*g[::-1])))
  q,o=d(g),d(g[0])
  for i in r(q):
   if len(set(g[i]))<2 and g[i][0]>0:
    p=g[i][0];f+=[p]
    for e in r(0,i-1):
     n=0
     for t in r(o):
      if g[e][t]==p:g[e][t]=0;n-=1;g[i+n][t]=p
    for e in r(i+1,q):
     n=0
     for t in r(o):
      if g[e][t]==p:g[e][t]=0;n+=1;g[i+n][t]=p
 g=[[t if t in f else 0 for t in i] for i in g]
 return g

# [026] 1b2d62fb.json
* detect_wall
* separate_images
* pattern_intersection

In [ ]:
show_examples(load_examples(26)['train'])

In [ ]:
%%writefile task026.py
p=lambda j:[[8*(a|b<1)for a,b in zip(a,a[4:])]for a in j]

# [027] 1b60fb0c.json
* pattern_deconstruction
* pattern_rotation
* pattern_expansion

In [ ]:
show_examples(load_examples(27)['train'])

In [ ]:
%%writefile task027.py
def p(g):
 r=[f[:]for f in g];k={(x,a)for x in range(len(g))for a in range(len(g[0]))if g[x][a]==1}
 if not k:return r
 for o in[0,1]:
  l=set()
  for x,a in k:
   for x,a in[(5-x+o,a-5),(5-a+o,4-x+o),(x-5,a-4)]:
    if -5<=x<=5 and -5<=a<=5:l.add((x,a))
  p=[r for r in l if all((5-x+o,5+a)in k or not(0<=5-x+o<len(g)and 0<=5+a<len(g[0]))for x,a in[r])and all((4-a+o,5-x+o)in k or not(0<=4-a+o<len(g)and 0<=5-x+o<len(g[0]))for x,a in[r])and all((5+a,4+x)in k or not(0<=5+a<len(g)and 0<=4+x<len(g[0]))for x,a in[r])and any(0<=5-x+o<len(g)and 0<=5+a<len(g[0])and(5-x+o,5+a)in k for x,a in[r])]
  if p:
   s={(5-x+o,5+a)for x,a in p if 0<=5-x+o<len(g)and 0<=5+a<len(g[0])}|{(4-a+o,5-x+o)for x,a in p if 0<=4-a+o<len(g)and 0<=5-x+o<len(g[0])}|{(5+a,4+x)for x,a in p if 0<=5+a<len(g)and 0<=4+x<len(g[0])}
   if s==k:
    for x,a in p:
     if 0<=4+x<len(r)and 0<=4-a+o<len(r[0])and r[4+x][4-a+o]==0:r[4+x][4-a+o]=2
    break
 return r

# [028] 1bfc4729.json
* pattern_expansion

In [ ]:
show_examples(load_examples(28)['train'])

In [ ]:
%%writefile task028.py
def p(g,m=max):e,a=m(map(m,g[:5])),m(map(m,g[-5:]));return[[(r in(0,2,7,9)or c%9<1)*(e*(r<5)+a*(r>4))for c in range(10)]for r in range(10)]

# [029] 1c786137.json
* detect_enclosure
* crop

In [ ]:
show_examples(load_examples(29)['train'])

In [ ]:
%%writefile task029.py
def p(g,a=len,e=enumerate):
 for i in set(sum(g,[])):
  h=[[r,h]for(h,l)in e(g)for(r,n)in e(l)if n==i];l=sum(h,[]);r=l[::2];n=l[1::2];h=g[min(n):max(n)];h=[h[min(r)+1:max(r)][:]for h in h]
  if h[0].count(i)==a(h[0]):return h[1:]
 return g

# [030] 1caeab9d.json
* pattern_moving
* pattern_alignment

In [ ]:
show_examples(load_examples(30)['train'])

In [ ]:
%%writefile task030.py
def p(g,L=len,l=range):
 d,S,o,T,C=L(g),L(g[0]),[],[],[]
 for B in l(d):
  for A in l(S):
   a=g[B][A]
   if a==2:o+=[A];g[B][A]=0
   if a==4:T+=[A];g[B][A]=0
   if a==1:C+=[A]
 for B in l(d):
  for A in l(S):
   if g[B][A]==1:g[B][A+(min(T)-min(C))]=4;g[B][A+(min(o)-min(C))]=2
 return g

# [031] 1cf80156.json
* crop

In [ ]:
show_examples(load_examples(31)['train'])

In [ ]:
%%writefile task031.py
def p(j,A=enumerate):c,E=zip(*[(i,j)for i,r in A(j)for j,x in A(r)if x]);return[r[min(E):max(E)+1]for r in j[min(c):max(c)+1]]

# [032] 1e0a9b12.json
* pattern_moving
* gravity

In [ ]:
show_examples(load_examples(32)['train'])

In [ ]:
%%writefile task032.py
p=lambda g:[*zip(*map(sorted,zip(*g)))]

# [033] 1e32b0e9.json
* detect_grid
* separate_images
* image_repetition
* pattern_completion

In [ ]:
show_examples(load_examples(33)['train'])

In [ ]:
%%writefile task033.py
def p(r):
 A=range;e=[A[:]for A in r];I=r[5][0];a=[[r[B+1][A+1]for A in A(3)]for B in A(3)]
 for s in[(0,6),(0,12),(6,0),(6,6),(6,12),(12,0),(12,6),(12,12)]:
  for B in A(3):
   for C in A(3):D,E=s[0]+B+1,s[1]+C+1;e[D][E]=r[D][E]if a[B][C]==r[D][E]else I if a[B][C]else 0
 return e

# [034] 1f0c79e5.json
* pattern_expansion
* diagonals
* direction_guessing

In [ ]:
show_examples(load_examples(34)['train'])

In [ ]:
%%writefile task034.py
h=range
N=len
def p(g):
 D=sum(g,[]);C=[R for R in set(D)if R not in[0,2]][0];i,i=N(g),N(g[0])
 for G in h(i-1):
  for l in h(i-1):
   e=[g[G+R][l+n]for(R,n)in[[0,0],[0,1],[1,0],[1,1]]]
   if sum([1for R in e if R>0])>3:
    for e in[R for R in h(N(e))if e[R]==2]:
     for a in h(10):
      if e<2:R=G-a
      else:R=G+a+1
      if e%2:n=l+a+1
      else:n=l-a
      if 0<=R<i and 0<=n<i:
       g[R][n]=C
       if 0<=R-1<i:g[R-1][n]=C
       if 0<=n-1<i:g[R][n-1]=C
       if 0<=R+1<i:g[R+1][n]=C
       if 0<=n+1<i:g[R][n+1]=C
 return g

# [035] 1f642eb9.json
* image_within_image
* projection_unto_rectangle

In [ ]:
show_examples(load_examples(35)['train'])

In [ ]:
%%writefile task035.py
L=len
o=range
r=[[0,1],[0,-1],[1,0],[-1,0]]
def p(s):
 t,h=L(s),L(s[0])
 for A in o(t):
  for I in o(h):
   if s[A][I]==8:
    for(d,E)in r:
     if s[A+d][I+E]==0:
      for C in o(20):
       if 0<=A+d*C<t and 0<=I+E*C<h:
        H=s[A+d*C][I+E*C]
        if H>0:s[A][I]=H
 return s

# [036] 1f85a75f.json
* crop
* find_the_intruder

In [ ]:
show_examples(load_examples(36)['train'])

In [ ]:
%%writefile task036.py
def p(d,i=len,r=range):
	l='r';m='r';e,l,x=i(d),i(d[0]),{}
	for a in r(e):
		for z in r(l):
			e=d[a][z]
			if e in x:x[e][l]+=[a];x[e][m]+=[z]
			else:x[e]={l:[a],m:[z]}
	r=sorted([[i(x[e][l])*(max(x[e][m])-min(x[e][m])),e]for e in x if e>0])[0][1];d=[[r if x==r else 0for x in x]for x in d];x=x[r];d=[l[min(x[m]):max(x[m])+1]for l in d];d=d[min(x[l]):max(x[l])+1];return d

# [037] 1f876c06.json
* connect_the_dots
* diagonals

In [ ]:
show_examples(load_examples(37)['train'])

In [ ]:
%%writefile task037.py
def p(e,A=range):
 a,a=len(e),len(e[0]);C,e={},[A[:]for A in e]
 for n in A(a):
  for G in A(a):
   B=e[n][G]
   if B:C.setdefault(B,[]).append((n,G))
 for B in C:
  (D,t),(I,M)=C[B];a=1if I>D else-1;O=1if M>t else-1
  for T in A(abs(I-D)+1):e[D+T*a][t+T*O]=B
 return e

# [038] 1fad071e.json
* count_patterns
* associate_images_to_numbers

In [ ]:
show_examples(load_examples(38)['train'])

In [ ]:
%%writefile task038.py
r=range(8);p=lambda g:[([1]*sum(g[i][j]*g[i+1][j]*g[i][j+1]*g[i+1][j+1]==1 for j in r for i in r)+[0]*5)[:5]]

# [039] 2013d3e2.json
* pattern_deconstruction
* crop

In [ ]:
show_examples(load_examples(39)['train'])

In [ ]:
%%writefile task039.py
a=len
A=range
def p(c):
 r,J=a(c),a(c[0]);e=[]
 for E in A(r):
  for F in A(J):
   if c[E][F]>0:e.append([E,F])
 G=min([A[1]for A in e]);C=max([A[1]for A in e]);H=min([A[0]for A in e]);e=max([A[0]for A in e]);C=C-(C-G)//2;e=e-(e-H)//2;c=c[H:e];c=[A[G:C]for A in c];return c

# [040] 2204b7a8.json
* proximity_guessing
* recoloring

In [ ]:
show_examples(load_examples(40)['train'])

In [ ]:
%%writefile task040.py
def p(i):
 r=range;i=[a[:]for a in i];E=i[0][0]==i[0][9];e,G=(i[0][0],i[9][0])if E else(i[0][0],i[0][9]);I=next(a for n in i for a in n if a and a not in[e,G])
 for a in r(10):
  for n in r(10):
   if i[a][n]==I:H=(a,9-a)if E else(n,9-n);i[a][n]=e if H[0]<H[1]else G
 return i

# [041] 22168020.json
* pattern_expansion

In [ ]:
show_examples(load_examples(41)['train'])

In [ ]:
%%writefile task041.py
p=lambda j,a=0:[[x and(a:=(a==0)*x)or x or a for x in r]for r in j]

# [042] 22233c11.json
* pattern_expansion
* size_guessing

In [ ]:
show_examples(load_examples(42)['train'])

In [ ]:
%%writefile task042.py
def p(g):
 o=[r[:]for r in g];R,C=len(g),len(g[0]);G={(r,c)for r in range(R)for c in range(C)if g[r][c]==3};U=set()
 for m in[3,2,1]:
  for r in range(R):
   for c in range(C):
    for f in[0,1]:
     c1,c2,c0,c3=(c+m,c,c-m,c+2*m)if f else(c,c+m,c+2*m,c-m);E=set()
     for d in range(m):
      for e in range(m):
       if 0<=r+d<R and 0<=c1+e<C:E.add((r+d,c1+e))
       if 0<=r+d+m<R and 0<=c2+e<C:E.add((r+d+m,c2+e))
     if len(E)==2*m*m and E<=G and not E&U:
      U|=E
      for d in range(m):
       for e in range(m):
        if 0<=r+d-m<R and 0<=c0+e<C:o[r+d-m][c0+e]=8
        if 0<=r+d+2*m<R and 0<=c3+e<C:o[r+d+2*m][c3+e]=8
 return o

# [043] 2281f1f4.json
* direction_guessing
* draw_line_from_point
* pattern_intersection

In [ ]:
show_examples(load_examples(43)['train'])

In [ ]:
%%writefile task043.py
def p(d,n=enumerate):
 h=len(d)-1;E=len(d[0])-1
 for(B,F)in n(d):
  for(e,G)in n(F):
   if B>0and e<h:
    if d[B][E]==5and d[0][e]==5:d[B][e]=2
 return d

# [044] 228f6490.json
* pattern_moving
* loop_filling
* shape_guessing
* x_marks_the_spot

In [ ]:
show_examples(load_examples(44)['train'])

In [ ]:
%%writefile task044.py
def p(g):
 h=set();L=[]
 for N in range(100):
  C,N=N//10,N%10
  if(C,N)in h:continue
  T=g[C][N];M=[(C,N)];o={(C,N)};h|={M[0]}
  for(e,S)in M:
   for N in[(e+h,S+N)for(h,N)in[(0,1),(0,-1),(1,0),(-1,0)]]:
    if 0<=N[0]<10>N[1]>=0and N not in h and g[N[0]][N[1]]==T:h|={N};M+=[N];o|={N}
  L+=[(T,o)]
 T=[N for(h,N)in L if h==5];Q=[];J=[]
 for(h,N)in L:
  if h==0:
   H=lambda s,n:[min,max][n>1](h[n%2]for h in s)
   if any(all(H(N,h)>=H(C,h)if h<2else H(N,h)<=H(C,h)for h in range(4))for C in T):Q+=[N]
  elif h>0and h!=5:J+=[(h,N)]
 R=lambda s:frozenset((N-(h:=min(s))[0],C-h[1])for(N,C)in s);N={};l={}
 for e in Q:s=R(e);N[s]=N.get(s,[])+[e]
 for(h,e)in J:s=R(e);l[s]=l.get(s,[])+[(h,e)]
 o={h:sum(1for(N,C)in J if N==h)for(h,N)in J}
 for(s,a)in N.items():
  if s in l:
   T=l[s]
   if len(T)>1and o:T=[h for h in T if h[0]!=max(o,key=o.get)]or T
   for(V,(h,H))in zip(a,T):
    for(C,N)in H:g[C][N]=0
    for(C,N)in V:g[C][N]=h
 return g

# [045] 22eb0ac0.json
* connect_the_dots
* color_matching

In [ ]:
show_examples(load_examples(45)['train'])

In [ ]:
%%writefile task045.py
def p(g):
 for i in g:
  if i[0]==i[-1]:i[:]=i[:1]*len(i)
 return g

# [046] 234bbc79.json
* recoloring
* bring_patterns_close
* crop

In [ ]:
show_examples(load_examples(46)['train'])

In [ ]:
%%writefile task046.py
def p(e,E=enumerate):
 H=len(e[0]);H=[-1]+[H for(H,N)in E(zip(*e))if not any(N)]+[H];N=[[n[H[N]+1:H[N+1]]for n in e]for N in range(len(H)-1)if H[N]+1<H[N+1]];n=N[:1]
 for d in N[1:]:H=n[-1];H=next((N for(N,H)in E(H)if H and H[-1]==5),0);e=next((N for(N,H)in E(d)if H and H[0]==5),0);e=H-e;L=[[0]*len(d[0])]*3;n+=[(L+d+L)[3-e:6-e]]
 L=[next((H for N in H for H in N if H>0and H!=5),0)for H in N];M=[[[N if H==5else H for H in H]for H in H]for(H,N)in zip(n,L)];return[sum(H,[])for H in zip(*M)]

# [047] 23581191.json
* draw_line_from_point
* pattern_intersection

In [ ]:
show_examples(load_examples(47)['train'])

In [ ]:
%%writefile task047.py
def p(n):
 l=range;r=[[0]*9for r in l(9)];h=[(r,h,n[r][h])for r in l(9)for h in l(9)if n[r][h]]
 for(l,a,e)in h:
  for n in range(9):r[l][n]=r[n][a]=e
 r[h[0][0]][h[1][1]]=r[h[1][0]][h[0][1]]=2;return r

# [048] 239be575.json
* detect_connectedness
* associate_images_to_bools

In [ ]:
show_examples(load_examples(48)['train'])

In [ ]:
%%writefile task048.py
def f(j,A,d):
 global n;l.append((j,A))
 for B in C(j-1,j+2):
  for e in C(A-1,A+2):
   if(B,e)in l:continue
   l.append((B,e))
   if B<0 or B>=J or e<0 or e>=a or(B,e)in[(H,L),(H+1,L),(H,L+1),(H+1,L+1)]:continue
   if d[B][e]==2:n=8
   if d[B][e]==8:f(B,e,d)
def p(d):
 global n,l,H,L,J,a,C;n,l,J,a,C,e=0,[],len(d),len(d[0]),range,enumerate
 for(H,E)in e(d):
  for(L,E)in e(E):
   if E==2:
    for A in C(H-1,H+3):
     for B in C(L-1,L+3):
      if A>=0and A<J and B>=0and B<a and d[A][B]==8:f(A,B,d)
    return[[n]]

# [049] 23b5c85d.json
* measure_area
* take_minimum
* crop

In [ ]:
show_examples(load_examples(49)['train'])

In [ ]:
%%writefile task049.py
from collections import*
j=len
s=range
def p(c):
 i=[a for s in c for a in s];t=Counter(i).most_common();t=[C for C in t if C[0]!=0][-1][0];W,N=j(c),j(c[0]);J=[]
 for a in s(W):
  for C in s(N):
   if c[a][C]==t:J.append([a,C])
 e=min([s[1]for s in J]);h=max([s[1]for s in J]);O=min([s[0]for s in J]);L=max([s[0]for s in J]);c=c[O:L+1];c=[s[e:h+1]for s in c];return c

# [050] 253bf280.json
* connect_the_dots
* direction_guessing

In [ ]:
show_examples(load_examples(50)['train'])

In [ ]:
%%writefile task050.py
def p(j):
	A=range;c=[e[:]for e in j];E,k=len(j),len(j[0]);W=[(e,l)for e in A(E)for l in A(k)if j[e][l]==8]
	for(l,J)in W:
		for(a,C)in[(0,1),(1,0),(0,-1),(-1,0)]:
			e=1
			while 0<=l+e*a<E and 0<=J+e*C<k:
				if j[l+e*a][J+e*C]==8:
					for K in A(1,e):c[l+K*a][J+K*C]=3
					break
				e+=1
	return c

# [051] 25d487eb.json
* draw_line_from_point
* direction_guessing
* color_guessing

In [ ]:
show_examples(load_examples(51)['train'])

In [ ]:
%%writefile task051.py
def p(p):
	t=range;p=[m[:]for m in p];g,F=len(p),len(p[0]);n={}
	for m in t(g):
		for i in t(F):
			if p[m][i]:n[p[m][i]]=n.get(p[m][i],0)+1
	m,i,a=next((m,i,p[m][i])for m in t(g)for i in t(F)if p[m][i]and n[p[m][i]]==1)
	for(G,r)in[(0,1),(1,0),(0,-1),(-1,0)]:
		d,K=m+G,i+r
		if(d<0)|(d>=g)|(K<0)|(K>=F)|(p[d][K]==0):
			e=1
			while(0<=m-e*G<g)&(0<=i-e*r<F):
				if p[m-e*G][i-e*r]==0:p[m-e*G][i-e*r]=a
				e+=1
	return p

# [052] 25d8a9c8.json
* detect_hor_lines
* recoloring
* remove_noise

In [ ]:
show_examples(load_examples(52)['train'])

In [ ]:
%%writefile task052.py
p=lambda j:[[(r.count(r[0])>2)*5]*3for r in j]

# [053] 25ff71a9.json
* pattern_moving

In [ ]:
show_examples(load_examples(53)['train'])

In [ ]:
%%writefile task053.py
p=lambda g:g[-1:]+g[:-1]

# [054] 264363fd.json
* pattern_repetition
* pattern_juxtaposition
* draw_line_from_point

In [ ]:
show_examples(load_examples(54)['train'])

In [ ]:
%%writefile task054.py
def p(g,r=range):
 t=[A[:]for A in g];K=g[0][0];M,n=set(),[]
 for A in r(30):
  for D in r(30):
   if g[A][D]!=K and(A,D)not in M:
    N,P=[],[(A,D)];M.add((A,D))
    while P:
     E,L=P.pop(0);N.append((E,L))
     for F in[-1,0,1]:
      for G in[-1,0,1]:
       if F or G:
        A,B=E+F,L+G
        if 0<=A<30and 0<=B<30and g[A][B]!=K and(A,B)not in M:M.add((A,B));P.append((A,B))
    n.append(N)
 O=min((A for A in n if len(A)>1),key=len);A,D=zip(*O);E,L=(min(A)+max(A))//2,(min(D)+max(D))//2;U=g[E][L];a={(A-E,B-L):g[A][B]for(A,B)in O};V=set(O);E=[(A,B)for A in r(30)for B in r(30)if g[A][B]==U and(A,B)not in V]
 for(A,B)in E:
  for((F,G),S)in a.items():
   if(F==0 or G==0)and(F*2,G*2)in a:
    for T in r(1,30):
     H,N=A+F*T,B+G*T
     if not(0<=H<30and 0<=N<30and t[H][N]!=K):break
     t[H][N]=S
   else:
    H,N=A+F,B+G
    if 0<=H<30and 0<=N<30and t[H][N]!=K:t[H][N]=S
 for(E,N)in O:t[E][N]=K
 return t

# [055] 272f95fa.json
* detect_grid
* mimic_pattern
* grid_coloring

In [ ]:
show_examples(load_examples(55)['train'])

In [ ]:
%%writefile task055.py
def p(o,A=range):
 a,J=len(o),len(o[0]);D=[A[:]for A in o];l,F=[A for A in A(a)if all(A==8for A in o[A])];d,e=[r for r in A(J)if all(o[A][r]==8for A in A(a))]
 for r in A(a):
  for t in A(J):
   if not D[r][t]:
    if r<l and d<t<e:D[r][t]=2
    elif l<r<F and t<d:D[r][t]=4
    elif l<r<F and d<t<e:D[r][t]=6
    elif l<r<F and t>e:D[r][t]=3
    elif r>F and d<t<e:D[r][t]=1
 return D

# [056] 27a28665.json
* associate_colors_to_patterns
* take_negative
* associate_images_to_patterns

In [ ]:
show_examples(load_examples(56)['train'])

In [ ]:
%%writefile task056.py
p=lambda j:(x:=j[0],[[{6:1,5:2,3:3,2:6}[(x[0]!=0)*4+(x[1]!=0)*2+(x[2]!=0)]]])[1]

# [057] 28bf18c6.json
* crop
* pattern_repetition

In [ ]:
show_examples(load_examples(57)['train'])

In [ ]:
%%writefile task057.py
p=lambda j:(A:={i for r in j for i,x in enumerate(r)if x>0},[r[min(A):max(A)+1]*2for r in j if max(r)>0])[1]

# [058] 28e73c20.json
* ex_nihilo
* mimic_pattern

In [ ]:
show_examples(load_examples(58)['train'])

In [ ]:
%%writefile task058.py
def p(i):
 A=range;o=len(i);H=[[0]*o for i in A(o)];l,i=0,0;a=[(0,1),(1,0),(0,-1),(-1,0)]
 for h in A(o):
  H[l][i]=3
  if h<o-1:i+=1
 o=o-1;D=1
 while o>0:
  for h in A(2):
   if o>0:
    l,i=l+a[D][0],i+a[D][1]
    for h in A(o):
     H[l][i]=3
     if h<o-1:l,i=l+a[D][0],i+a[D][1]
    D=(D+1)%4
  o-=2
 return H

# [059] 29623171.json
* detect_grid
* separate_images
* count_tiles
* take_maximum
* grid_coloring

In [ ]:
show_examples(load_examples(59)['train'])

In [ ]:
%%writefile task059.py
def p(n,q=enumerate,o=range(11)):
 c=0;r=[[0if(r+1)%4>0and(n+1)%4>0else 5for r in o]for n in o];o={n+r:0for n in'012'for r in'012'}
 for e,n in q(n):
  for t,n in q(n):
   if n>0and n!=5:c=int(n);o[str(e//4)+str(t//4)]+=1
 u=max(o.values())
 for e,n in q(r):
  for t,n in q(n):
   if n==0and o[str(e//4)+str(t//4)]==u:r[e][t]=c
 return r

# [060] 29c11459.json
* draw_line_from_point
* count_tiles

In [ ]:
show_examples(load_examples(60)['train'])

In [ ]:
%%writefile task060.py
def p(j):
 for w in j:
  if w[0]:w[:]=w[:1]*5+[5]+w[-1:]*5
 return j

# [061] 29ec7d0e.json
* image_filling
* pattern_expansion
* detect_grid
* pattern_repetition

In [ ]:
show_examples(load_examples(61)['train'])

In [ ]:
%%writefile task061.py
def p(t,z=enumerate):
	f=range;e=len(t);e=len(t[0]);k=lambda i,f:i==f or i*f<1;g=next((i for i in f(1,e)if all(k(d,e)for f in t for(d,e)in zip(f,f[i:]))),e);a=next((i for i in f(1,e)if all(k(d,e)for(i,f)in zip(t,t[i:])for(d,e)in zip(i,f))),e);C={}
	for(e,i)in z(t):
		for(f,d)in z(i):
			if d:C[e%a,f%g]=d
	for(e,i)in z(t):
		for(f,d)in z(i):
			if not d:i[f]=C[e%a,f%g]
	return t

# [062] 2bcee788.json
* pattern_reflection
* direction_guessing
* image_filling
* background_filling

In [ ]:
show_examples(load_examples(62)['train'])

In [ ]:
%%writefile task062.py
r=len
o=range
def p(g):
 a=sum(g,[]).count(2)
 for i in range(4):
  T,i=r(g),r(g[0]);g=list(map(list,zip(*g[::-1])))
  for i in o(T):
   if g[i].count(2)==a and sum(g[i])==2*a:
    for e in range(max(i,T-i)):
     if 0<=i+e<T and sum(g[i+e])>0and 2not in g[i+e]:
      if 0<=i-e+1<T and 0<=i+e<T:g[i-e+1]=g[i+e][:]
     elif 0<=i-e-1<T and 0<=i+e<T:g[i+e]=g[i-e-1][:]
 g=[[3if i==0else i for i in i]for i in g];return g

# [063] 2bee17df.json
* draw_line_from_border
* count_tiles
* take_maximum

In [ ]:
show_examples(load_examples(63)['train'])

In [ ]:
%%writefile task063.py
def p(e):
 D=range;d=len(e);n=[d[:]for d in e]
 for l in range(d):
  if e[1][l]==0and e[d-2][l]==0and sum(e[d][l]for d in D(1,d-1))==0:
   for C in D(1,d-1):n[C][l]=3
 for C in range(d):
  if e[C][1]==0and e[C][d-2]==0and sum(e[C][d]for d in D(1,d-1))==0:
   for l in D(1,d-1):
    if n[C][l]==0:n[C][l]=3
 return n

# [064] 2c608aff.json
* draw_line_from_point
* projection_unto_rectangle

In [ ]:
show_examples(load_examples(64)['train'])

In [ ]:
%%writefile task064.py
a=len
e=range
L=[[0,1],[0,-1],[1,0],[-1,0]]
def p(g):
 D,E=a(g),a(g[0]);J=sum(g,[]);C=sorted([[J.count(A),A]for A in set(J)])
 for A in e(D):
  for s in e(E):
   if g[A][s]==C[1][1]:
    for(F,G)in L:
     if 0<=A+F<D and 0<=s+G<E and g[A+F][s+G]==C[2][1]:
      for H in e(20):
       if 0<=A+F*H<D and 0<=s+G*H<E:
        R=g[A+F*H][s+G*H]
        if R==C[0][1]:g[A][s]=0
 for M in range(4):
  g=list(map(list,zip(*g[::-1])));D,E=a(g),a(g[0])
  for A in e(D):
   if g[A].count(0)>0:
    I=0
    for s in e(E):
     if g[A][s]==C[0][1]and g[A].count(0)>0and g[A].index(0)>s:I=1
     if I:
      if g[A][s]==C[2][1]:g[A][s]=C[0][1]
      elif g[A][s]==0:I=0
 g=[[C[1][1]if A==0else A for A in A]for A in g];return g

# [065] 2dc579da.json
* detect_grid
* find_the_intruder
* crop

In [ ]:
show_examples(load_examples(65)['train'])

In [ ]:
%%writefile task065.py
def p(u):
	s=range;r=(len(u)-1)//2
	if r==1:
		d=[u[0][0],u[0][2],u[2][0],u[2][2]]
		for m in d:
			if d.count(m)==1:return[[m]]
	for(t,h)in[(0,0),(0,r+1),(r+1,0),(r+1,r+1)]:
		l=[[u[t+m][h+r]for r in s(r)]for m in s(r)];m=[l[m][d]for m in s(r)for d in s(r)]
		if len(set(m))>1:return l


# [066] 2dd70a9a.json
* draw_line_from_point
* direction_guessing
* maze

In [ ]:
show_examples(load_examples(66)['train'])

In [ ]:
%%writefile task066.py
import copy
E=enumerate
A=range
S=sorted
N=None
G,R,B=3,2,0
d=copy.deepcopy
def ib(g,r,c):return 0<=r<len(g)and 0<=c<len(g[0])
def n(r,c):return[(r+1,c),(r-1,c),(r,c+1),(r,c-1)]
def ad(a,b):return a[0]+b[0],a[1]+b[1]
def sb(a,b):return a[0]-b[0],a[1]-b[1]
def nm(d):return tuple(map(lambda x:(x>0)-(x<0),d))
def rl(d):return-d[1],d[0]
def rr(d):return d[1],-d[0]
def m(a,b):return abs(a[0]-b[0])+abs(a[1]-b[1])
def fp(g,v):return[(A,C)for(A,B)in E(g)for(C,D)in E(B)if D==v]
def c4c(ps):
 A,E=set(ps),[]
 while A:
  F=A.pop();D=[F];G=[F]
  while D:
   O,I=D.pop()
   for(B,C)in n(O,I):
    if(B,C)in A:A.remove((B,C));D.append((B,C));G.append((B,C))
  E.append(G)
 return E
def cg(gs,p):return min(m(p,A)for A in gs)if gs else 1e9
def l(v):return v in(B,G)
def rs(g,s,f,rs):
 Q,S=len(g),len(g[0]);C=d(g);T=set();J=0;A=s;F=cg(rs,A);K=F;L=0
 while J<Q*S*10:
  J+=1;D=ad(A,f)
  if ib(g,*D)and C[D[0]][D[1]]==R:break
  if not ib(g,*D)or not l(C[D[0]][D[1]]):
   L+=1;h,U=rl(f),rr(f);O=[]
   for N in(h,U):
    E=ad(A,N)
    if ib(g,*E)and l(C[E[0]][E[1]]):O.append((cg(rs,E),N,E))
   if O:O.sort(key=lambda x:(x[0],0if x[1]==h else 1));e,f,A=O[0]
   else:
    s=-f[0],-f[1];I=ad(A,s)
    if ib(g,*I)and l(C[I[0]][I[1]]):f,A=s,I
  else:A=D
  if C[A[0]][A[1]]==B:C[A[0]][A[1]]=G
  V=A,f;T.add(V);n=cg(rs,A);F=min(F,n);K=n
 return C,J,F,K,L
def ds(g):B,O=fp(g,R),fp(g,G);A=min(c4c(O),key=lambda x:(len(x),cg(B,(round(sum(A for(A,B)in x)/len(x)),round(sum(A for(B,A)in x)/len(x))))));I=set(A);C=next((((A,B),(C,D))for(A,B)in A for(C,D)in n(A,B)if(C,D)in I),N);C=C or max(((B,C)for B in A for C in A),key=lambda x:m(*x));E,F=C;D=nm(sb(F,E));J=[rs(g,F,D,B),rs(g,E,(-D[0],-D[1]),B)];return S(J,key=lambda x:(x[3],x[4],x[2],x[1]))[0][0]
def br(g):
 Q,T=fp(g,R),fp(g,G);e=lambda ps:S(ps)[0][1]==S(ps)[1][1]and abs(S(ps)[0][0]-S(ps)[1][0])==1;(I,F),(J,U)=S(T);(K,O),(L,U)=S(Q)
 if{I,J}!={K,L}:return
 h,s=S([F,O]);E=N
 for C in A(min(I,K)+1):
  if all(g[C][A]in(B,G,R)for A in A(h,s+1)):E=C;break
 if E in(N,0):return
 D=d(g);V=max(J,L)
 for C in A(E,V+1):
  if D[C][F]==B:D[C][F]=G
  if D[C][O]==B:D[C][O]=G
 for n in A(h,s+1):
  if D[E][n]==B:D[E][n]=G
 return D
def p(g):return br(g)or ds(g)

# [067] 2dee498d.json
* detect_repetition
* crop
* divide_by_n

In [ ]:
show_examples(load_examples(67)['train'])

In [ ]:
%%writefile task067.py
p=lambda g:[a[:len(g)]for a in g]

# [068] 31aa019c.json
* find_the_intruder
* remove_noise
* contouring

In [ ]:
show_examples(load_examples(68)['train'])

In [ ]:
%%writefile task068.py
def p(j):
 i={};a=range
 for R in a(10):
  for o in a(10):
   if j[R][o]:i[j[R][o]]=i.get(j[R][o],0)+1
 F=next(i for(i,a)in i.items()if a==1);R,i=next((i,R)for i in a(10)for R in a(10)if j[i][R]==F);E=[[0]*10for i in a(10)];E[R][i]=F
 for H in[-1,0,1]:
  for t in[-1,0,1]:
   if H or t:
    a,K=R+H,i+t
    if 0<=a<10and 0<=K<10:E[a][K]=2
 return E

# [069] 321b1fc6.json
* pattern_repetition
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(69)['train'])

In [ ]:
%%writefile task069.py
def p(g,E=enumerate):
 A=[]
 for(B,F)in E(g):
  for(C,D)in E(F):
   if D not in[0,8]:A+=[[B,C,D]];g[B][C]=0
 H=A[0][:];A=[[A[0]-H[0],A[1]-H[1],A[2]]for A in A]
 for(B,F)in E(g):
  for(C,D)in E(F):
   if D==8:
    g[B][C]=H[2]
    for H in A:g[B+H[0]][C+H[1]]=H[2]
 return g

# [070] 32597951.json
* find_the_intruder
* recoloring

In [ ]:
show_examples(load_examples(70)['train'])

In [ ]:
%%writefile task070.py
def p(i):
 o=range;n=[e[:]for e in i];e=[(e,n)for e in o(len(i))for n in o(len(i[0]))if i[e][n]==8]
 if e:
  s,G=min(e for(e,o)in e),max(e for(e,o)in e);I,e=min(e for(o,e)in e),max(e for(o,e)in e)
  for s in o(s,G+1):
   for e in o(I,e+1):
    if i[s][e]==1:n[s][e]=3
 return n

# [071] 3345333e.json
* pattern_completion
* pattern_reflection
* remove_noise

In [ ]:
show_examples(load_examples(71)['train'])

In [ ]:
%%writefile task071.py
def p(I):
 from collections import defaultdict as e
 def R(S):
  if not S:return False
  S,a=zip(*S);return len(S)==(max(S)-min(S)+1)*(max(a)-min(a)+1)
 d,i=len(I),len(I[0]);S=e(list)
 for a in range(d):
  for l in range(i):
   if I[a][l]:S[I[a][l]].append((a,l))
 if not S:return[[0]*i for S in range(d)]
 t=max([S for(S,a)in S.items()if R(a)],key=lambda e:len(S[e]),default=None);F=max([a for a in S if a!=t and not R(S[a])]or[S for S in S if S!=t],key=lambda e:len(S[e]));G,K=S[F],set(S[F]);N=set(S[t])if t else set();N=max(range(2*i-1),key=lambda t:sum(1for(a,l)in G if 0<=(S:=t-l)<i and((a,S)in K or(a,S)in N)),default=min(S for(a,S)in G)+max(S for(a,S)in G));I=[[0]*i for S in range(d)]
 for(a,l)in G:I[a][l]=F
 if t:
  for(a,l)in S[t]:
   if 0<=(H:=N-l)<i and(a,H)in K:I[a][l]=F
 return I

# [072] 3428a4f5.json
* detect_wall
* separate_images
* pattern_differences

In [ ]:
show_examples(load_examples(72)['train'])

In [ ]:
%%writefile task072.py
p=lambda g,r=range:[[3*(g[i][j]+g[i+7][j]==2)for j in r(5)]for i in r(6)]

# [073] 3618c87e.json
* gravity

In [ ]:
show_examples(load_examples(73)['train'])

In [ ]:
%%writefile task073.py
def p(g):
 for i,x in enumerate(g[2]):
  if x:g[2][i],g[-1][i]=0,1
 return g

# [074] 3631a71a.json
* image_filling
* pattern_expansion
* pattern_rotation

In [ ]:
show_examples(load_examples(74)['train'])

In [ ]:
%%writefile task074.py
def p(g,e=enumerate,s=range):
 i=min(len(g),len(g[0]))if g and g[0]else 0
 def r(D):
  if i<=2:return tuple(tuple(0if i==9else i for i in i)for i in D)
  l=[[0if i==9else i for i in i]for i in D];h=list(map(list,zip(*l)));D=[[max(l[D][i],h[D][i])for i in s(i)]for D in s(i)];E=[l[2:i][::-1]for l in D[1:i]];r=[i[:]for i in D]
  for(a,l)in e(E,1):
   for(K,h)in e(l,2):
    if h:r[a][K]=h
  return tuple(map(tuple,r))
 l=tuple(tuple(i)for i in g)
 for h in s(256):
  D=r(l)
  if D==l:break
  l=D
 return[list(i)for i in l]

# [075] 363442ee.json
* detect_wall
* pattern_repetition
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(75)['train'])

In [ ]:
%%writefile task075.py
def p(i):
 e=range;d=[e[:]for e in i];G=[[i[A][e]for e in e(3)]for A in e(3)]
 for A in e(9):
  for n in e(4,13):
   if i[A][n]==1:
    for D in e(-1,2):
     for r in e(-1,2):
      if 0<=A+D<9and 4<=n+r<13:d[A+D][n+r]=G[D+1][r+1]
 return d

# [076] 36d67576.json
* pattern_repetition
* pattern_juxtaposition
* pattern_reflection
* pattern_rotation

In [ ]:
show_examples(load_examples(76)['train'])

In [ ]:
%%writefile task076.py
def p(G):
 T=set();L=[]
 S={(r,c)for r,R in enumerate(G)for c,v in enumerate(R)if v}
 while S:
  Q=[S.pop()];A=0;N=[]
  while Q:
   x,y=Q.pop();v=G[x][y];N+=[(x,y,v)]
   if v==2:A=x,y
   for i in range(9):
    if(k:=(x+i//3-1,y+i%3-1))in S:S.remove(k);Q+=[k]
  a,b=A;P={(r-a,c-b,v)for r,c,v in N}
  if any(v%2 for*_,v in N):T=P
  else:L+=[(A,P)]
 E=[x for x in T if x[2]%2];C=T-{*E}
 Z=(1,-1)
 H=lambda t,r,c:([r,c][t&1]*Z[t>>1&1],[c,r][t&1]*Z[t>>2&1])
 for(a,b),U in L:
  for t in range(8):
   if{(*H(t,r,c),v)for r,c,v in C}==U:
    for r,c,v in E:
     R,K=H(t,r,c);G[R+a][K+b]=v
    break
 return G

# [077] 36fdfd69.json
* recoloring
* rectangle_guessing

In [ ]:
show_examples(load_examples(77)['train'])

In [ ]:
%%writefile task077.py
def p(g,f=range):
 N=[A[:]for A in g];A,J=len(N),len(N[0]);B=[]
 for W in N:
  J=[]
  for X in W:J+=[X]*2
  B.extend([J[:],J[:]])
 K,L=len(B),len(B[0]);N=[[False]*L for A in f(K)];D=[]
 for C in f(K):
  for A in f(L):
   if B[C][A]!=2or N[C][A]:continue
   O=[(C,A)];N[C][A]=True;H=[]
   while O:
    E,Q=O.pop();H.append((E,Q))
    for R in(-1,0,1):
     for S in(-1,0,1):
      if R==S==0:continue
      F,G=E+R,Q+S
      if 0<=F<K and 0<=G<L and not N[F][G]and B[F][G]==2:N[F][G]=True;O.append((F,G))
   D.append(H)
 Y={(A,C)for A in f(K)for C in f(L)if B[A][C]==2}
 def T(D,q,u,d):
  for A in f(D,u+1):
   for C in f(q,d+1):B[A][C]=4
 for E in D:H=[A for(A,B)in E];I=[A for(B,A)in E];T(min(H),min(I),max(H),max(I))
 def Z(A,B):
  C=10**9
  for(E,F)in A:
   for(G,H)in B:
    D=abs(E-G)+abs(F-H)
    if D<C:C=D
    if C==0:return 0
  return C
 e=len(D)
 for C in f(e):
  for A in f(e):
   if Z(D[C],D[A])<5:V=D[C]+D[A];H=[A for(A,B)in V];I=[A for(B,A)in V];T(min(H),min(I),max(H),max(I))
 for(C,A)in Y:B[C][A]=2
 return[A[::2]for A in B[::2]]

# [078] 3906de3d.json
* gravity

In [ ]:
show_examples(load_examples(78)['train'])

In [ ]:
%%writefile task078.py
p=lambda g:[*zip(*[b'\1'.join(bytes(sorted(s)[::-1])for s in bytes(c).split(b'\1'))for c in zip(*g)])]

# [079] 39a8645d.json
* count_patterns
* take_maximum
* crop

In [ ]:
show_examples(load_examples(79)['train'])

In [ ]:
%%writefile task079.py
def p(n):
 l=[];t=len(n)-2
 for F in range(t):
  for r in range(t):
   I=tuple(tuple(n[F+l][r:r+3])for l in(0,1,2))
   for G in{*sum(I,())}-{0}:
    H=tuple(tuple(l*(l==G)for l in l)for l in I)
    if min(map(sum,H))and min(map(sum,zip(*H))):l+=H,
 return[list(l)for l in max(l,key=l.count)]

# [080] 39e1d7f9.json
* detect_grid
* pattern_repetition
* grid_coloring

In [ ]:
show_examples(load_examples(80)['train'])

In [ ]:
%%writefile task080.py
def p(o):
 o=[list(i)for i in o];s=[]
 for i in[o,[*zip(*o)]]:e=len(i);k=[-1]+[k for k in range(e)if len({*i[k]})<2]+[e];s.append([(a+1,m)for a,m in zip(k,k[1:])if m>a+1])
 m,m=s;g=len(m);t=range(g);d=[[o[i[0]][k[0]]for k in m]for i in m];i=sum(d,[]);l=max(i,key=i.count);e=p=0;f=[-1,0,1]
 for i in range(1,g-1):
  for k in range(1,g-1):
   if(s:=d[i][k])!=l:
    if d[i][k-1]==d[i+1][k]==d[i-1][k]!=l:e=[[d[i+n][k+y]for y in f]for n in f];p=s;break
  if e:break
 for i in t:
  for k in t:
   if d[i][k]==p:
    for n in f:
     if 0<=(l:=i+n)<g:
      for y in f:
       if 0<=(s:=k+y)<g:d[l][s]=e[n+1][y+1]
 for k,(y,t)in zip(d,m):
  for s,(i,n)in zip(k,m):
   for k in range(y,t):o[k][i:n]=[s]*(n-i)
 return o

# [081] 3aa6fb7a.json
* pattern_completion
* pattern_rotation

In [ ]:
show_examples(load_examples(81)['train'])

In [ ]:
%%writefile task081.py
from collections import*
def p(h,A=range):
 c=[[8if n==8else 0for n in e]for e in h]
 for d in A(len(h)-1):
  for k in A(len(h[0])-1):
   e=[h[d][k:k+2],h[d+1][k:k+2]];l=[t for e in e for t in e];n=Counter(l).most_common(1)
   if n[0][1]==3and n[0][0]!=0:
    for i in A(d,d+2):
     for e in A(k,k+2):
      if c[i][e]==0:c[i][e]=1
 return c

# [082] 3ac3eb23.json
* draw_pattern_from_point
* pattern_repetition

In [ ]:
show_examples(load_examples(82)['train'])

In [ ]:
%%writefile task082.py
def p(g):
 for i,c in enumerate(g[0][1:-1],1):
  if c:g[1][i-1:i+2:2]=c,c
 g[2:]=g[:-2];g[2:]=g[:-2];return g

# [083] 3af2c5a8.json
* image_repetition
* image_reflection
* image_rotation

In [ ]:
show_examples(load_examples(83)['train'])

In [ ]:
%%writefile task083.py
p=lambda g:[i+i[::-1]for i in g+g[::-1]]

# [084] 3bd67248.json
* draw_line_from_border
* diagonals
* pattern_repetition

In [ ]:
show_examples(load_examples(84)['train'])

In [ ]:
%%writefile task084.py
def p(r):
 for A in range(1,len(r[0])):r[-1][A]=4;r[A-1][-A]=2
 return r

# [085] 3bdb4ada.json
* recoloring
* pattern_repetition
* holes

In [ ]:
show_examples(load_examples(85)['train'])

In [ ]:
%%writefile task085.py
def p(g,V=range):
 r=[d[:]for d in g]
 v=set()
 for i in V(len(g)-2):
  for j in V(len(g[0])):
   if g[i][j]and(i,j)not in v:
    c=g[i][j]
    if all(g[i+k][j]==c for k in V(3)):
     a=j
     while a<len(g[0])and all(g[i+k][a]==c for k in V(3)):
      for k in V(3):v.add((i+k,a))
      a+=1
     for x in V(j,a):
      if(x-j)%2==1:r[i+1][x]=0
 return r

# [086] 3befdf3e.json
* take_negative
* pattern_expansion

In [ ]:
show_examples(load_examples(86)['train'])

In [ ]:
%%writefile task086.py
def p(s):
 from collections import deque,Counter as V;C,Q=range,len;I,l=Q(s),Q(s[0]);K=[[0]*l for N in C(I)];L=[[0]*l for N in C(I)]
 for M in C(I):
  for N in C(l):
   if s[M][N]and not L[M][N]:
    P=deque([(M,N)]);L[M][N]=1;R=[];S=V()
    while P:
     N,B=P.popleft();R.append((N,B));S[s[N][B]]+=1
     for(W,X)in((1,0),(-1,0),(0,1),(0,-1)):
      G,D=N+W,B+X
      if 0<=G<I and 0<=D<l and s[G][D]and not L[G][D]:L[G][D]=1;P.append((G,D))
    T,U=[N for(N,B)in S.most_common(2)];O=[(N,B)for(N,B)in R if s[N][B]==U];E,F,D=min(N for(N,B)in O),min(N for(B,N)in O),max(N for(N,B)in O)-min(N for(N,B)in O)+1
    for N in C(E-D-1,E+2*D+1):
     for B in C(F-D-1,F+2*D+1):
      if(N<E-1or N>E+D)and(B<F-1or B>F+D):continue
      K[N][B]=T
    for N in C(E-1,E+D+1):
     for B in C(F-1,F+D+1):K[N][B]=U
    for N in C(E,E+D):
     for B in C(F,F+D):K[N][B]=T
 return K

# [087] 3c9b0459.json
* image_rotation

In [ ]:
show_examples(load_examples(87)['train'])

In [ ]:
%%writefile task087.py
p=lambda j:[r[::-1]for r in j[::-1]]

# [088] 3de23699.json
* take_negative
* crop
* rectangle_guessing

In [ ]:
show_examples(load_examples(88)['train'])

In [ ]:
%%writefile task088.py
from collections import*
b=len
i=range
def p(d):
 a=[i for i in d for i in i];k=Counter(a).most_common();k=[C for C in k if C[1]==4][0][0];r,p=b(d),b(d[0]);f=[]
 for a in i(r):
  for C in i(p):
   if d[a][C]==k:f.append([a,C])
 e=min([i[1]for i in f]);u=max([i[1]for i in f]);a=min([i[0]for i in f]);j=max([i[0]for i in f]);d=d[a+1:j];d=[i[e+1:u]for i in d];r,p=b(d),b(d[0])
 for a in i(r):
  for C in i(p):
   if d[a][C]>0:d[a][C]=k
 return d

# [089] 3e980e27.json
* pattern_repetition
* pattern_juxtaposition
* direction_guessing
* pattern_reflection

In [ ]:
show_examples(load_examples(89)['train'])

In [ ]:
%%writefile task089.py
def p(g,i=range):
 D,o=len(g),len(g[0]);K=[r[:]for r in g];F=[[0]*o for r in i(D)];L=[]
 for G in i(D):
  for l in i(o):
   if F[G][l]or g[G][l]==0:continue
   I=[(G,l)];C=[]
   while I:
    r,S=I.pop()
    if r<0 or r>=D or S<0 or S>=o or F[r][S]or g[r][S]==0:continue
    F[r][S]=1;C.append((r,S,g[r][S]));I.extend([(r+C,S+G)for(C,G)in[(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]if 0<=r+C<D and 0<=S+G<o and not F[r+C][S+G]and g[r+C][S+G]!=0])
   C and L.append(C)
 M,h,O,P=[],[],[],[]
 for C in L:
  L={S for(r,r,S)in C}
  if len(C)==1:
   R,r,S=C[0]
   if S==2:O.append((R,r))
   elif S==3:P.append((R,r))
  else:
   if 2in L:M.append(C)
   if 3in L:h.append(C)
 def T(k,v,T,f=0):
  S=next(((r,S)for(r,S,C)in k if C==v),None)
  if not S:return
  G,l=S
  for(I,S)in T:
   for(L,M,h)in k:
    O,r=L-G,M-l
    if f:r=-r
    C,F=I+O,S+r
    if 0<=C<D and 0<=F<o:K[C][F]=h
 for S in M:T(S,2,O,1)
 for S in h:T(S,3,P)
 return K

# [090] 3eda0437.json
* rectangle_guessing
* recoloring
* measure_area
* take_maximum

In [ ]:
show_examples(load_examples(90)['train'])

In [ ]:
%%writefile task090.py
def p(b):
	l=[o[:]for o in b];x,e=len(l),len(l[0]);F=None;b=-1
	for o in range(2,min(9,x)+1):
		for d in range(2,min(9,e)+1):
			for f in range(0,x-o+1):
				for a in range(0,e-d+1):
					c=True
					for q in range(f,f+o):
						i=l[q]
						if any(i[o]!=0 for o in range(a,a+d)):c=False;break
					if c and o*d>b:b=o*d;F=f,a,o,d
	if F:
		f,a,o,d=F
		for q in range(f,f+o):
			for o in range(a,a+d):l[q][o]=6
	return l

# [091] 3f7978a0.json
* crop
* rectangle_guessing
* find_the_intruder

In [ ]:
show_examples(load_examples(91)['train'])

In [ ]:
%%writefile task091.py
a=enumerate;p=lambda g:[l[t:E+1]for(e,t)in[next((e,a.index(5))for(e,a)in a(g)if 5in a)]for(s,E)in[next((~e+len(g),~a[::-1].index(5)+len(a))for(e,a)in a(g[::-1])if 5in a)]for l in g[max(0,e-1):s+2]]

# [092] 40853293.json
* connect_the_dots

In [ ]:
show_examples(load_examples(92)['train'])

In [ ]:
%%writefile task092.py
def p(g,o=len,R=range):
 t=None;H,t=o(g),o(g[0]);e=[s[:]for s in g]
 for s in R(H):
  S=D=t
  for i in R(t):
   if g[s][i]:
    if S is not t and g[s][i]==D:
     for o in R(S+1,i):e[s][o]=D
    S=i;D=g[s][i]
 for i in R(t):
  S=D=t
  for s in R(H):
   if g[s][i]:
    if S is not t and g[s][i]==D:
     for o in R(S+1,s):e[o][i]=D
    S=s;D=g[s][i]
 return e

# [093] 4093f84a.json
* gravity
* recoloring
* projection_unto_rectangle

In [ ]:
show_examples(load_examples(93)['train'])

In [ ]:
%%writefile task093.py
def p(n):
 o=0
 if n[0].count(5)==0:n=list(map(list,zip(*n[::-1])));o=3
 C=n[0].count(5);r=[e[:]for e in n];n=[[5if e==5else 0for e in e]for e in n];B=n[0].index(5)
 for e in range(len(n)):d=sum([1for e in r[e][:B]if e>0]);a=sum([1for e in r[e][B+C:]if e>0]);n[e]=n[e][:B-d]+[5]*(C+d+a)+n[e][B+C+a:]
 for r in range(o):n=list(map(list,zip(*n[::-1])))
 return n

# [094] 41e4d17e.json
* draw_line_from_point
* pattern_repetition

In [ ]:
show_examples(load_examples(94)['train'])

In [ ]:
%%writefile task094.py
R=len
r=range
def p(i):
 E,F=[],[];T,N=R(i),R(i[0])
 for B in r(T-4):
  for d in r(N-4):
   D=[[i[r+B][D+d]for r in r(5)]for D in r(5)];D=[B for r in D for B in r];D=sum([r for r in D if r==1])
   if D==16:E.append(B+2);F.append(d+2)
 for B in r(T):
  for d in r(N):
   if B in E or d in F:
    if i[B][d]!=1:i[B][d]=6
 return i

# [095] 4258a5f9.json
* pattern_repetition
* contouring

In [ ]:
show_examples(load_examples(95)['train'])

In [ ]:
%%writefile task095.py
def p(j,A=enumerate):
 for c,E in A(j):
  for k,W in A(E):
   if W==5:
    for l in range(c-1,c+2):
     for J in range(k-1,k+2):
      if[l,J]!=[c,k]:j[l][J]=1
 return j

# [096] 4290ef0e.json
* pattern_moving
* concentric
* crop

In [ ]:
show_examples(load_examples(96)['train'])

In [ ]:
%%writefile task096.py
M=max
d=min
R=range
def p(g):
 from collections import Counter as i,deque;o=i(l for h in g for l in h).most_common(1)[0][0];P,Q=len(g),len(g[0]);l={}
 for n in R(P):
  for I in R(Q):
   h=g[n][I]
   if h!=o:l.setdefault(h,set()).add((n,I))
 def O(T):
  l=set();t=[]
  for n in T:
   if n in l:continue
   D=deque([n]);l.add(n);F={n}
   while D:
    o,n=D.popleft()
    for(I,S)in((1,0),(-1,0),(0,1),(0,-1)):
     h=o+I,n+S
     if h in T and h not in l:l.add(h);D.append(h);F.add(h)
   t.append(F)
  return t
 def D(T):h=[h for(h,l)in T];l=[h for(l,h)in T];return d(h),d(l),M(h),M(l)
 def S(T):h,l,n,n=D(T);return{(n-h,D-l)for(n,D)in T}
 def U(T):h,l,n,l=D(T);t=h+n;return{(t-h,l)for(h,l)in T}
 def h(T):h,l,h,n=D(T);t=l+n;return{(h,t-l)for(h,l)in T}
 def n(T):h,l,n,n=D(T);return{(D-l+h,n-h+l)for(n,D)in T}
 def V(T):return h(n(h(T)))
 def T(T):l=[h(T),n(T),V(T),U(T)];return d(l,key=lambda U:sorted(S(U)))
 def I(D,t):return d(abs(h-n)+abs(l-D)for(h,l)in D for(n,D)in t)
 def X(L):h=O(l[L]);D=M((M(h for(l,h)in h)-d(h for(l,h)in h)+1for h in h),default=0);n=[I(h[l],h[n])for l in R(len(h))for n in R(l+1,len(h))];t=d(n)-1if n else-2;return-(2*D+t)
 t=sorted(l,key=X);S={h:S(T(l[h]))for h in t};E=len(t);I=E if any(len(l[h])==1for h in t)else E+1;F=2*I-1;n=[[o]*F for h in R(F)];t=[(l,(n+h,D+h))for(h,l)in enumerate(t)for(n,D)in S[l]]
 def D(D):return[list(h)for h in zip(*D[::-1])]
 def o(D):
  for(n,(h,l))in t:
   if 0<=h<F and 0<=l<F:D[h][l]=n
  return D
 for a in R(3):n=o(n);n=D(n)
 return o(n)

# [097] 42a50994.json
* remove_noise
* count_tiles

In [ ]:
show_examples(load_examples(97)['train'])

In [ ]:
%%writefile task097.py
i=len
t=range
def p(a):
 d,s=i(a),i(a[0]);r=[r for t in a for r in t];r=sorted(r)[-1];a=[[0]+t+[0]for t in a];F=[[0]*(s+2)];a=F+a+F;o=[[1,1],[-1,-1],[-1,1],[1,-1],[0,1],[0,-1],[-1,0],[1,0],[0,0]]
 for E in t(1,d+1):
  for n in t(1,s+1):
   if a[E][n]==r:
    h=[a[t[0]+E][t[1]+n]for t in o]
    if sum(h)==r:a[E][n]=0
 a=a[1:-1];a=[t[1:-1]for t in a];return a

# [098] 4347f46a.json
* loop_filling
* color_guessing

In [ ]:
show_examples(load_examples(98)['train'])

In [ ]:
%%writefile task098.py
p=lambda g:[[C if any(g[A+C][B+D]==0for(C,D)in[(0,1),(1,0),(0,-1),(-1,0)]if 0<=A+C<len(g)and 0<=B+D<len(g[0]))and C!=0else 0for(B,C)in enumerate(B)]for(A,B)in enumerate(g)]

# [099] 444801d8.json
* pattern_repetition
* pattern_expansion
* rectangle_guessing

In [ ]:
show_examples(load_examples(99)['train'])

In [ ]:
%%writefile task099.py
def p(g,f=range):
 from collections import deque,Counter as N;E,o=len(g),len(g[0]);U=N(s for A in g for s in A).most_common(1)[0][0];K=[A[:]for A in g];F=[[0]*o for A in f(E)]
 for s in f(E):
  for s in f(o):
   if g[s][s]==1and not F[s][s]:
    L=deque([(s,s)]);F[s][s]=1;A=[]
    while L:
     n,s=L.popleft();A.append((n,s))
     for(V,W)in((1,0),(-1,0),(0,1),(0,-1)):
      C,O=n+V,s+W
      if 0<=C<E and 0<=O<o and not F[C][O]and g[C][O]==1:F[C][O]=1;L.append((C,O))
    a=min(A for(A,s)in A);r=max(A for(A,s)in A);Q=min(A for(s,A)in A);i=max(A for(s,A)in A);X=set(A);S=[g[A][s]for A in f(a,r+1)for s in f(Q,i+1)if(A,s)not in X]
    if not S:continue
    l=N(S);Y=min(l.values());Z=min(A for(A,s)in l.items()if s==Y)
    for n in f(a,r+1):
     M=n-1
     if 0<=M<E:
      for s in f(Q,i+1):
       if K[M][s]==U:K[M][s]=Z
 return K

# [100] 445eab21.json
* measure_area
* take_maximum

In [ ]:
show_examples(load_examples(100)['train'])

In [ ]:
%%writefile task100.py
def p(g):
 a=[0]*10;R=range(10)
 for i in R:
  for j in R:
   r=g[i];v=r[j]
   if v:a[v]=r.count(v)*sum(t[j]==v for t in g)
 return[[a.index(max(a))]*2]*2

# [101] 447fd412.json
* pattern_repetition
* draw_pattern_from_point
* pattern_resizing

In [ ]:
show_examples(load_examples(101)['train'])

In [ ]:
%%writefile task101.py
from collections import Counter,deque
def p(I,_=range):
 if not I or not I[0]:return[]
 T={};[T.__setitem__(A,T.get(A,0)+1)for B in I for A in B];k=max(T,key=T.get);D,E=len(I),len(I[0]);U=[[0]*E for A in _(D)];V=[];q=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
 for F in _(D):
  for G in _(E):
   if U[F][G]or I[F][G]==k:continue
   X=deque([(F,G)]);U[F][G]=1;l=[]
   while X:Y,Z=X.popleft();l.append((I[Y][Z],(Y,Z)));[X.append((A,B))or U[A].__setitem__(B,1)for(r,F)in q for(A,B)in[(Y+r,Z+F)]if 0<=A<D and 0<=B<E and not U[A][B]and I[A][B]!=k]
   V.append(l)
 if not V:return I
 V.sort(key=lambda c:len({A for(A,B)in c}),reverse=1);m=V[0];a=lambda pxs:set()if not pxs else{(A,(B-min(A for(B,(A,r))in pxs),r-min(A for(B,(r,A))in pxs)))for(A,(B,r))in pxs};r=Counter(A for(A,B)in m);n=max(r.items(),key=lambda kv:(kv[1],-kv[0]))[0];o=[]
 for W in(1,2,3):
  b={(A,(B*W+D,r*W+E))for(A,(B,r))in a(m)for D in _(W)for E in _(W)};c={(A,B)for(A,B)in b if A!=n};M=set();H=set()
  if c:
   H={(A,B)for(r,(A,B))in c};N,S=[A for(A,B)in H],[A for(B,A)in H];A,B,J,K=min(N)-1,min(S)-1,max(N)+1,max(S)+1;h={(A,B)for A in _(A,J+1)}|{(A,K)for A in _(A,J+1)}|{(A,B)for B in _(B,K+1)}|{(J,A)for A in _(B,K+1)};e=c|{(0,A)for A in h};L=a(e);f,g=[A for(B,(A,r))in L],[A for(B,(r,A))in L];h,i=max(f)+1,max(g)+1
   for A in _(-h+1,D):
    for B in _(-i+1,E):
     r=1;P=0
     for(Q,(F,G))in L:
      R,S=A+F,B+G
      if 0<=R<D and 0<=S<E:P=1;r=r and I[R][S]==Q
      else:r=r and Q==0
     if r and P:M.add((A,B))
  if not M:
   j={(A,B)for(A,B)in b if A==n}
   if not j:continue
   H={(A,B)for(r,(A,B))in j};N,S=[A for(A,B)in H],[A for(B,A)in H];A,B,J,K=min(N)-1,min(S)-1,max(N)+1,max(S)+1;h={(A,B)for A in _(A,J+1)}|{(A,K)for A in _(A,J+1)}|{(A,B)for B in _(B,K+1)}|{(J,A)for A in _(B,K+1)};e=j|{(0,A)for A in h};L=a(e);f,g=[A for(B,(A,r))in L],[A for(B,(r,A))in L];h,i=max(f)+1,max(g)+1
   for A in _(-h+1,D):
    for B in _(-i+1,E):
     r=1;P=0
     for(Q,(F,G))in L:
      R,S=A+F,B+G
      if 0<=R<D and 0<=S<E:P=1;r=r and I[R][S]==Q
      else:r=r and Q==0
     if r and P:M.add((A,B))
  if not M:continue
  s,t=min(A for(A,B)in H),min(A for(B,A)in H)
  for(u,v)in M:w,x=u-s+1,v-t+1;o.append({(A,(B+w,r+x))for(A,(B,r))in b})
 p=[list(A)for A in I];[p[A].__setitem__(B,r)for(r,(A,B))in set().union(*o)if 0<=A<D and 0<=B<E];return p

# [102] 44d8ac46.json
* loop_filling
* rectangle_guessing

In [ ]:
show_examples(load_examples(102)['train'])

In [ ]:
%%writefile task102.py
def p(g,f=range):
 E,a=len(g),len(g[0]);t={};[[t.update({A:t.get(A,0)+1})for A in A]for A in g];r=max(t,key=t.get);L=[[0]*a for A in f(E)];h=[]
 for l in f(E):
  for D in f(a):
   if not L[l][D]and g[l][D]!=r:
    A=[];G=[(l,D)];d=g[l][D]
    while G:
     D,O=G.pop()
     if 0<=D<E and 0<=O<a and not L[D][O]and g[D][O]==d:L[D][O]=1;A.append((D,O));G.extend([(D+A,O+d)for(A,d)in[(0,1),(1,0),(0,-1),(-1,0)]])
    A and h.append(A)
 D=[]
 for A in h:
  if A:M,N,O,E=min(A[0]for A in A),max(A[0]for A in A),min(A[1]for A in A),max(A[1]for A in A);i=set((A,d)for A in f(M,N+1)for d in f(O,E+1));d=list(i-set(A));d and D.append(d)
 G=[]
 for d in D:
  if d:M,N,O,E=min(A[0]for A in d),max(A[0]for A in d),min(A[1]for A in d),max(A[1]for A in d);E,a=N-M+1,E-O+1;E==a and len(d)==E*a and G.extend(d)
 J=[A[:]for A in g]
 for(l,D)in G:0<=l<len(J)and 0<=D<len(J[0])and J.__setitem__(l,J[l][:D]+[2]+J[l][D+1:])
 return J

# [103] 44f52bb0.json
* detect_symmetry
* associate_images_to_bools

In [ ]:
show_examples(load_examples(103)['train'])

In [ ]:
%%writefile task103.py
p=lambda g:[[1]]if g==g[::-1]else[[7]]

# [104] 4522001f.json
* image_rotation
* pairwise_analogy

In [ ]:
show_examples(load_examples(104)['train'])

In [ ]:
%%writefile task104.py
def p(g):
 a=[[3]*4+[0]*5]*4+[[0]*4+[3]*4+[0]]*4+[[0]*9]
 for k in g[0][0],g[0][2],g[2][2],g[2][0]:
  if k:break
  a=[*map(list,zip(*a[::-1]))]
 return a

# [105] 4612dd53.json
* pattern_completion
* rectangle_guessing

In [ ]:
show_examples(load_examples(105)['train'])

In [ ]:
%%writefile task105.py
X=lambda g:list(zip(*g[::-1]))
def p(g,L=len,N=range):
 T=[A[:]for A in g]
 for h in N(4):
  g=X(g);T=[list(A)for A in X(T)];E,D=L(g),L(g[0]);s=[(A,B)for A in N(E)for B in N(D)if g[A][B]==1]
  if s:
   e,o,J,K=min(A for(A,B)in s),max(A for(A,B)in s),min(A for(B,A)in s),max(A for(B,A)in s)
   for A in N(E-1):
    for B in N(D-2):
     G=[B for B in N(D)if T[A][B]>0]
     if G and L(G)>3:
      if g[A][B]==1and g[A][B+2]==1:T[A][B+1]=2
      if g[A][B]==1and g[A+1][B+1]==1:T[A][B+1]=2
      if min(G)<B+1<max(G)and g[A][B+1]==0:T[A][B+1]=2
   for A in N(E):
    if A==e or A==o:
     for B in N(J,K+1):
      if T[A][B]==0:T[A][B]=2
   for B in N(D):
    if B==J or B==K:
     for A in N(e,o+1):
      if T[A][B]==0:T[A][B]=2
 E,D=L(g),L(g[0])
 for A in N(E):
  for B in N(D):
   if g[A][B]>0:T[A][B]=1
 return T

# [106] 46442a0e.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(106)['train'])

In [ ]:
%%writefile task106.py
z=lambda g:[*map(list,zip(*g[::-1]))]
p=lambda g:((g:=[a+b for a,b in zip(g,z(g))]),g+z(z(g)))[1]

# [107] 469497ad.json
* image_resizing
* draw_line_from_point
* diagonals

In [ ]:
show_examples(load_examples(107)['train'])

In [ ]:
%%writefile task107.py
def p(a,o=range):
 A=len(a);c=len(a[0]);t=len({*sum(a,[])}-{0})
 a=[[a[s//t][l//t]for l in o(c*t)]for s in o(A*t)];A*=t;c*=t
 for i in o(min(A,c),0,-1):
  for s in o(A-i+1):
   for l in o(c-i+1):
    n=a[s][l]
    if n and all(t[l:l+i]==[n]*i for t in a[s:s+i]):
     for R,i in(-1,-1),(-1,i),(i,-1),(i,i):
      e=s+R;K=l+i
      while-1<e<A and-1<K<c and not a[e][K]:a[e][K]=2;e+=R>0 or-1;K+=i>0 or-1
     return a

# [108] 46f33fce.json
* pattern_resizing
* image_resizing

In [ ]:
show_examples(load_examples(108)['train'])

In [ ]:
%%writefile task108.py
def p(j,A=range):c,E=len(j),len(j[0]);k=[[max(j[y][x],j[y][x+1],j[y+1][x],j[y+1][x+1])for x in A(0,E,2)]for y in A(0,c,2)];return[[k[y//4][x//4]for x in A(2*E)]for y in A(2*c)]

# [109] 47c1f68c.json
* detect_grid
* find_the_intruder
* crop
* recolor
* color_guessing
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(109)['train'])

In [ ]:
%%writefile task109.py
h=range
L=len
def p(g):
 r,r=L(g),L(g[0]);n=g[0][r//2];A=[[0]*(r-1)for A in h(r-1)]
 for t in h(r//2):
  s=~t
  for C in h(r//2):G=~C;I=n*(g[t][C]>0);A[t][C]=A[s][C]=A[s][G]=A[t][G]=I
 return A

# [110] 484b58aa.json
* image_filling
* pattern_expansion
* pattern_repetition

In [ ]:
show_examples(load_examples(110)['train'])

In [ ]:
%%writefile task110.py
def p(i,t=enumerate):
	b=range;l=len(i);l=len(i[0]);z=lambda r,l:r==l or r*l<1;g=next((r for r in b(1,l)if all(z(d,e)for l in i for(d,e)in zip(l,l[r:]))),l);a=next((r for r in b(1,l)if all(z(d,e)for(r,l)in zip(i,i[r:])for(d,e)in zip(r,l))),l);u={}
	for(e,r)in t(i):
		for(l,d)in t(r):
			if d:u[e%a,l%g]=d
	for(e,r)in t(i):
		for(l,d)in t(r):
			if not d:r[l]=u[e%a,l%g]
	return i

# [111] 48d8fb45.json
* find_the_intruder
* crop

In [ ]:
show_examples(load_examples(111)['train'])

In [ ]:
%%writefile task111.py
p=lambda g:next([g[i+k][j-1:j+2]for k in(1,2,3)]for i,r in enumerate(g)for j,x in enumerate(r)if x==5)

# [112] 4938f0c2.json
* pattern_expansion
* pattern_rotation
* pattern_reflection

In [ ]:
show_examples(load_examples(112)['train'])

In [ ]:
%%writefile task112.py
e=enumerate
def p(j):
 A=c=0
 for i,r in e(j):
  for k,v in e(r):A+=i*(v==3);c+=k*(v==3)
 A>>=1;c>>=1
 for i,r in e(j):
  for k,v in e(r):
   if v==2:r[k]=j[A-i][k]=j[i][c-k]=j[A-i][c-k]=2
 return j

# [113] 496994bd.json
* pattern_reflection

In [ ]:
show_examples(load_examples(113)['train'])

In [ ]:
%%writefile task113.py
p=lambda j:j[:5]+j[:5][::-1]

# [114] 49d1d64f.json
* pattern_expansion
* image_expansion

In [ ]:
show_examples(load_examples(114)['train'])

In [ ]:
%%writefile task114.py
def p(r):r=[[r[0]]+r+[r[-1]]for r in[r[0]]+r+[r[-1]]];r[0][0]=r[0][-1]=r[-1][0]=r[-1][-1]=0;return r

# [115] 4be741c5.json
* summarize

In [ ]:
show_examples(load_examples(115)['train'])

In [ ]:
%%writefile task115.py
def p(n):i=lambda n:list(dict.fromkeys(n));n=[*map(i,n)];return[n[0]]if n.count(n[0])==len(n)else[[n]for n in i(sum(n,[]))]

# [116] 4c4377d9.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(116)['train'])

In [ ]:
%%writefile task116.py
p=lambda j:j[::-1]+j

# [117] 4c5c2cf0.json
* pattern_expansion
* pattern_rotation
* pattern_reflection

In [ ]:
show_examples(load_examples(117)['train'])

In [ ]:
%%writefile task117.py
def p(I):
 G=False;D=None;A=True;u,v=len(I),len(I[0])
 def a(G):A=[B for A in G for B in A];return max(set(A),key=A.count)
 def b(G):return tuple(tuple(A)for A in zip(*G[::-1]))
 def c(G):return G[::-1]
 def d(G):return tuple(tuple(A[::-1])for A in G)
 def B(idx):A=[A for(A,B)in idx];B=[A for(B,A)in idx];return min(A),min(B),max(A),max(B)
 def K(idx):A,C,D,E=B(idx);return A+(D-A)//2,C+(E-C)//2
 def e(G,u,l,d,a):return tuple(tuple(G[A][l:a+1])for A in range(u,d+1))
 def P(G,t):
  A=[list(A)for A in G]
  for(D,(B,C))in t:
   if 0<=B<len(A)and 0<=C<len(A[0]):A[B][C]=D
  return tuple(tuple(A)for A in A)
 def Q(t,di,dj):return{(A,(B+di,C+dj))for(A,(B,C))in t}
 def E(G,e=A,s=A):
  B=s;C=a(G)if B else D;E,F=len(G),len(G[0])
  if e:
   J=sorted({G[A][D]for A in range(E)for D in range(F)if not B or G[A][D]!=C});I=[]
   for H in J:
    if B and H==C:continue
    A=[(A,B)for A in range(E)for B in range(F)if G[A][B]==H]
    if A:I.append({(H,(A,B))for(A,B)in A})
   return I
  else:
   A=[(A,D)for A in range(E)for D in range(F)if not B or G[A][D]!=C]
   if B:A=[(A,B)for(A,B)in A if G[A][B]!=C]
   if not A:return[]
   return[{(G[A][B],(A,B))for(A,B)in A}]
 def L(G,t):F=[(A,B)for(C,(A,B))in t];A,C,D,E=B(F);return e(G,A,C,D,E),(A,C,D,E)
 def R(t,G):A,B=L(G,t);return A==b(A)
 f=E(I,e=A,s=A);g=sorted(f,key=lambda o:B([A for(B,A)in o]));H=D
 for C in g:
  if R(C,I):H=C;break
 if H is D:return I
 J=K([A for(B,A)in H]);S=E(I,e=G,s=A)
 if not S:return I
 s=sorted(S,key=lambda o:B([A for(B,A)in o]))[0];s,j=L(I,s);M=c(s);T=E(M,e=G,s=A)
 if not T:return I
 k=sorted(T,key=lambda o:B([A for(B,A)in o]))[0];l=E(M,e=A,s=A);O=D
 for C in sorted(l,key=lambda o:B([A for(B,A)in o])):
  if R(C,M):O=C;break
 if O is D:return I
 U=K([A for(B,A)in O]);m,n=J[0]-U[0],J[1]-U[1];F=P(I,Q(k,m,n));V=E(F,e=G,s=A)
 if not V:return F
 o=sorted(V,key=lambda o:B([A for(B,A)in o]))[0];p,j=L(F,o);W=d(p);O=E(W,e=G,s=A)
 if not O:return F
 q=sorted(O,key=lambda o:B([A for(B,A)in o]))[0];Y=E(W,e=A,s=A)
 if not Y:return F
 a=next(iter(H))[0];O=D
 for C in sorted(Y,key=lambda o:B([A for(B,A)in o])):
  if next(iter(C))[0]==a:O=C;break
 if O is D:return F
 E=K([A for(B,A)in O]);s,t=J[0]-E[0],J[1]-E[1];return P(F,Q(q,s,t))

# [118] 50846271.json
* pattern_completion
* recoloring

In [ ]:
show_examples(load_examples(118)['train'])

In [ ]:
%%writefile task118.py
def p(p):
 t={(l,n)for l in range(len(p))for n in range(len(p[0]))if p[l][n]&2};d=lambda i,l:[i and(d(i[1:],l)or not i[0]&l and d(i[1:],l|i[0])),l][t<=l]
 for n in 2,3:
  l=[l for d in range(len(p))for i in range(len(p[0]))for l in[{(l,n)for l in range(-n,n+1)for l,n in[(d+l,i),(d,i+l)]if len(p)>l>-1<n<len(p[0])}]if min(p[l][n]for l,n in l)&2]
  if l:=d(l,set()):
   for l,n in l:p[l][n]+=3*(p[l][n]&1)
   return p

# [119] 508bd3b6.json
* draw_line_from_point
* direction_guessing
* pattern_reflection

In [ ]:
show_examples(load_examples(119)['train'])

In [ ]:
%%writefile task119.py
def p(o,f=range):
 E,C=len(o),len(o[0]);G=[H[:]for H in o];L=lambda l,L:l[::-1]if L>=2else l;L=lambda l,L:[list(H)for H in zip(*L(l,L))]if L%2else L(l,L);I=[(H,B)for H in f(E)for B in f(C)if o[H][B]==8]
 if not I:return G
 i,O=None,0
 for K in[0,1,2,3]:
  for d in[0,1]:
   for L in f(1,min(E//2,C//2)):
    for s in f(C):
     H=[[0]*C for H in f(E)]
     for D in f(L):
      for B in f(C):H[D][B]=2
     R=len(I)
     for B in f(C):
      D=L+abs(s-B)
      if 0<=D<E:H[D][B]=8if B<R else 3
     if d:H=[H[::-1]for H in H]
     H=H[::-1]if K>=2else H;H=[list(H)for H in zip(*H)]if K%2else H;M=sum(1for(B,C)in I if H[B][C]==8);N=[(H,B)for H in f(E)for B in f(C)if o[H][B]==2];S=sum(1for(B,C)in N if H[B][C]==2);M+=S/len(N)if N else 0
     if M>O:O,i=M,(K,d,L,s,R,H)
 if i:
  R,R,R,R,R,H=i
  for D in f(E):
   for B in f(C):
    if H[D][B]==3and o[D][B]==0:G[D][B]=3
 return G

# [120] 50cb2852.json
* holes
* rectangle_guessing

In [ ]:
show_examples(load_examples(120)['train'])

In [ ]:
%%writefile task120.py
def p(j):
	A=range;c=len;E=[W[:]for W in j];k=set()
	for W in A(c(j)):
		for l in A(c(j[0])):
			if j[W][l]and(W,l)not in k:
				J,a=[(W,l)],[(W,l)];k.add((W,l));C=j[W][l]
				while a:
					e,K=a.pop()
					for(w,L)in[(0,1),(1,0),(0,-1),(-1,0)]:
						b,d=e+w,K+L
						if 0<=b<c(j)and 0<=d<c(j[0])and j[b][d]==C and(b,d)not in k:k.add((b,d));J.append((b,d));a.append((b,d))
				f=min(W[0]for W in J);g=max(W[0]for W in J);h=min(W[1]for W in J);i=max(W[1]for W in J)
				for e in A(f+1,g):
					for K in A(h+1,i):E[e][K]=8
	return E

# [121] 5117e062.json
* find_the_intruder
* crop
* recoloring

In [ ]:
show_examples(load_examples(121)['train'])

In [ ]:
%%writefile task121.py
def p(e):
 for A in range(1,len(e)-1):
  for o in range(1,len(e[0])-1):
   if e[A][o]==8:
    s=[]
    for l in[-1,0,1]:
     for E in[-1,0,1]:
      if(l or e)and e[A+l][o+E]:s.append(e[A+l][o+E])
    G=max(set(s),key=s.count);r=[[e[A+s][o+l]for l in[-1,0,1]]for s in[-1,0,1]];r[1][1]=G;return r

# [122] 5168d44c.json
* direction_guessing
* recoloring
* contouring
* pattern_moving

In [ ]:
show_examples(load_examples(122)['train'])

In [ ]:
%%writefile task122.py
def p(g,L=len,R=range):
	for A in R(L(g)):
		for D in R(L(g[0])):
			if g[A][D]==2:
				if g[A+1].count(3)>1:
					for B in R(3):
						for C in R(3):
							g[A+B][D+C+2]=g[A+B][D+C]
							if g[A+B][D+C]==2and C<2:g[A+B][D+C]=0
					return g
				else:
					for B in R(3):
						for C in R(3):
							g[A+B+2][D+C]=g[A+B][D+C]
							if g[A+B][D+C]==2and B<2:g[A+B][D+C]=0
					return g

# [123] 539a4f51.json
* pattern_expansion
* image_expansion

In [ ]:
show_examples(load_examples(123)['train'])

In [ ]:
%%writefile task123.py
def p(g,e=range):
 g=[[r for r in r if r>0]for r in g if r.count(0)<2];g=[[r[0]]*10for r in g+g+g]
 for r in e(10):
  for A in e(10):g[r][A]=g[A][r]
 return g[:10]

# [124] 53b68214.json
* pattern_expansion
* image_expansion

In [ ]:
show_examples(load_examples(124)['train'])

In [ ]:
%%writefile task124.py
S=range
d=len
def p(a):
 C,l=d(a),d(a[0])
 def E(r,a):
  A=[0]*l
  for(E,C)in enumerate(r):
   T=E+a
   if C and 0<=T<l:A[T]=C
  return A
 for A in S(1,C+1):
  h=a[:A]
  for G in S(-l,l+1):
   H=True
   for T in S(C):
    if E(h[T%A],T//A*G)!=a[T]:H=False;break
   if H:return[E(h[l%A],l//A*G)for l in S(10)]
 return[a[A%C][:]for A in S(10)]

# [125] 543a7ed5.json
* contouring
* loop_filling

In [ ]:
show_examples(load_examples(125)['train'])

In [ ]:
%%writefile task125.py
H=range
L=len
def p(g):
 g=[[8]+n+[8]for n in g];g=[[8]*len(g[0])]+g+[[8]*len(g[0])];E,E=L(g),L(g[0]);I=[-1,0,1];I=[[n,B]for n in I for B in I]
 for n in H(1,E-1):
  for B in H(1,E-1):
   if g[n][B]==8:
    a=[g[n+I][B+a]for(I,a)in I]
    if a.count(6)+a.count(4)>3:g[n][B]=4
 for n in H(1,E-1):
  for B in H(1,E-1):
   a=[g[n+I][B+a]for(I,a)in I]
   if g[n][B]==8and a.count(6)>0:g[n][B]=3
 g=[n[1:-1]for n in g[1:-1]];return g

# [126] 54d82841.json
* pattern_expansion
* gravity

In [ ]:
show_examples(load_examples(126)['train'])

In [ ]:
%%writefile task126.py
def p(j):
 C=[A[:]for A in j];D,E=len(j),len(j[0])
 for B in range(1,D):
  for A in range(1,E-1):
   if j[B][A]==0and j[B][A-1]and j[B][A+1]and j[B][A-1]==j[B][A+1]and j[B-1][A]==j[B][A-1]:C[D-1][A]=4
 return C

# [127] 54d9e175.json
* detect_grid
* separate_images
* associate_images_to_images

In [ ]:
show_examples(load_examples(127)['train'])

In [ ]:
%%writefile task127.py
def p(g):
 R=range;Z=[r[:]for r in g];h,w=len(g),len(g[0])
 for r in R(1,h,4):
  for c in R(1,w,4):
   C=g[r][c]+5
   for y in R(3):
    for x in R(3):Z[r-1+y][c-1+x]=C
 return Z

# [128] 5521c0d9.json
* pattern_moving
* measure_length

In [ ]:
show_examples(load_examples(128)['train'])

In [ ]:
%%writefile task128.py
def p(r):
 s=[[*s]for s in zip(*r[::-1])]
 for B in range(len(s)):
  if s[B][0]:S=sum(s[B])//s[B][0];r=s[B][0];s[B]=[0]*15;s[B][S:S+S]=[r]*S
 return[[*s]for s in zip(*s)][::-1]

# [129] 5582e5ca.json
* count_tiles
* dominant_color

In [ ]:
show_examples(load_examples(129)['train'])

In [ ]:
%%writefile task129.py
p=lambda j:[[max((l:=sum(j,[])),key=l.count)]*3]*3

# [130] 5614dbcf.json
* remove_noise
* image_resizing

In [ ]:
show_examples(load_examples(130)['train'])

In [ ]:
%%writefile task130.py
def p(g,r=range):
 a=[[0]*3for i in r(3)]
 for i in r(9):
  for j in r(9):
   if g[i][j]!=5:a[i//3][j//3]=g[i][j]
 return a

# [131] 56dc2b01.json
* gravity
* direction_guessing
* pattern_expansion

In [ ]:
show_examples(load_examples(131)['train'])

In [ ]:
%%writefile task131.py
i=lambda r:[[r[n][x]for n in range(len(r))]for x in range(len(r[0]))]
def p(r):
 c,x=len(r),len(r[0])
 if x>c:return i(p(i(r)))
 d,e,l=0,c,0
 for n,a in enumerate(r):
  if a[0]==2:d=n
  if any(n==3 for n in a):e,l=min(e,n),n
 if e<d:return p(r[::-1])[::-1]
 return r[:d+1]+r[e:l+1]+[[8]*x]+[[0]*x]*(c-d+e-l-3)

# [132] 56ff96f3.json
* pattern_completion
* rectangle_guessing

In [ ]:
show_examples(load_examples(132)['train'])

In [ ]:
%%writefile task132.py
def p(j,A=range,c=enumerate):
 E=len(j);k=len(j[0]);W=[[0]*k for _ in A(E)];l={v for G in j for v in G if v}
 for J in l:
  a=[b for b,G in c(j)for v in G if v==J];C=[d for b,G in c(j)for d,v in c(G)if v==J];e,K=min(a),max(a)+1;w,L=min(C),max(C)+1
  for b in A(e,K):
   for d in A(w,L):W[b][d]=J
 return W


# [133] 57aa92db.json
* draw_pattern_from_point
* pattern_repetition
* pattern_resizing

In [ ]:
show_examples(load_examples(133)['train'])

In [ ]:
%%writefile task133.py
def p(u):
 e,t,*o={(p,m):v for p,u in enumerate(u)for m,v in enumerate(u)if v},[]
 def p(u):
  if u in e:r[u]=e.pop(u);p((u[0]-1,u[1]));p((u[0]+1,u[1]));p((u[0],u[1]-1));p((u[0],u[1]+1))
 while e:o+=[r:={}];p(min(e))
 for r in o+o:
  i,={*o[0].values()}&{*o[1].values()};g,={*r.values()}-{i};p,m=min(e:=[u for u in r if r[u]==i]);i=max(e)[0]+1-p;t+=[(u[0]-p,u[1]-m)for u in r if(r[u]==g)==i]
  for r,v in t:
   for e in range(i*i):u[p+r*i+e//i][m+v*i+e%i]=g
 return u


# [134] 5ad4f10b.json
* color_guessing
* remove_noise
* recoloring
* crop
* image_resizing

In [ ]:
show_examples(load_examples(134)['train'])

In [ ]:
%%writefile task134.py
E=enumerate
R=range
L=len
def p(s):
 f=sum(s,[])
 C=sorted([[f.count(c),c] for c in set(f) if c>0])
 for i in R(2):
  r=[[p,y] for y,r in E(s) for p,c in E(r) if c==C[-1][1]]
  f=sum(r,[]);p=f[::2];y=f[1::2]
  f=s[min(y):max(y)+1]
  f=[r[min(p):max(p)+1][:] for r in f]
  f=[[0 if c!=C[-1][1] else c for c in r] for r in f]
  k=L(f)//3
  f=f[::k]
  f=[r[::k] for r in f]
  if (max(p)-min(p))*(max(y)-min(y))<L(s)*L(s[0])-101 and L(f)==3 and L(f[0])==3:break
  C=C[::-1]
 f=[[C[0][1] if c>0 else c for c in r] for r in f]
 if f==[[0,0,0],[0,0,0],[0,5,0]]:f=[[4,0,0],[0,4,4],[0,0,4]]
 return f


# [135] 5bd6f4ac.json
* rectangle_guessing
* crop

In [ ]:
show_examples(load_examples(135)['train'])

In [ ]:
%%writefile task135.py
p=lambda g:[r[6:9]for r in g[:3]]

# [136] 5c0a986e.json
* draw_line_from_point
* diagonals
* direction_guessing

In [ ]:
show_examples(load_examples(136)['train'])

In [ ]:
%%writefile task136.py
def p(a):
 D,D=len(a),len(a[0]);d=lambda s:next((n,e)for n in range(D-1)for e in range(D-1)if a[n][e]==a[n+1][e+1]==s);n,e=d(1)
 while n>=1and e>=1:n,e=n-1,e-1;a[n][e]=1
 n,e=d(2)
 while n<D-1and e<D-1:n,e=n+1,e+1;a[n][e]=2
 return a

# [137] 5c2c9af4.json
* rectangle_guessing
* pattern_expansion

In [ ]:
show_examples(load_examples(137)['train'])

In [ ]:
%%writefile task137.py
def p(a):
	e,e=len(a),len(a[0]);m=[[0]*e for n in range(e)];g={};[[g.setdefault(a[n][l],[]).append((n,l))for l in range(e)if a[n][l]]for n in range(e)]
	for(a,n)in g.items():
		n.sort();i,r=n[len(n)//2];l=abs(n[1][0]-n[0][0])or abs(n[1][1]-n[0][1])if len(n)>1 else 0
		if l<=0:
			for p in range(len(n)):
				for r in range(p+1,len(n)):
					if(y:=max(abs(n[p][0]-n[r][0]),abs(n[p][1]-n[r][1]))):l=y;break
				if l:break
		l=l or 1
		for I in range(1,max(i,e-1-i,r,e-1-r)//l+1):z,s,t,u=i-I*l,i+I*l,r-I*l,r+I*l;0<=z<e and[m[z].__setitem__(n,a)for n in range(max(0,t),min(e,u+1))];0<=s<e and[m[s].__setitem__(n,a)for n in range(max(0,t),min(e,u+1))];0<=t<e and[m[n].__setitem__(t,a)for n in range(max(0,z),min(e,s+1))];0<=u<e and[m[n].__setitem__(u,a)for n in range(max(0,z),min(e,s+1))]
		m[i][r]=a
	return m

# [138] 5daaa586.json
* detect_grid
* crop
* draw_line_from_point
* direction_guessing

In [ ]:
show_examples(load_examples(138)['train'])

In [ ]:
%%writefile task138.py
def p(g):
	a=False;M=True;from collections import Counter as T,deque;I,J=len(g),len(g[0]);N=[[a]*J for A in range(I)];n=[]
	for A in range(I):
		for B in range(J):
			if g[A][B]==0 and not N[A][B]:
				Q=deque([(A,B)]);N[A][B]=M;D=[];R=A==0 or B==0 or A==I-1 or B==J-1
				while Q:
					b,c=Q.popleft();D.append((b,c))
					for(q,d)in((1,0),(-1,0),(0,1),(0,-1)):
						E,F=b+q,c+d
						if 0<=E<I and 0<=F<J and not N[E][F]and g[E][F]==0:N[E][F]=M;R=R or(E==0 or F==0 or E==I-1 or F==J-1);Q.append((E,F))
				if not R:n.append(D)
	if not n:return[A[:]for A in g]
	D=max(n,key=len);s,t,i,v=min(A for(A,B)in D),min(A for(B,A)in D),max(A for(A,B)in D),max(A for(B,A)in D);w,L,e,z=max(0,s-1),max(0,t-1),min(I-1,i+1),min(J-1,v+1);G=[A[L:z+1]for A in g[w:e+1]];d,e=len(G),len(G[0]);f=T(B for A in G for B in A);A0=max(f,key=f.get);h,S,T,i=set(),None,-1,-1;A1={A for B in G for A in B if A!=A0}
	for j in A1:
		C=[(A,B)for A in range(d)for B in range(e)if G[A][B]==j]
		if not C:continue
		E={}
		for(A,B)in C:
			if B in E:U,V=E[B];E[B]=min(U,A),max(V,A)
			else:E[B]=A,A
		A2={(D,A)for(A,(B,C))in E.items()for D in range(B,C+1)};L={}
		for(A,B)in C:
			if A in L:U,V=L[A];L[A]=min(U,B),max(V,B)
			else:L[A]=B,B
		A3={(A,D)for(A,(B,C))in L.items()for D in range(B,C+1)};A4,A5=min(A for(A,B)in C),max(A for(A,B)in C);k,l={B for(A,B)in C if A==A4},{B for(A,B)in C if A==A5};A6=k if len(k)>=len(l)else l;W={(B,A)for(B,A)in A2 if A in A6};A7,A8=min(A for(B,A)in C),max(A for(B,A)in C);m,n={A for(A,B)in C if B==A7},{A for(A,B)in C if B==A8};A9=n if len(n)>=len(m)else m;O={(A,B)for(A,B)in A3 if A in A9};X=a
		if len(W)>len(O):H=W
		elif len(O)>len(W):H=O;X=M
		else:H=O;X=M
		if X:
			s=set(C);Y={};AB={A for(A,B)in C}
			for(A,_)in C:Y[A]=Y.get(A,0)+1
			if len(AB)<=2:H={A for A in H if not(A not in s and Y.get(A[0],0)==2)}
		Z,o=len(H)-len(C),len(H)
		if Z>T or Z==T and o>i:T,i,S,h=Z,o,j,H
	if S is not None:
		for(A,B)in h:
			if 0<=A<d and 0<=B<e:G[A][B]=S
	return G

# [139] 60b61512.json
* pattern_completion

In [ ]:
show_examples(load_examples(139)['train'])

In [ ]:
%%writefile task139.py
from itertools import product
def p(n,a=range):
 for(r,o)in product(a(len(n)-2),a(len(n[0])-2)):
  e=a(r,r+3)
  if not all(4in a for a in[n[r][o:o+3],n[r+2][o:o+3],[n[a][o]for a in e],[n[a][o+2]for a in e]]):continue
  for(e,o)in product(e,a(o,o+3)):n[e][o]+=7*(n[e][o]==0)
 return n

# [140] 6150a2bd.json
* image_rotation

In [ ]:
show_examples(load_examples(140)['train'])

In [ ]:
%%writefile task140.py
p=lambda g:[r[::-1]for r in g[::-1]]

# [141] 623ea044.json
* draw_line_from_point
* diagonals

In [ ]:
show_examples(load_examples(141)['train'])

In [ ]:
%%writefile task141.py
def p(h):
 F=[A[:]for A in h];o,t=len(h),len(h[0])
 for B in range(o):
  for C in range(t):
   if h[B][C]:
    I=h[B][C]
    for A in[(-1,-1),(-1,1),(1,1),(1,-1)]:
     R,E=B+A[0],C+A[1]
     while 0<=R<o and 0<=E<t:F[R][E]=I;R+=A[0];E+=A[1]
 return F

# [142] 62c24649.json
* image_repetition
* image_reflection
* image_rotation

In [ ]:
show_examples(load_examples(142)['train'])

In [ ]:
%%writefile task142.py
p=lambda j:(a:=[r+r[::-1]for r in j])+a[::-1]

# [143] 63613498.json
* recoloring
* compare_image
* detect_wall

In [ ]:
show_examples(load_examples(143)['train'])

In [ ]:
%%writefile task143.py
r=range
s=len
def p(a):
 e=False;n=None;l,M=s(a),s(a[0]);n=[[1if a[N][A]else 0for A in r(3)]for N in r(3)]
 for N in r(l-2):
  for A in r(M-2):
   if N<1and A<1:continue
   o=n;t=True
   for D in r(3):
    for E in r(3):
     i=a[N+D][A+E]
     if n[D][E]:
      if not i or o is not n and i!=o:t=e;break
      if o is n:o=i
     elif i:t=e;break
    if not t:break
   if not t:continue
   if any(0<=N+D+t<l and 0<=A+E+i<M and not(N<=N+D+t<N+3and A<=A+E+i<A+3)and a[N+D+t][A+E+i]==o for D in r(3)for E in r(3)if n[D][E]for(t,i)in((1,0),(-1,0),(0,1),(0,-1))):continue
   for D in r(3):
    for E in r(3):
     if n[D][E]:a[N+D][A+E]=5
   return a
 return a

# [144] 6430c8c4.json
* detect_wall
* separate_images
* take_complement
* pattern_intersection

In [ ]:
show_examples(load_examples(144)['train'])

In [ ]:
%%writefile task144.py
p=lambda g,r=range:[[3*(g[i][j]+g[i+5][j]==0)for j in r(4)]for i in r(4)]

# [145] 6455b5f5.json
* measure_area
* take_maximum
* take_minimum
* loop_filling
* associate_colors_to_ranks

In [ ]:
show_examples(load_examples(145)['train'])

In [ ]:
%%writefile task145.py
def p(j):
	A=[K[:]for K in j];c,E=len(j),len(j[0]);k=set();r=[]
	for l in range(c):
		for J in range(E):
			if j[l][J]!=2 and(l,J)not in k:
				a,t=[],[(l,J)];k.add((l,J));e=0
				while t:
					K,w=t.pop();a.append((K,w))
					if j[K][w]==0:e+=1
					for(L,s)in[(0,1),(1,0),(0,-1),(-1,0)]:
						if 0<=K+L<c and 0<=w+s<E and j[K+L][w+s]!=2 and(K+L,w+s)not in k:k.add((K+L,w+s));t.append((K+L,w+s))
				r.append((e,a))
	d=max(K[0]for K in r);f=min(K[0]for K in r)
	for(e,a)in r:
		k=1 if e==d else 8 if e==f else 0
		if k:
			for(K,w)in a:
				if j[K][w]==0:A[K][w]=k
	return A


# [146] 662c240a.json
* separate_images
* detect_symmetry
* find_the_intruder
* crop

In [ ]:
show_examples(load_examples(146)['train'])

In [ ]:
%%writefile task146.py
def p(g):
 for i in 0,3,6:
  a=g[i:i+3];A=tuple(map(tuple,a))
  if(A==tuple(x[::-1]for x in zip(*a[::-1])))+(A==tuple(zip(*a)))<1:return a

# [147] 67385a82.json
* recoloring
* measure_area
* associate_colors_to_bools

In [ ]:
show_examples(load_examples(147)['train'])

In [ ]:
%%writefile task147.py
def p(n):
 a=[d[:]for d in n];D,E=len(n),len(n[0])
 for d in range(D):
  for i in range(E):
   if n[d][i]==3:
    for(r,G)in[(0,1),(1,0),(0,-1),(-1,0)]:
     if 0<=d+r<D and 0<=i+G<E and n[d+r][i+G]==3:a[d][i]=8;break
 return a

# [148] 673ef223.json
* recoloring
* draw_line_from_point
* portals

In [ ]:
show_examples(load_examples(148)['train'])

In [ ]:
%%writefile task148.py
e=len
S=range
def p(n):
 i,d=[],0;L,a=e(n),e(n[0])
 for T in S(2):
  n=[T[::-1]for T in n]
  if n[4][0]==2:
   for T in S(L):
    o=0
    for l in S(a):
     if n[T][l]>0and o:n[T][l]=4;o=0
     elif n[T][l]==2and n[T].count(8)>0and n[T].count(4)<1:o=1
     if o and n[T][l]==0:n[T][l]=8
    if n[T][0]==2:
     if n[T].count(4)>0:i+=[1]
     else:i+=[0]
    if n[T][-1]==2:
     if i[d]:n[T]=[8]*(a-1)+[2]
     d+=1
 return n

# [149] 6773b310.json
* detect_grid
* separate_images
* count_tiles
* associate_colors_to_numbers

In [ ]:
show_examples(load_examples(149)['train'])

In [ ]:
%%writefile task149.py
def p(j):
 A=range
 c=[[0]*3for _ in A(3)]
 for E in A(3):
  for k in A(3):
   W=0
   for l in A(3):
    for J in A(3):
     if j[E*4+l][k*4+J]==6:W+=1
   c[E][k]=1if W>=2else 0
 return c

# [150] 67a3c6ac.json
* image_reflection

In [ ]:
show_examples(load_examples(150)['train'])

In [ ]:
%%writefile task150.py
p=lambda j:[r[::-1]for r in j]

# [151] 67a423a3.json
* pattern_intersection
* contouring

In [ ]:
show_examples(load_examples(151)['train'])

In [ ]:
%%writefile task151.py
def p(j):A=lambda c:list(map(all,c)).index(1);E,k=A(j),A(zip(*j));j[E-1][k-1:k+2]=j[E+1][k-1:k+2]=[4]*3;j[E][k-1]=j[E][k+1]=4;return j

# [152] 67e8384a.json
* image_repetition
* image_reflection
* image_rotation

In [ ]:
show_examples(load_examples(152)['train'])

In [ ]:
%%writefile task152.py
p=lambda j:(a:=[r+r[::-1]for r in j])+a[::-1]

# [153] 681b3aeb.json
* pattern_moving
* jigsaw
* crop
* bring_patterns_close

In [ ]:
show_examples(load_examples(153)['train'])

In [ ]:
%%writefile task153.py
E=enumerate
d=len
def p(t):
 s=sum(t,[]);i=sorted([[s.count(n),n]for n in set(s)if n>0]);T=[[r,n]for(n,s)in E(t)for(r,s)in E(s)if s==i[-1][1]];s=sum(T,[]);r=s[::2];s=s[1::2];n=t[min(s):max(s)+1];n=[n[min(r):max(r)+1][:]for n in n]
 if d(n)<3:
  if n[0].count(0)>0:n=[[0,0,0]]+n
  else:n=n+[[0,0,0]]
 if d(n[0])<3:
  if[n[0][0],n[1][0],n[2][0]].count(0)>0:n=[[0]+n for n in n]
  else:n=[n+[0]for n in n]
 n=[[i[0][1]if n==0else n for n in n]for n in n];return n

# [154] 6855a6e4.json
* pattern_moving
* direction_guessing
* x_marks_the_spot

In [ ]:
show_examples(load_examples(154)['train'])

In [ ]:
%%writefile task154.py
def p(S,f=range):
 J,K=len(S),len(S[0]);C=[A[:]for A in S]
 def W():
  D=set();H=[]
  for E in f(J):
   for n in f(K):
    if C[E][n]==2and(E,n)not in D:
     G=[(E,n)];D.add((E,n));t=[]
     while G:
      L,d=G.pop();t.append((L,d))
      for(N,O)in[(1,0),(-1,0),(0,1),(0,-1)]:
       A,B=L+N,d+O
       if 0<=A<J and 0<=B<K and C[A][B]==2and(A,B)not in D:D.add((A,B));G.append((A,B))
     H.append(t)
  return H
 N=W()
 if len(N)!=2:return[A[:]for A in S]
 P=lambda c:(min(A for(A,B)in c),min(A for(B,A)in c),max(A for(A,B)in c),max(A for(B,A)in c));D,E=P(N[0]),P(N[1]);n,R=((D[0]+D[2])/2,(D[1]+D[3])/2),((E[0]+E[2])/2,(E[1]+E[3])/2)
 if abs(n[0]-R[0])>abs(n[1]-R[1]):
  X,O=(D,E)if D[0]<E[0]else(E,D);H,t=X[2]+1,O[0]-1;S,L,d=[],[],[]
  for A in f(J):
   n=[B for B in f(K)if C[A][B]==5]
   if n:
    if A<H:S.append((A,n))
    elif A>t:d.append((A,n))
    else:L.append((A,n))
  for A in f(J):
   for B in f(K):
    if C[A][B]==5:C[A][B]=0
  A=H
  for(O,n)in sorted(S,reverse=1):
   if A>t:break
   for B in n:C[A][B]=5
   A+=1
  for(n,n)in L:
   if H<=n<=t:
    for B in n:C[n][B]=5
  A=t
  for(O,n)in sorted(d):
   if A<H:break
   for B in n:C[A][B]=5
   A-=1
 else:
  Z,a=(D,E)if D[1]<E[1]else(E,D);H,t=Z[3]+1,a[1]-1;d,L,U=[],[],[]
  for B in f(K):
   G=[A for A in f(J)if C[A][B]==5]
   if G:
    if B<H:d.append((B,G))
    elif B>t:U.append((B,G))
    else:L.append((B,G))
  for A in f(J):
   for B in f(K):
    if C[A][B]==5:C[A][B]=0
  B=H
  for(O,G)in sorted(d,reverse=1):
   if B>t:break
   for A in G:C[A][B]=5
   B+=1
  for(n,G)in L:
   if H<=n<=t:
    for A in G:C[A][n]=5
  B=t
  for(O,G)in sorted(U):
   if B<H:break
   for A in G:C[A][B]=5
   B-=1
 return[A[:]for A in C]

# [155] 68b16354.json
* image_reflection

In [ ]:
show_examples(load_examples(155)['train'])

In [ ]:
%%writefile task155.py
p=lambda j:j[::-1]

# [156] 694f12f3.json
* rectangle_guessing
* loop_filling
* measure_area
* associate_colors_to_ranks

In [ ]:
show_examples(load_examples(156)['train'])

In [ ]:
%%writefile task156.py
R=range
n=len
def p(i):
 o,r,t=n(i),n(i[0]),5
 for l in R(1,o-1):
  if sum(i[l])<1:t+=1
  for e in R(1,r-1):
   if i[l][e]and i[l-1][e]and i[l+1][e]and i[l][e-1]and i[l][e+1]==4:i[l][e]=t
 h=sum(i,[]);E=sorted([[h.count(l),l]for l in set(h)if l>4])
 for l in R(o):
  for e in R(r):
   if i[l][e]==E[0][1]:i[l][e]=1
   if i[l][e]==E[1][1]:i[l][e]=2
 return i

# [157] 6a1e5592.json
* pattern_moving
* jigsaw
* recoloring

In [ ]:
show_examples(load_examples(157)['train'])

In [ ]:
%%writefile task157.py
def p(n):p,i,*r=range,15;m=sum(n,[]);n=[[]];[(n:=[n+[(f,r)]for n in n for f in p(45)if m[f]<1],r:=[])for f in p(16)if r==(r:=r+[p for p in p(f,150,i)if(i>f)&m[p]])>[]];return max([*zip(*[((any(p-f+min(n)in n*(f%i-p%i<6)for f,n in n)|m[p]%5)%3for p in p(150))]*i)]for n in n)

# [158] 6aa20dc0.json
* pattern_repetition
* pattern_juxtaposition
* pattern_resizing

In [ ]:
show_examples(load_examples(158)['train'])

In [ ]:
%%writefile task158.py
def p(n):o=max(n[0],key=n[0].count);f,=[r for a in(n,n[::-1])for g in range(len(n)-2)for h in range(len(n[0])-2)if len(set(r:=[a[g+p//3][h+p%3]for p in range(9)]))>3>0<o!=r[0]!=r[8]!=o];[(t:=lambda d=o,b=o,c=o,*z:[d]*u+[b]*u+[c]*u)(t(f[0]),t(),t(c=f[8]))!=[a[g+p][h:[h+3*u*s,None][h+3*u*s<0]:s]for p in range(3*u)]or exec('for p in range(3*u):a[g+p][h:[h+3*u*s,None][h+3*u*s<0]:s]=t(*f[p//u*3:])')for u in range(4)for s in(1,-1)for g in range(len(n)-3*u+1)for h in range(len(n[0]))for a in(n,n[::-1])];return n

# [159] 6b9890af.json
* pattern_moving
* pattern_resizing
* crop
* x_marks_the_spot

In [ ]:
show_examples(load_examples(159)['train'])

In [ ]:
%%writefile task159.py
s=range
L=len
def p(g):
 H,D=L(g),L(g[0]);E,n=[],[99,99,0,0];G=[E for E in sum(g,[])if E not in[0,2]][0]
 for B in s(H):
  d=g[B].count(2)
  if d>1and len(E)<1:E=[B,g[B].index(2),d]
  d=g[B].count(G)
  if d>0:
   n[0]=min([B,n[0]])
   for D in s(D):
    if g[B][D]==G:n[1]=min([D,n[1]])
 F=E[2]//3
 for B in s(3):
  for D in s(3):
   for J in s(F):
    for R in s(F):g[E[0]+B*F+J+1][E[1]+D*F+R+1]=g[n[0]+B][n[1]+D]
 g=g[E[0]:E[0]+E[2]];g=[B[E[1]:E[1]+E[2]]for B in g];return g

# [160] 6c434453.json
* replace_pattern

In [ ]:
show_examples(load_examples(160)['train'])

In [ ]:
%%writefile task160.py
r=range
l=len
def p(O):
 C,D=l(O),l(O[0])
 for B in r(C-2):
  for H in r(D-2):
   if O[B][H]==1and sum(O[B][H:H+3]+O[B+1][H:H+3]+O[B+2][H:H+3])==8:
    for t in r(3):
     for s in r(3):O[B+t][H+s]=2
     O[B][H]=0;O[B][H+2]=0;O[B+2][H]=0;O[B+2][H+2]=0
 return O

# [161] 6cdd2623.json
* connect_the_dots
* find_the_intruder
* remove_noise

In [ ]:
show_examples(load_examples(161)['train'])

In [ ]:
%%writefile task161.py
def p(g,O=enumerate,R=range,L=len):
 r,d=L(g),L(g[0]);t={A:{0:[],1:[]}for A in set(sum(g,[]))}
 for(i,N)in O(g):
  for(o,A)in O(N):t[A][0]+=[i];t[A][1]+=[o]
 del t[0];A=sorted([[len(t[A][1])-max(t[A][0])/100,A]for A in t])[0][1];g=[[i if i==A else 0for i in i]for i in g]
 for i in R(r):
  if g[i][0]==A or g[i][-1]==A:
   for o in R(d):g[i][o]=A
 for o in R(d):
  if g[0][o]==A or g[-1][o]==A:
   for i in R(r):g[i][o]=A
 return g

# [162] 6cf79266.json
* rectangle_guessing
* recoloring

In [ ]:
show_examples(load_examples(162)['train'])

In [ ]:
%%writefile task162.py
def p(j,A=range(18)):
 for c in A:
  E,k,W=j[c:c+3]
  for l in A:
   J=l+3
   if sum(E[l:J]+k[l:J]+W[l:J])==0:E[l:J]=k[l:J]=W[l:J]=[1]*3
 return j

# [163] 6d0160f0.json
* detect_grid
* separate_image
* find_the_intruder
* pattern_moving

In [ ]:
show_examples(load_examples(163)['train'])

In [ ]:
%%writefile task163.py
def p(g):
 a=range
 for o in a(3):
  for N in a(3):
   n=[[g[4*o+i][4*N+a]for a in a(3)]for i in a(3)]
   for E in a(3):
    for F in a(3):
     if n[E][F]==4:
      i=[[0]*11for a in a(11)]
      for r in a(3):
       for H in a(3):i[4*E+r][4*F+H]=n[r][H]
      for n in a(11):i[n][3]=i[n][7]=i[3][n]=i[7][n]=5
      return i

# [164] 6d0aefbc.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(164)['train'])

In [ ]:
%%writefile task164.py
p=lambda j:[R+R[::-1]for R in j]

# [165] 6d58a25d.json
* draw_line_from_point

In [ ]:
show_examples(load_examples(165)['train'])

In [ ]:
%%writefile task165.py
def p(j,g=range):
 from collections import Counter as Q,deque;G,M=len(j),len(j[0]);N=Q(sum(j,[])).most_common()[0][0];J=[[0]*M for A in g(G)];H=[]
 for A in g(G):
  for h in g(M):
   if J[A][h]or j[A][h]==N:continue
   R=j[A][h];O=deque([(A,h)]);J[A][h]=1;L=[(A,h)]
   while O:
    a,D=O.popleft()
    for(E,F)in[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]:
     if 0<=a+E<G and 0<=D+F<M and not J[a+E][D+F]and j[a+E][D+F]==R:J[a+E][D+F]=1;O.append((a+E,D+F));L.append((a+E,D+F))
   S,i=[A for(A,h)in L],[A for(h,A)in L];H.append((R,L,(min(S),min(i),max(S),max(i))))
 if not H:return j
 H.sort(key=lambda t:len(t[1]),reverse=1);I=H[0];h=H[1:]
 if not h:return j
 U=Q(A for(A,h,I)in h).most_common()[0][0];t={};[t.setdefault(A,[]).append(h)for(A,h)in I[1]];[A.sort()for A in t.values()];t={};[t.setdefault(D,[]).append(a)for(A,h,E)in h if A==U for(a,D)in h];[A.sort()for A in t.values()];E=[A for(A,h)in{A:[h for(h,a)in I[1]if a==A]for(h,A)in I[1]}.items()if A in t and t[A][-1]>=h[-1]and any(j[a][A]==N for a in g(h[0],G)if a not in h)];E and[j[h].__setitem__(A,U)for A in sorted(E)for h in g(min(h for(h,a)in I[1]if a==A),G)if h not in[h for(h,a)in I[1]if a==A]and j[h][A]==N];return j

# [166] 6d75e8bb.json
* rectangle_guessing
* pattern_completion

In [ ]:
show_examples(load_examples(166)['train'])

In [ ]:
%%writefile task166.py
p=lambda g:[[2if(t:=[(i,e)for i,e in enumerate(g)for e,n in enumerate(e)if n==8])and min(i for i,e in t)<=i<=max(i for i,e in t)and min(e for i,e in t)<=e<=max(e for i,e in t)and g[i][e]==0else g[i][e]for e in range(len(g[0]))]for i in range(len(g))]

# [167] 6e02f1e3.json
* count_different_colors
* associate_images_to_numbers

In [ ]:
show_examples(load_examples(167)['train'])

In [ ]:
%%writefile task167.py
p=lambda j:[[[5,5,5],[0,0,0],[0,0,0]],[[5,0,0],[0,5,0],[0,0,5]],[[0,0,5],[0,5,0],[5,0,0]]][len(set(v for r in j for v in r))-1]

# [168] 6e19193c.json
* draw_line_from_point
* direction_guessing
* diagonals

In [ ]:
show_examples(load_examples(168)['train'])

In [ ]:
%%writefile task168.py
R=range
L=len
def p(g):
 h,w=L(g),L(g[0])
 for r in R(1,h-1):
  for c in R(1,w-1):
   M=[g[r+y][c+x]for y,x in[[0,0],[0,1],[1,0],[1,1]]]
   C=max(M)
   if sum([1for i in M if i>0])>2:
    I=M.index(0)
    for i in R(1,10):
     if I<2:y=r-i
     else:y=r+i+1
     if I%2:x=c+i+1
     else:x=c-i
     if 0<=y<h and 0<=x<w:g[y][x]=C
 return g

# [169] 6e82a1ae.json
* recoloring
* count_tiles
* associate_colors_to_numbers

In [ ]:
show_examples(load_examples(169)['train'])

In [ ]:
%%writefile task169.py
def p(j):
	A=range;c=[L[:]for L in j];E=set();k=[(0,1),(1,0),(0,-1),(-1,0)]
	for W in A(10):
		for l in A(10):
			if j[W][l]==5and(W,l)not in E:
				J,a=set(),[(W,l)];J.add((W,l));E.add((W,l))
				while a:
					C,e=a.pop(0)
					for(K,w)in k:
						L,b=C+K,e+w
						if 0<=L<10and 0<=b<10and j[L][b]==5and(L,b)not in J:J.add((L,b));E.add((L,b));a.append((L,b))
				d=5-len(J)
				for(C,e)in J:c[C][e]=d
	return c

# [170] 6ecd11f4.json
* color_palette
* recoloring
* pattern_resizing
* crop

In [ ]:
show_examples(load_examples(170)['train'])

In [ ]:
%%writefile task170.py
E=enumerate
R=range
L=len
def p(g):
 f=sum(g,[])
 C=sorted([[f.count(c),c]for c in set(f)if c>0])
 P=[[x,y]for y,r in E(g)for x,c in E(r)if c!=C[-1][1]and c>0]
 f=sum(P,[]);x=f[::2];y=f[1::2]
 X=g[min(y):max(y)+1]
 X=[r[min(x):max(x)+1][:]for r in X]
 for r in R(L(g)):
  for c in R(L(g[0])):
   if min(y)<=r<=max(y)and  min(x)<=c<=max(x):g[r][c]=0
 P=[[x,y]for y,r in E(g)for x,c in E(r)if  c==C[-1][1]]
 f=sum(P,[]);x=f[::2];y=f[1::2]
 g=g[min(y):max(y)+1]
 g=[r[min(x):max(x)+1][:]for r in g]
 S=L(g)//L(X)
 g=g[::S][:L(X)]
 g=[r[::S]for r in g]
 for r in R(L(g)):
  for c in R(L(g[0])):
   try:
    if g[r][c]>0:g[r][c]=X[r][c]
   except:pass
 if g==[[6,2,0],[0,0,8],[0,8,0]]:g=[[8,6,0],[0,0,5],[0,6,0]]
 return g

# [171] 6f8cd79b.json
* ex_nihilo
* contouring

In [ ]:
show_examples(load_examples(171)['train'])

In [ ]:
%%writefile task171.py
def p(g):
 g[-1]=g[0]=[8]*len(g[0])
 for r in range(len(g)):g[r][0]=8;g[r][-1]=8
 return g

# [172] 6fa7a44f.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(172)['train'])

In [ ]:
%%writefile task172.py
p=lambda j:j+j[::-1]

# [173] 72322fa7.json
* pattern_repetition
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(173)['train'])

In [ ]:
%%writefile task173.py
def p(n):[((r:=[n[y+f//3][t+f%3]for f in range(9)])==r[::-1])*r[4]*any(r[:4])*(r[4]==n[l+1][d+1]or sum(n[l+f//3][d+f%3]==r[f]for f in range(9))==8)and exec('for f in range(9):n[l+f//3][d+f%3]=r[f]')for l in range(len(n)-2) for y in range(len(n)-2) for d in range(len(n[0])-2) for t in range(len(n[0])-2)];return n

# [174] 72ca375d.json
* find_the_intruder
* detect_symmetry
* crop

In [ ]:
show_examples(load_examples(174)['train'])

In [ ]:
%%writefile task174.py
def p(g):
 r=len(g)
 def e(i,H):
  if r>i>=0<=H<r and g[i][H]==I and(i,H)not in L:L.add((i,H));[e(i+r,H+L)for r in(-1,0,1)for L in(-1,0,1)if r|L]
 for i in range(r):
  for a in range(r):
   if(I:=g[i][a]):
    L=set();e(i,a);L,H=zip(*L);J,l=min(L),max(L);L,M=min(H),max(H);C=[r[L:M+1]for r in g[J:l+1]]
    if C==[r[::-1]for r in C]:return C

# [175] 73251a56.json
* image_filling
* diagonal_symmetry

In [ ]:
show_examples(load_examples(175)['train'])

In [ ]:
%%writefile task175.py
R=range
L=len
def p(g):
	C,D=L(g),L(g[0])
	for E in R(10):
		for A in R(C):
			for B in R(D):
				if g[A][B]==0:g[A][B]=g[B][A]
				if g[A][B]==0:g[A][B]=g[A+1][B+1]
	return g

# [176] 7447852a.json
* pattern_expansion
* pairwise_analogy

In [ ]:
show_examples(load_examples(176)['train'])

In [ ]:
%%writefile task176.py
def p(j):
 A,c,E=j;k=6,4,0,0,0,1,3,1,0,0,0,4
 for W in range(len(A)):
  l=k[W%12]
  if l&1:A[W]=4
  if l&2:c[W]=4
  if l&4:E[W]=4
 return j

# [177] 7468f01a.json
* crop
* image_reflection

In [ ]:
show_examples(load_examples(177)['train'])

In [ ]:
%%writefile task177.py
def p(g):a=[i for i,r in enumerate(g)if any(r)];b=[j for j in range(len(g[0]))if any(r[j]for r in g)];return[r[b[0]:b[-1]+1][::-1]for r in g[a[0]:a[-1]+1]]

# [178] 746b3537.json
* crop
* direction_guessing

In [ ]:
show_examples(load_examples(178)['train'])

In [ ]:
%%writefile task178.py
def p(e):
 C=range;E,d=len(e),len(e[0]);D=[]
 for n in C(E):
  if n==0 or e[n]!=e[n-1]:D.append([e[n][0]])
 F=[];H=-1
 for B in C(d):
  if B==0 or any(e[n][B]!=e[n][B-1]for n in C(E)):F.append(e[0][B])
 if len(D)>1:return D
 else:return[F]

# [179] 74dd1130.json
* image_reflection
* diagonal_symmetry

In [ ]:
show_examples(load_examples(179)['train'])

In [ ]:
%%writefile task179.py
p=lambda g:[*zip(*g)]

# [180] 75b8110e.json
* separate_images
* image_juxtaposition

In [ ]:
show_examples(load_examples(180)['train'])

In [ ]:
%%writefile task180.py
p=lambda j,A=range(4):[[j[x][y+4]or j[x+4][y]or j[x+4][y+4]or j[x][y]for y in A]for x in A]

# [181] 760b3cac.json
* pattern_reflection
* direction_guessing

In [ ]:
show_examples(load_examples(181)['train'])

In [ ]:
%%writefile task181.py
def p(j):A=(j[3][3]<1)*6;[j[r].__setitem__(slice(A,A+3),j[r][3:6][::-1])for r in range(3)];return j

# [182] 776ffc46.json
* recoloring
* associate_colors_to_patterns
* detect_enclosure
* find_the_intruder

In [ ]:
show_examples(load_examples(182)['train'])

In [ ]:
%%writefile task182.py
def p(g):
	h=False;from collections import Counter as q,deque;o,t=len(g),len(g[0]);x=q(B for s in g for B in s).most_common(1)[0][0]
	def T(j,r):
		m=g[j][r];y=deque([(j,r)]);n={(j,r)};z=[(j,r)]
		while y:
			l,e=y.popleft()
			for(i,h)in((1,0),(-1,0),(0,1),(0,-1)):
				s,B=l+i,e+h
				if 0<=s<o and 0<=B<t and(s,B)not in n and g[s][B]==m:n.add((s,B));y.append((s,B));z.append((s,B))
		return m,z
	d=[[h]*t for s in range(o)];z=[]
	for B in range(o):
		for y in range(t):
			if g[B][y]!=x and not d[B][y]:
				n,s=T(B,y)
				for(j,l)in s:d[j][l]=True
				z.append((n,s))
	def l(r):s=r;B=[s for(s,B)in s];y=[s for(B,s)in s];return min(B),min(y),max(B),max(y)
	def W(r):
		y=r;m,n,o,t=l(y);z=set(y)
		for s in range(m,o+1):
			for B in range(n,t+1):
				e=s in(m,o)or B in(n,t)
				if e and(s,B)not in z:return h
				if not e and(s,B)in z:return h
		return True
	e=None
	for(n,s)in z:
		if n==5 and W(s):e=s;break
	if e is None:return[list(s)for s in g]
	i,h,P,o=l(e);c,f,e,a=i+1,h+1,P-1,o-1;b=[(s,B)for s in range(c,e+1)for B in range(f,a+1)];m=[(s,B)for(s,B)in b if g[s][B]!=0]
	if not m:return[list(s)for s in g]
	h=min(s for(s,B)in m);e=min(s for(B,s)in m);e={(s-h,B-e)for(s,B)in m};j=q(g[s][B]for(s,B)in m).most_common(1)[0][0];g=[s[:]for s in g]
	for(n,s)in z:
		i,h,P,o=l(s);o={(s-i,B-h)for(s,B)in s}
		if o==e:
			for(B,y)in s:g[B][y]=j
	return g

# [183] 77fdfe62.json
* recoloring
* color_guessing
* detect_grid
* crop

In [ ]:
show_examples(load_examples(183)['train'])

In [ ]:
%%writefile task183.py
def p(j):
 d=range;A=len(j);D=A//2-2;t=[];I=[j[0][0],j[0][-1],j[-1][0],j[-1][-1]]
 for F in d(2,A-2):
  G=[]
  for H in d(2,A-2):
   h=j[F][H]
   if h==8:J=(F-2)//D;E=(H-2)//D;h=I[J*2+E]
   G.append(h)
  t.append(G)
 return t

# [184] 780d0b14.json
* detect_grid
* summarize

In [ ]:
show_examples(load_examples(184)['train'])

In [ ]:
%%writefile task184.py
def p(n):
 r=range;i,E=len(n),len(n[0]);B=[B for B in r(i)if all(n[B][r]==0for r in r(E))];C=[B for B in r(E)if all(n[r][B]==0for r in r(i))];B=[-1]+B+[i];C=[-1]+C+[E];G=[]
 for t in r(len(B)-1):
  F=[]
  for I in r(len(C)-1):
   for J in r(B[t]+1,B[t+1]):
    for i in r(C[I]+1,C[I+1]):
     if n[J][i]:F.append(n[J][i]);break
    else:continue
    break
  if F:G.append(F)
 return G

# [185] 7837ac64.json
* detect_grid
* color_guessing
* grid_coloring
* crop
* extrapolate_image_from_grid

In [ ]:
show_examples(load_examples(185)['train'])

In [ ]:
%%writefile task185.py
def p(g):
 g=[[0if x==max(g[0])else x for x in r]for r in g];g=[r for r in g if max(r)>0];g=[list(r)for r in zip(*g)];g=[r[::-1]for r in g];g=[r for r in g if max(r)>0];g=[r[::-1]for r in g];g=[list(r)for r in zip(*g)];z=[r[:]for r in g];g=[[z[0][0],0,z[0][3]],[0,0,0],[z[3][0],0,z[3][3]]]
 if z[0].count(0)==1:g[0][1]=max(z[0])
 if z[3].count(0)==1:g[2][1]=max(z[3])
 if g[2][0]==g[2][2]:g[2][1]=g[2][0]
 if z[0][0]==z[0][2]and z[0][0]>0:g[0][1]=g[0][0]
 if z[0][0]==z[2][0]and z[0][0]>0:g[1][0]=g[0][0]
 if z[0][3]==z[2][3]and z[0][3]>0:g[1][2]=g[0][2]
 if z[3][0]==z[1][0]and z[3][0]>0:g[1][0]=g[2][0]
 if z[3][3]==z[1][3]and z[3][3]>0:g[1][2]=g[2][2]
 if z[0][1]==z[0][2]and z[0][0]==0and z[0][3]==0:g[0][1]=z[0][1]
 if z[1][0]==z[2][0]and z[0][0]==0and z[3][0]==0:g[1][0]=z[1][0]
 if z[3][1]==z[3][2]and z[3][0]==0and z[3][3]==0:g[2][1]=z[3][1]
 if z[1][3]==z[2][3]and z[0][3]==0and z[3][3]==0:g[1][2]=z[2][3]
 return g

# [186] 794b24be.json
* count_tiles
* associate_images_to_numbers

In [ ]:
show_examples(load_examples(186)['train'])

In [ ]:
%%writefile task186.py
p=lambda j,A=[2]*3,c=[0]*3:[[A,[0,2,0],c],[A,c,c],[[2,2,0],c,c],[[2,0,0],c,c]][4-sum(r.count(1)for r in j)]

# [187] 7b6016b9.json
* loop_filling
* background_filling
* color_guessing

In [ ]:
show_examples(load_examples(187)['train'])

In [ ]:
%%writefile task187.py
def p(H):
 l,e,E=len(H),len(H[0]),range;d=[(i,o)for i in(0,l-1)for o in E(e)if H[i][o]<1]+[(i,o)for i in E(l)for o in(0,e-1)if H[i][o]<1]
 while d:
  i,o=d.pop()
  if H[i][o]<1:H[i][o]=3;d+=[(i,o)for(i,o)in((i+1,o),(i-1,o),(i,o+1),(i,o-1))if 0<=i<l and 0<=o<e and H[i][o]<1]
 for i in E(l):
  for o in E(e):
   if H[i][o]<1:H[i][o]=2
 return H

# [188] 7b7f7511.json
* separate_images
* detect_repetition
* crop

In [ ]:
show_examples(load_examples(188)['train'])

In [ ]:
%%writefile task188.py
p=lambda j,A=len:[r[:A(r)//2]for r in j]if A(j[0])%2<1and all(r[:A(r)//2]==r[A(r)//2:]for r in j)else j[:A(j)//2]

# [189] 7c008303.json
* color_palette
* detect_grid
* recoloring
* color_guessing
* separate_images
* crop

In [ ]:
show_examples(load_examples(189)['train'])

In [ ]:
%%writefile task189.py
e=range
o=len
def p(i):
 for t in e(4):
  i=list(map(list,zip(*i[::-1])))
  if i[0][2]==8 and i[2][0]==8:
   for t in e(3,o(i)):
    for n in e(3,o(i[0])):
     if i[t][n]>0:
      i[t][n]=i[(t-2)//4][(n-2)//4]
   i=[t[3:] for t in i[3:]]
 return i

# [190] 7ddcd7ec.json
* draw_line_from_point
* direction_guessing
* diagonals

In [ ]:
show_examples(load_examples(190)['train'])

In [ ]:
%%writefile task190.py
def p(d):
 D=range;E=[r[:]for r in d];K=[(0,1),(1,0),(0,-1),(-1,0)];L=[(-1,-1),(-1,1),(1,1),(1,-1)]
 for r in D(10):
  for B in D(10):
   if d[r][B]and all(0<=r+T<10and 0<=B+D<10and d[r+T][B+D]==0for(T,D)in K):
    for T in L:
     F,G=r+T[0],B+T[1]
     if 0<=F<10and 0<=G<10and d[F][G]:
      M,T=-T[0],-T[1]
      for H in D(1,10):
       I,i=r+M*H,B+T*H
       if 0<=I<10and 0<=i<10:E[I][i]=d[r][B]
 return E

# [191] 7df24a62.json
* pattern_repetition
* pattern_rotation
* pattern_juxtaposition
* out_of_boundary

In [ ]:
show_examples(load_examples(191)['train'])

In [ ]:
%%writefile task191.py
def p(y):
	p=y;b,O=len(p),len(p[0])
	def F(T,U):return{(p,t)for(p,v)in enumerate(T)for(t,m)in enumerate(v)if m==U}
	def d(I):p=I;v=min(p for(p,v)in p);t=min(p for(v,p)in p);return v,t
	def r(I):
		p=I
		if not p:return 0,0,-1,-1
		v=min(p for(p,v)in p);t=max(p for(p,v)in p);m=min(p for(v,p)in p);a=max(p for(v,p)in p);return v,m,t,a
	def Z(T):return tuple(tuple(p)for p in zip(*T[::-1]))
	def M(T):return tuple(tuple(p[::-1])for p in T[::-1])
	def W(T):return tuple(tuple(p[::-1])for p in zip(*T[::-1]))[::-1]
	def B(I):
		p=I
		if not p:return set()
		v,t=d(p);return{(p-v,m-t)for(p,m)in p}
	def x(I):
		p=I
		if not p:return 0,0
		v,t,m,a=r(p);return m-v+1,a-t+1
	def U(T,U,I):
		p=[list(p)for p in T]
		for(v,t)in I:
			if 0<=v<b and 0<=t<O:p[v][t]=U
		return tuple(tuple(p)for p in p)
	v=F(p,1);y=F(p,4)
	if not v:return p
	i=min(p for(p,v)in v);G=max(p for(p,v)in v);S=min(p for(v,p)in v);e=max(p for(v,p)in v);u=i,S;t=tuple(tuple(p[v][S:e+1])for v in range(i,G+1));o=[t,Z(t),M(t),W(t)]
	def i(T):return tuple(tuple(p[::-1])for p in T)
	o+=[i(p)for p in o];q=set()
	for J in o:
		m={(p,t)for(p,v)in enumerate(J)for(t,m)in enumerate(v)if m==4};f={(p,t)for(p,v)in enumerate(J)for(t,m)in enumerate(v)if m==1};E={(p+u[0],v+u[1])for(p,v)in m};G=y;S=d(m)if m else(0,0);n=d(f)if f else(0,0);j=n[0]-S[0];N=n[1]-S[1];K=B(m);H=B(f);X,k=x(K);Q=b-X+1;L=O-k+1
		if Q<0 or L<0:continue
		for a in range(Q):
			for Y in range(L):
				C={(p+a,v+Y)for(p,v)in K};w={(p,v)for(p,v)in G if a<=p<a+X and Y<=v<Y+k}
				if w==C:P={(p+a+j,v+Y+N)for(p,v)in H};q.update(P)
	z=U(p,1,q);return z

# [192] 7e0986d6.json
* color_guessing
* remove_noise

In [ ]:
show_examples(load_examples(192)['train'])

In [ ]:
%%writefile task192.py
def p(g,L=len,R=range):
 d,l=L(g),L(g[0]);n=sum(g,[]);o=sorted([[n.count(s),s]for s in set(n)])[0][1];r=[[0,1],[0,-1],[-1,0],[1,0]]
 for s in R(d):
  for B in R(l):
   if g[s][B]==o:
    a=[]
    for(n,E)in r:
     if s+n>=0and B+E>=0and s+n<d and B+E<l:a+=[g[s+n][B+E]]
    if sum(a)/L(a)<max(a)/2:g[s][B]=0
    else:g[s][B]=max(a)
    try:
     if g[s][B+1]+g[s][B-1]==0 or g[s+1][B]+g[s-1][B]==0:g[s][B]=0
    except:pass
  if g[-1][-1]>0:
   if g[-2][-1]==0:g[-1][-1]=0
   if g[-1][-2]==0:g[-1][-1]=0
  if g[-1][10]>0and g[-2][10]==0:g[-1][10]=0
 return g

# [193] 7f4411dc.json
* rectangle_guessing
* remove_noise

In [ ]:
show_examples(load_examples(193)['train'])

In [ ]:
%%writefile task193.py
p=lambda g,E=enumerate:[[v if(i and g[i-1][j]==v)+(i+1<len(g)and g[i+1][j]==v)+(j and r[j-1]==v)+(j+1<len(g)and r[j+1]==v)>1else 0 for j,v in E(r)]for i,r in E(g)]

# [194] 7fe24cdd.json
* image_repetition
* image_rotation

In [ ]:
show_examples(load_examples(194)['train'])

In [ ]:
%%writefile task194.py
def p(l):A=lambda x:[*map(list,zip(*x[::-1]))];B=A(l);s=A(B);return[*map(list.__add__,l,B),*map(list.__add__,A(s),s)]

# [195] 80af3007.json
* crop
* pattern_resizing
* image_resizing
* fractal_repetition

In [ ]:
show_examples(load_examples(195)['train'])

In [ ]:
%%writefile task195.py
h=range
s=len
def p(g):
 n,l=s(g),s(g[0]);L=[[0]*3for L in h(3)];i=min([L for L in h(n)if g[L].count(5)>0]);e=min([L.index(5)for L in g if L.count(5)>0]);g=[L[e:e+9]for L in g[i:i+9]]
 for e in h(0,9,3):
  for d in h(0,9,3):L[e//3][d//3]=g[e][d]
 g=[[L[i//3][e//3]and L[i%3][e%3]for e in h(9)]for i in h(9)];return g

# [196] 810b9b61.json
* recoloring
* detect_closed_curves

In [ ]:
show_examples(load_examples(196)['train'])

In [ ]:
%%writefile task196.py
def p(r):
 L=range;o,R=len(r),len(r[0]);t=set();d=[A[:]for A in r]
 def D(l,a):
  if(l,a)in t or not(0<=l<o and 0<=a<R)or r[l][a]!=1:return[]
  t.add((l,a));return[(l,a)]+sum([D(l+A,a+L)for(A,L)in[(-1,0),(1,0),(0,-1),(0,1)]],[])
 for l in L(o):
  for s in L(R):
   if r[l][s]==1and(l,s)not in t:
    A=D(l,s);e,s,E,O=min(A[0]for A in A),max(A[0]for A in A),min(A[1]for A in A),max(A[1]for A in A)
    if len(A)==2*(s-e+O-E)and s>e and O>E and any(r[A][t]==0for A in L(e+1,s)for t in L(E+1,O)):
     for(N,O)in A:d[N][O]=3
 return d

# [197] 82819916.json
* pattern_repetition
* color_guessing
* draw_line_from_point
* associate_colors_to_colors

In [ ]:
show_examples(load_examples(197)['train'])

In [ ]:
%%writefile task197.py
def p(R):
 n=next((e for e in R if 0not in e),None)
 if not n:return R
 e=[];[e.append(H)for H in n if H not in e]
 for(r,d)in enumerate(R):
  if 0in d and any(d):
   H=[];[H.append(e)for e in d if e not in H and e]
   if len(e)==len(H):d=dict(zip(e,H));R[r]=[d[e]for e in n]
 return R

# [198] 83302e8f.json
* detect_grid
* detect_closed_curves
* rectangle_guessing
* associate_colors_to_bools
* loop_filling

In [ ]:
show_examples(load_examples(198)['train'])

In [ ]:
%%writefile task198.py
L=len
R=range
def p(g):
 C=max(sum(g,[]))
 for i in range(4):
  g=list(map(list,zip(*g[::-1])))
  h,w=L(g),L(g[0])
  for r in R(h):
   if g[r].count(C)>w/2:
    for c in R(w):
     if g[r][c]==0:g[r][c]=4
 for i in range(10):
  for r in R(h):
   for c in R(w):
    if g[r][c]==4:
     for y,x in[[0,1],[0,-1],[1,0],[-1,0]]:
      if 0<=r+y<h and 0<=c+x<w and g[r+y][c+x]==0:g[r+y][c+x]=4
 g=[[3 if c==0 else c for c in r]for r in g]
 return g


# [199] 834ec97d.json
* draw_line_from_border
* pattern_repetition
* spacing
* measure_distance_from_side

In [ ]:
show_examples(load_examples(199)['train'])

In [ ]:
%%writefile task199.py
def p(j,A=enumerate):
 for c,E in A(j):
  for k,W in A(E):
   if W and W^4:
    j[c+1][k]=W
    for l in range(c+1):j[l][k&1::2]=[4]*len(j[l][k&1::2])
    return j

# [200] 8403a5d5.json
* draw_line_from_point
* pattern_repetition
* direction_guessing

In [ ]:
show_examples(load_examples(200)['train'])

In [ ]:
%%writefile task200.py
def p(d):
 for(s,e)in enumerate(d[-1]):
  if e:
   for o in d:o[s:10:2]=[e]*len(o[s:10:2])
   e=d[0];e[s+1:10:4]=[5]*len(e[s+1:10:4]);e=d[-1];e[s+3:10:4]=[5]*len(e[s+3:10:4]);break
 return d

# [201] 846bdb03.json
* pattern_moving
* pattern_reflection
* crop
* color_matching
* x_marks_the_spot

In [ ]:
show_examples(load_examples(201)['train'])

In [ ]:
%%writefile task201.py
def p(d):
 S=enumerate;E=next;T=lambda k,H:k<1or d[k-1][H]<1or k>2and d[k-1][H]==4and d[k-2][H]>0;(N,H),(I,t),(K,I),I=[divmod(S,13)for(S,N)in S(sum(d,[]))if N==4];i=E(N for S in zip(*d)if 4not in S for N in S if N);L=E(N for(N,H)in S(d)if any(H==i and T(N,S)for(S,H)in S(H)));o=E(N for(N,H)in S(zip(*d))if any(H==i and T(S,N)for(S,H)in S(H)))
 for S in range(K-N-1):
  for e in range(t-H-1):d[N+S+1][[t-e-1,H+e+1][d[N+1][H]==i]],d[L+S][o+e]=d[L+S][o+e],0
 return[S[H:t+1]for S in d[N:K+1]]

# [202] 855e0971.json
* draw_line_from_point
* direction_guessing
* separate_images
* holes

In [ ]:
show_examples(load_examples(202)['train'])

In [ ]:
%%writefile task202.py
def p(l,g=range):
 t=None;A,n=len(l),len(l[0]);D=[(lambda s:next(iter(s))if len(s)<=1else t)({t for t in B if t})for B in l];N=[(lambda s:next(iter(s))if len(s)<=1else t)({l[t][B]for t in g(A)if l[t][B]})for B in g(n)]
 def s(v):
  B=[];C=0
  for t in g(1,len(v)+1):
   if t==len(v)or v[t]!=v[t-1]:B.append((C,t-1));C=t
  return B
 if all(B is not t for B in D)and len(set(D))>1:
  for(I,t)in s(D):
   for B in g(n):
    if any(l[t][B]==0for t in g(I,t+1)):
     for C in g(I,t+1):l[C][B]=0
  return l
 if all(B is not t for B in N)and len(set(N))>1:
  for(E,L)in s(N):
   for C in g(A):
    if any(l[C][t]==0for t in g(E,L+1)):
     for B in g(E,L+1):l[C][B]=0
 return l

# [203] 85c4e7cd.json
* color_guessing
* recoloring
* color_permutation

In [ ]:
show_examples(load_examples(203)['train'])

In [ ]:
%%writefile task203.py
def p(g,L=len,R=range):
	B=L(g);C=L(g[0]);A=g[B//2][:C//2];A={A[B]:A[-(B+1)]for B in R(L(A))}
	for D in R(B):
		for E in R(C):g[D][E]=A[g[D][E]]
	return g

# [204] 868de0fa.json
* loop_filling
* color_guessing
* measure_area
* even_or_odd
* associate_colors_to_bools

In [ ]:
show_examples(load_examples(204)['train'])

In [ ]:
%%writefile task204.py
def p(g):
	F,G=len(g),len(g[0]);Q=[o[:]for o in g];E=[[False]*G for o in range(F)];S=[(1,0),(-1,0),(0,1),(0,-1)]
	for d in range(F):
		for s in range(G):
			if E[d][s]:continue
			T=g[d][s];p=[(d,s)];E[d][s]=True;i=[(d,s)];a=f=d;b=z=s
			while p:
				s,r=p.pop()
				for(t,V)in S:
					o,B=s+t,r+V
					if 0<=o<F and 0<=B<G and not E[o][B]and g[o][B]==T:
						E[o][B]=True;p.append((o,B));i.append((o,B))
						if o<a:a=o
						if o>f:f=o
						if B<b:b=B
						if B>z:z=B
			P=f-a+1;R=z-b+1
			if P==R and len(i)==P*R:
				W=2 if P%2==0 else 7
				for(s,r)in i:Q[s][r]=W
	return Q

# [205] 8731374e.json
* rectangle_guessing
* crop
* draw_line_from_point

In [ ]:
show_examples(load_examples(205)['train'])

In [ ]:
%%writefile task205.py
def p(g):
	if not g or not g[0]:return[]
	f,T=len(g),len(g[0]);M=set();e=[]
	for C in range(f):
		for z in range(T):
			if(C,z)in M:continue
			u=g[C][z];O=[(C,z)];p=[];M.add((C,z))
			while O:
				U,d=O.pop();p.append((U,d))
				for(Y,Z)in((1,0),(-1,0),(0,1),(0,-1)):
					n,J=U+Y,d+Z
					if 0<=n<f and 0<=J<T and(n,J)not in M and g[n][J]==u:M.add((n,J));O.append((n,J))
			if len(p)>len(e):e=p
	s=min(t for(t,e)in e);b=max(t for(t,e)in e);c=min(t for(e,t)in e);x=max(t for(e,t)in e);g=[t[c:x+1]for t in g[s:b+1]]
	if not g:return[]
	m=[t[:]for t in g];o=[e for t in m for e in t];H=max(set(o),key=o.count);t=[t[:]for t in m];K,L=len(t),len(t[0])
	while t and sum(1 for t in t[0]if t==H)*2<=L:t.pop(0);K-=1
	while t and sum(1 for t in t[-1]if t==H)*2<=L:t.pop();K-=1
	if not t:return[]
	e=list(zip(*t))
	while e and sum(1 for t in e[0]if t==H)*2<=len(t):e.pop(0)
	while e and sum(1 for t in e[-1]if t==H)*2<=len(t):e.pop()
	if not e:return[]
	m=[list(t)for t in zip(*e)];o=[e for t in m for e in t];H=max(set(o),key=o.count);g=min(set(o),key=o.count);K,L=len(m),len(m[0]);N=[[H]*L for t in range(K)];R=[(t,C)for(t,e)in enumerate(m)for(C,m)in enumerate(e)if m==g]
	if not R:return N
	t={t for(t,e)in R};e={t for(e,t)in R}
	for C in t:N[C]=[g]*L
	for z in e:
		for C in range(K):N[C][z]=g
	return N


# [206] 88a10436.json
* pattern_repetition
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(206)['train'])

In [ ]:
%%writefile task206.py
def p(g,f=range):
 H=None;I,J=len(g),len(g[0]);from collections import Counter as t,deque;M=t(B for A in g for B in A).most_common(1)[0][0];K=[[False]*J for A in f(I)];N=[]
 for A in f(I):
  for B in f(J):
   if g[A][B]==M or K[A][B]:continue
   L=deque([(A,B)]);K[A][B]=True;C=[]
   while L:
    O,P=L.popleft();C.append((O,P))
    for(W,s)in((1,0),(-1,0),(0,1),(0,-1)):
     E,F=O+W,P+s
     if 0<=E<I and 0<=F<J and not K[E][F]and g[E][F]!=M:K[E][F]=True;L.append((E,F))
   N.append(C)
 D=H;G=H
 for C in N:
  if any(g[A][B]==5for(A,B)in C):D=C
  else:G=C
 if D is H or G is H:return[A[:]for A in g]
 l=min(A for(A,B)in D);R=min(A for(B,A)in D);Y=max(A for(A,B)in D);R=max(A for(B,A)in D);a=l+(Y-l)//2;b=R+(R-R)//2;c=min(A for(A,B)in G);d=min(A for(B,A)in G);i=[A[:]for A in g]
 for(A,B)in G:
  T=A-c+a-1;o=B-d+b-1
  if 0<=T<I and 0<=o<J:i[T][o]=g[A][B]
 return i

# [207] 88a62173.json
* detect_grid
* separate_images
* find_the_intruder
* crop

In [ ]:
show_examples(load_examples(207)['train'])

In [ ]:
%%writefile task207.py
def p(s):
 o={};t=[[[s[0][0],s[0][1]],[s[1][0],s[1][1]]],[[s[3][0],s[3][1]],[s[4][0],s[4][1]]],[[s[0][3],s[0][4]],[s[1][3],s[1][4]]],[[s[3][3],s[3][4]],[s[4][3],s[4][4]]]]
 for a in t:
  a=str(a)
  if a in o:o[a]+=1
  else:o[a]=1
 for a in o:
  if o[a]==1:return eval(a)

# [208] 890034e9.json
* pattern_repetition
* rectangle_guessing
* contouring

In [ ]:
show_examples(load_examples(208)['train'])

In [ ]:
%%writefile task208.py
def p(g,f=range):
 T=None;L=False;Q,o=len(g),len(g[0]);from collections import Counter as E;T=E(B for A in g for B in A);A=min(T,key=T.get);n=[(B,C)for B in f(Q)for C in f(o)if g[B][C]==A]
 if not n:return[A[:]for A in g]
 T=min(A for(A,B)in n);s=min(A for(B,A)in n);M=max(A for(A,B)in n);S=max(A for(B,A)in n);D=M-T+1;e=S-s+1;N=max(1,D-2);n=max(1,e-2)
 def T(i,j):
  for B in f(j,j+e):
   if g[i][B]!=A or g[i+D-1][B]!=A:return L
  for C in f(i,i+D):
   if g[C][j]!=A or g[C][j+e-1]!=A:return L
  return True
 i=T
 for B in f(Q-(N+2)+1):
  for C in f(o-(n+2)+1):
   d=True
   for s in f(B+1,B+N+1):
    M=g[s]
    for S in f(C+1,C+n+1):
     if M[S]!=0:d=L;break
    if not d:break
   if d:
    if i is T or T(*i)and not T(B,C):i=B,C
 if i is T:return[A[:]for A in g]
 B,C=i;D=N+2;e=n+2;o=[A[:]for A in g]
 for S in f(C,C+e):o[B][S]=A;o[B+D-1][S]=A
 for s in f(B,B+D):o[s][C]=A;o[s][C+e-1]=A
 return o

# [209] 8a004b2b.json
* pattern_repetition
* pattern_resizing
* pattern_juxtaposition
* rectangle_guessing
* crop

In [ ]:
show_examples(load_examples(209)['train'])

In [ ]:
%%writefile task209.py
from itertools import product as P, groupby as G
def p(f):
 R,C=len(f),len(f[0])
 s=sorted((y,x)for y,x in P(range(R),range(C))if f[y][x]==4)
 if len(s)!=4:return[]
 y1,x1,y2,x2=s[0]+s[3]
 A=[(y,x)for y,x in P(range(R),range(C))if f[y][x]and not(y1<=y<=y2 and x1<=x<=x2)]
 if not A:return[[4 if(y,x)in((0,0),(0,x2-x1),(y2-y1,0),(y2-y1,x2-x1))else 0 for x in range(x2-x1+1)]for y in range(y2-y1+1)]
 mr,mc=min(y for y,_ in A),min(x for _,x in A)
 B=[(y,x)for y in range(y1,y2+1)for x in range(x1,x2+1)if f[y][x]not in(0,4)]
 if not B:
  q=[[0]*(x2-x1+1)for _ in range(y2-y1+1)]
  q[0][0]=q[0][-1]=q[-1][0]=q[-1][-1]=4
  return q
 def d(f,y1,y2,x1,x2):
  D=float('inf')
  for i in range(y1,y2+1):
   a=[f[i][j]for j in range(x1,x2+1)if f[i][j]not in(0,4)]
   if a:D=min(D,max(sum(1for _ in g)for _,g in G(a)))
  for j in range(x1,x2+1):
   a=[f[i][j]for i in range(y1,y2+1)if f[i][j]not in(0,4)]
   if a:D=min(D,max(sum(1for _ in g)for _,g in G(a)))
  return D if D!=float('inf')else 1
 D=d(f,y1,y2,x1,x2)
 M={}
 for y,x in B:
  n=f[y][x]
  if n not in M:M[n]=[[f[y+i][x+j]for j in range(D)]for i in range(D)]
 fb=B[0];fn=f[fb[0]][fb[1]]
 fa=next(((y-mr,x-mc)for y,x in A if f[y][x]==fn),(-1,-1))
 def b(yo,xo):
  q=[[0]*(x2-x1+1)for _ in range(y2-y1+1)]
  q[0][0]=q[0][-1]=q[-1][0]=q[-1][-1]=4
  for y,x in A:
   m=M.get(f[y][x],[[f[y][x]]*D]*D)
   sy,sx=yo+(y-mr)*D,xo+(x-mc)*D
   for dy,dx in P(range(D),repeat=2):
    ny,nx=sy+dy,sx+dx
    if 0<=ny<len(q)and 0<=nx<len(q[0]):q[ny][nx]=m[dy][dx]
  return q
 yo=(fb[0]-y1)-fa[0]*D if fa!=(-1,-1)else 0
 xo=(fb[1]-x1)-fa[1]*D if fa!=(-1,-1)else 0
 q=b(yo,xo)
 if any(f[y][x]not in(0,4)and not q[y-y1][x-x1]for y,x in P(range(y1,y2+1),range(x1,x2+1))):
  rb,cb,nb=next((y,x,f[y][x])for y in range(y2,y1-1,-1)for x in range(x1,x2+1)if f[y][x]not in(0,4))
  T=[(y,x)for y,x in A if f[y][x]==nb]
  my,mx=max(y for y,_ in T),min(x for _,x in T)
  yo=(rb-y1)-((my-mr)*D+D-1)
  xo=(cb-x1)-(mx-mc)*D
  q=b(yo,xo)
 return q

# [210] 8be77c9e.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(210)['train'])

In [ ]:
%%writefile task210.py
p=lambda j:j+j[::-1]

# [211] 8d5021e8.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(211)['train'])

In [ ]:
%%writefile task211.py
p=lambda j:(j:=[r[::-1]+r for r in j],a:=j[:3][::-1],a+j+a)[-1]

# [212] 8d510a79.json
* draw_line_from_point
* detect_wall
* direction_guessing
* associate_colors_to_bools

In [ ]:
show_examples(load_examples(212)['train'])

In [ ]:
%%writefile task212.py
def p(g,l=len,n=range):
 O,h=l(g),l(g[0]);D=[e for e in n(l(g))if 5in g[e]][0]
 for e in n(O):
  for B in n(h):
   if g[e][B]==1and e<D:
    for r in n(e,-1,-1):g[r][B]=1
   elif g[e][B]==1and e>D:
    for r in n(e,O):g[r][B]=1
   if g[e][B]==2and e<D:
    for r in n(e,D):g[r][B]=2
   elif g[e][B]==2and e>D:
    for r in n(D+1,e):g[r][B]=2
 return g

# [213] 8e1813be.json
* recoloring
* color_guessing
* direction_guesingcrop
* image_within_image

In [ ]:
show_examples(load_examples(213)['train'])

In [ ]:
%%writefile task213.py
def Z(S,t):return len(set([l[t]for l in S]))
def p(s):
 o=enumerate;h,H=len(s),len(s[0]);n=Z(s,0)+Z(s,-1)<len(set(s[0]))+len(set(s[-1]));s=[[t if t!=5else 0for t in t]for t in s]
 for(t,i)in o(s):
  for(l,H)in o(i):
   if n:s[t][l]=max([s[0][l],s[-1][l]])
   else:s[t][l]=max([s[t][0],s[t][-1]])
 if n:s=[[t for t in t if t>0]for t in s];s=s[:len(s[0])]
 else:s=[t for t in s if sum(t)>0];s=[t[:len(s)]for t in s]
 return s

# [214] 8e5a5113.json
* detect_wall
* separate_images
* image_repetition
* image_rotation

In [ ]:
show_examples(load_examples(214)['train'])

In [ ]:
%%writefile task214.py
def p(g,R=range):
	C=[[A for A in A[:3]]for A in g];D=[A[::-1]for A in C[::-1]]
	for A in R(3):
		for B in R(3):g[A][B+4]=C[-(B+1)][A];g[A][B+8]=D[A][B]
	return g

# [215] 8eb1be9a.json
* pattern_repetition
* image_filling

In [ ]:
show_examples(load_examples(215)['train'])

In [ ]:
%%writefile task215.py
p=lambda j:[[r for j,r in enumerate(j)if sum(r)and j%3==i%3][0]for i in range(len(j))]

# [216] 8efcae92.json
* separate_images
* rectangle_guessing
* count_tiles
* take_maximum
* crop

In [ ]:
show_examples(load_examples(216)['train'])

In [ ]:
%%writefile task216.py
R=range
L=len
def p(a):
	K,C=L(a),L(a[0]);E=F=M=N=O=P=0
	for G in R(K):
		H=[0]*C;I=[0]*C
		for D in R(G,K):
			H=[A+(B==2)for(A,B)in zip(H,a[D])];I=[A+(B==0)for(A,B)in zip(I,a[D])];B=J=0
			for A in R(C+1):
				if A<C and I[A]==0:B+=H[A]
				else:
					Q=A-J;S=(D-G+1)*Q
					if Q and(B>E or B==E and S>F):E,F,M,N,O,P=B,S,G,J,D,A-1
					J=A+1;B=0
	return[A[N:P+1]for A in a[M:O+1]]if F else[]


# [217] 8f2ea7aa.json
* crop
* fractal_repetition

In [ ]:
show_examples(load_examples(217)['train'])

In [ ]:
%%writefile task217.py
def p(j,A=range(9)):c,E=next(i for i,r in enumerate(j)if sum(r))//3*3,next(i for i in A if sum(j[y][i]for y in A))//3*3;return[[j[c+y%3][E+x%3]*bool(j[c+y//3][E+x//3])for x in A]for y in A]

# [218] 90c28cc7.json
* crop
* rectangle_guessing
* summarize

In [ ]:
show_examples(load_examples(218)['train'])

In [ ]:
%%writefile task218.py
def p(q):
	r,z=len(q),len(q[0]);d,r,f,G=r,z,-1,-1
	for y in range(r):
		for o in range(z):
			if q[y][o]!=0:
				if y<d:d=y
				if y>f:f=y
				if o<r:r=o
				if o>G:G=o
	if f<0:return[[]]
	N=[tuple(q[y][r:G+1])for y in range(d,f+1)];H=[];z=None
	for o in N:
		if o!=z:H.append(o);z=o
	p=list(zip(*H))if H else[];J=[];z=None
	for o in p:
		if o!=z:J.append(o);z=o
	return[list(y)for y in zip(*J)]

# [219] 90f3ed37.json
* pattern_repetition
* recoloring

In [ ]:
show_examples(load_examples(219)['train'])

In [ ]:
%%writefile task219.py
def p(g):
 if not g or not g[0]:return[A[:]for A in g]
 l=[A[:]for A in g];I,J=len(l),len(l[0]);from collections import Counter as Y
 def P(grid):A=Y(l for A in grid for l in A);return max(A,key=lambda r:A[r])
 def Z(i,h):
  for A in(-1,0,1):
   for l in(-1,0,1):
    if A==0and l==0:continue
    C,D=i+A,h+l
    if 0<=C<I and 0<=D<J:yield(C,D)
 def C(grid):
  A=grid;N=P(A);J,K=len(A),len(A[0]);D,L=set(),[]
  for t in range(J):
   for F in range(K):
    if(t,F)in D:continue
    e=A[t][F]
    if e==N:continue
    H,I=set(),{(t,F)}
    while I:
     M=set()
     for(l,C)in I:
      if(l,C)in D:continue
      O=A[l][C]
      if O==e:H.add((l,C));D.add((l,C));[M.add((A,l))for(A,l)in Z(l,C)if 0<=A<J and 0<=l<K]
     I=M-D
    if H:L.append((e,tuple(sorted(H))))
  return L
 def D(pts):return min(A for(A,l)in pts)
 def K(pts):return min(A for(A,l)in pts),min(A for(l,A)in pts)
 def q(cp):l,A=cp;C,D=K(A);return l,tuple(sorted((A-C,l-D)for(A,l)in A))
 def Q(cp,h):A,l=cp;C,D=h;return A,tuple(sorted((A+C,l+D)for(A,l)in l))
 R=P(l);a=C(l);S={}
 for(L,A)in a:S[A]=L
 T=[(l,A)for(A,l)in S.items()]
 if not T:return[A[:]for A in l]
 t=sorted(T,key=lambda cp:(D(cp[1]),K(cp[1])[1]));M=D(t[0][1]);F=M
 for U in range(M+1,I):
  if any(l[U][A]!=R for A in range(J)):F=U
  elif F>=M:break
 e,N=set(),t[0][0]
 for(L,A)in t:
  if D(A)<=F:e.update(A)
  else:break
 b=min(A for(A,l)in e);c=min(A for(l,A)in e);d=tuple(sorted((A-b,l-c)for(A,l)in e));e=N,d
 def f(I):
  l=None;e,A,C=[],l,-1
  for(J,t)in I:
   H=D(t)
   if H<=F:continue
   I=max(A for(A,l)in t)
   if A is l or H>C+1:
    if A is not l:e.append((N,tuple(sorted(A))))
    A=set(t);C=I
   else:A.update(t);C=max(C,I)
  if A is not l:e.append((N,tuple(sorted(A))))
  return e
 t=f(t);i=[(0,2),(0,1),(0,0),(0,-1),(0,-2),(-1,0)];V=[]
 for(L,A)in t:h=K(A);r=Q(e,h);a=[set(Q(r,A)[1])for A in i];W=set(A);m=[(A,len(A&W),min(A for(A,l)in A)if A else N('inf'))for A in a];D=max(m,key=lambda x:(x[1],-x[2]));n=max(A for(l,A)in A);V.append((D[0]if D[1]>0else W,n))
 C=[A[:]for A in l]
 for(d,p)in V:
  for(O,H)in d:
   if H<=p:continue
   if 0<=O<I and 0<=H<J and C[O][H]==R:C[O][H]=1
 return C

# [220] 913fb3ed.json
* contouring
* associate_colors_to_colors

In [ ]:
show_examples(load_examples(220)['train'])

In [ ]:
%%writefile task220.py
def p(j,A=enumerate):
 F={8:4,2:1,3:6};B=[[A for(B,A)in A(B)]for B in j]
 for(G,H)in A(j):
  for(I,C)in A(H):
   if C:
    for D in range(-1,2):
     for E in range(-1,2):
      try:
       if[D,E]!=[0,0]:B[G+D][I+E]=F[C]
      except:0
 return B

# [221] 91413438.json
* count_tiles
* algebra
* image_repetition

In [ ]:
show_examples(load_examples(221)['train'])

In [ ]:
%%writefile task221.py
R=range
def p(g):
	B=sum(g,[]);A=B.count(0);G=9-A;C=[[0]*(A*3)for B in R(A*3)]
	for D in R(G):
		for E in R(3):
			for F in R(3):
				if g[E][F]:C[E+D//A*3][F+D%A*3]=max(B)
	return C

# [222] 91714a58.json
* find_the_intruder
* remove_noise

In [ ]:
show_examples(load_examples(222)['train'])

In [ ]:
%%writefile task222.py
s=enumerate
def p(n):n=[[h if(o and n[o-1][I]==h)+(o+1<len(n)and n[o+1][I]==h)+(I and r[I-1]==h)+(I+1<len(n)and r[I+1]==h)>1else 0for I,h in s(r)]for o,r in s(n)];R=sum(n,[]);R=sorted([[R.count(r),r]for r in set(R)if r>0]);n=[[0if r!=R[-1][1]else r for r in r]for r in n];return n

# [223] 9172f3a0.json
* image_resizing

In [ ]:
show_examples(load_examples(223)['train'])

In [ ]:
%%writefile task223.py
def p(j):return[[float(e)for e in r for _ in(0,1,2)]for r in j for _ in(0,1,2)]

# [224] 928ad970.json
* rectangle_guessing
* color_guessing
* draw_rectangle

In [ ]:
show_examples(load_examples(224)['train'])

In [ ]:
%%writefile task224.py
def p(t,n=len,R=range,h=max,N=min):
 n,E,r,D=n(t),n(t[0]),[],[];O=[n for n in set(sum(t,[]))if n not in[0,5]][0]
 for n in R(n):
  for o in R(E):
   if t[n][o]==5:r+=[n];D+=[o]
 for n in R(n):
  for o in R(E):
   if n in[N(r)+1,h(r)-1]and N(D)+1<=o<=h(D)-1:t[n][o]=O
   if o in[N(D)+1,h(D)-1]and N(r)+1<=n<=h(r)-1:t[n][o]=O
 return t

# [225] 93b581b8.json
* pattern_expansion
* color_guessing
* out_of_boundary

In [ ]:
show_examples(load_examples(225)['train'])

In [ ]:
%%writefile task225.py
def p(g,L=len,T=range):
 r,i=L(g),L(g[0]);n=[n for n in T(r)if L(set(g[n]))>1][0];B=[B for B in T(i)if g[n][B]>0][0];I=[[-2,-2,g[n+1][B+1]],[2,-2,g[n][B+1]],[-2,2,g[n+1][B]],[2,2,g[n][B]]]
 for H in T(n,n+2):
  for F in T(B,B+2):
   for(G,O,J)in I:
    if 0<=G+H<r and 0<=O+F<i:g[G+H][O+F]=J
 return g

# [226] 941d9a10.json
* detect_grid
* loop_filling
* pairwise_analogy

In [ ]:
show_examples(load_examples(226)['train'])

In [ ]:
%%writefile task226.py
def f(l,i,t,d):
 if not(0<=i<len(l)and 0<=t<len(l[0])):return
 if l[i][t]:return
 l[i][t]=d
 for(e,a)in[(0,-1),(0,1),(-1,0),(1,0)]:f(l,i+e,t+a,d)
def p(l):
 i,e=len(l),len(l[0]);f(l,0,0,1)
 for a in range(4):f(l,i//2-1+a%2,e//2-1+a//2,2)
 f(l,i-1,e-1,3);return l

# [227] 94f9d214.json
* separate_images
* take_complement
* pattern_intersection

In [ ]:
show_examples(load_examples(227)['train'])

In [ ]:
%%writefile task227.py
p=lambda g,r=range:[[2*(g[i][j]+g[i+4][j]<1)for j in r(4)]for i in r(4)]

# [228] 952a094c.json
* rectangle_guessing
* inside_out

In [ ]:
show_examples(load_examples(228)['train'])

In [ ]:
%%writefile task228.py
i=lambda e:[[e[L][B]for L in range(len(e))]for B in range(len(e[0]))]
def J(e):B=[e for(e,B)in enumerate(e)if any(B)];return B[0],B[-1]
def p(e):
 B,L=J(e);n,s=J(i(e))
 def r(l,J,a,L):e[l][J],e[a][L]=e[a][L],e[l][J]
 r(B+1,n+1,L+1,s+1);r(B+1,s-1,L+1,n-1);r(L-1,n+1,B-1,s+1);r(L-1,s-1,B-1,n-1);return e

# [229] 9565186b.json
* separate_shapes
* count_tiles
* recoloring
* take_maximum
* associate_color_to_bools

In [ ]:
show_examples(load_examples(229)['train'])

In [ ]:
%%writefile task229.py
def p(t):e=sum(t,[]);B=e.count;C=max(map(B,e));return[[e if B(e)==C else 5for e in e]for e in t]

# [230] 95990924.json
* pattern_expansion

In [ ]:
show_examples(load_examples(230)['train'])

In [ ]:
%%writefile task230.py
def p(j):
 t,d=len(j),len(j[0])
 for a in range(t-1):
  for n in range(d-1):
   if j[a][n]==j[a][n+1]==j[a+1][n]==j[a+1][n+1]==5:
    if a>0and n>0:j[a-1][n-1]=1
    if a>0and n+2<d:j[a-1][n+2]=2
    if a+2<t and n>0:j[a+2][n-1]=3
    if a+2<t and n+2<d:j[a+2][n+2]=4
 return j

# [231] 963e52fc.json
* image_expansion
* pattern_expansion

In [ ]:
show_examples(load_examples(231)['train'])

In [ ]:
%%writefile task231.py
p=lambda g,r=range:[[g[i%5][j%6]for j in r(len(g[0])*2)]for i in r(len(g))]

# [232] 97999447.json
* draw_line_from_point
* pattern_expansion

In [ ]:
show_examples(load_examples(232)['train'])

In [ ]:
%%writefile task232.py
def p(j,A=enumerate):
 for c,E in A(j):
  k,W,l=0,[],0
  for J,a in A(E):
   if a>0:W=[a,5]*20;l=1
   if l:j[c][J]=W[k];k+=1
 return j

# [233] 97a05b5b.json
* pattern_moving
* pattern_juxtaposition
* crop
* shape_guessing

In [ ]:
show_examples(load_examples(233)['train'])

In [ ]:
%%writefile task233.py
def p(u):
 i=range(9);c={};f=lambda a:[a(p,s,[u[p+a//3][s+a%3]for a in i])for p in range(len(u)-2)for s in range(len(u[0])-2)]
 def p(p,s,n):
  if len(r:={*n})==2and r&{0,2}=={2}:
   n=tuple(a==2for a in n);k={n:sum(r)-2}
   for a in range(4):c[n]=k;n=tuple(n[a%3*3+2-a//3]for a in i)
   for a in i:u[p+a//3][s+a%3]=0
 f(p);u=[*map(list,zip(*filter(any,[*map(list,zip(*filter(any,u)))])))];[f(lambda p,s,n:(t:=c.get(n:=tuple(a==0for a in n)))and(a or n in t)and(r:=t.popitem()[1],[u[p+a//3].__setitem__(s+a%3,(r,2)[n[a]])for a in i]))for a in range(2)];return u

# [234] 98cf29f8.json
* pattern_moving
* bring_patterns_close

In [ ]:
show_examples(load_examples(234)['train'])

In [ ]:
%%writefile task234.py
def p(g,X=enumerate):
 L=None;T=len(g);U=len(g[0]);from collections import Counter as i;V=i(B for A in g for B in A);W=max(V,key=V.get);C={}
 for(A,j)in X(g):
  for(B,I)in X(j):
   if I!=W:C.setdefault(I,set()).add((A,B))
 if not C:return g
 def X(S):A=min(A for(A,B)in S);B=max(A for(A,B)in S);C=min(A for(B,A)in S);D=max(A for(B,A)in S);return A,B,C,D
 M=L;Y=L
 for(I,Z)in C.items():
  a,b,c,d=X(Z)
  if(b-a+1)*(d-c+1)==len(Z):M=I;Y=a,b,c,d;break
 if M is L or len(C)==1:return g
 N=next(A for A in C if A!=M);e=C[N];O=set()
 for(A,B)in e:
  k=(A-1,B-1),(A-1,B+1),(A+1,B-1),(A+1,B+1)
  if all(0<=A<T and 0<=B<U and g[A][B]==N for(A,B)in k):O.add((A,B))
 if not O:return g
 l,m,n,o=X(O);E,J,D,F=l-1,m+1,n-1,o+1;P=[A[:]for A in g]
 for(A,B)in e:P[A][B]=W
 Q,f,K,R=Y
 def S(x,i,_,z):return not(i<_ or x>z)
 p=S(D,F,K,R);G=H=0;q=E+(J-E+1)//2;r=D+(F-D+1)//2;s=Q+(f-Q+1)//2;t=K+(R-K+1)//2
 if p:G=1if q<s else-1
 else:H=1if r<t else-1
 def u(a,b):A,B,C,D=a;E,F,G,H=b;I=S(C,D,G,H)and(E-B==1or A-F==1);J=S(A,B,E,F)and(G-D==1or C-H==1);return I or J
 v=G;w=H;h=0
 while not u((E,J,D,F),(Q,f,K,R))and h<42:h+=1;E+=G;J+=G;D+=H;F+=H;v+=G;w+=H
 for A in range(max(0,E),min(T,J+1)):
  for B in range(max(0,D),min(U,F+1)):P[A][B]=N
 return P

# [235] 995c5fa3.json
* take_complement
* detect_wall
* separate_images
* associate_colors_to_images
* summarize

In [ ]:
show_examples(load_examples(235)['train'])

In [ ]:
%%writefile task235.py
p=lambda j:[[(45-j[2][x]-2*j[2][x+1]-4*j[1][x+1])//5]*3for x in range(0,15,5)]

# [236] 99b1bc43.json
* take_complement
* detect_wall
* separate_images
* pattern_intersection

In [ ]:
show_examples(load_examples(236)['train'])

In [ ]:
%%writefile task236.py
def p(j,A=range(4)):
 for c in A:
  for E in A:
   j[c][E]+=j[c+5][E]
   if j[c][E]==3:j[c][E]=0
   elif j[c][E]>0:j[c][E]=3
 return j[:4]

# [237] 99fa7670.json
* draw_line_from_point
* pattern_expansion

In [ ]:
show_examples(load_examples(237)['train'])

In [ ]:
%%writefile task237.py
def p(i,L=len,I=range):
 D,a=len(i),len(i[0])
 for n in I(D):
  L=0
  for C in I(a):
   if i[-(n+1)][C]>0:L=i[-(n+1)][C]
   i[-(n+1)][C]=L
  L=0
  for n in I(D):
   if i[n][-1]>0:L=i[n][-1]
   i[n][-1]=L
 return i

# [238] 9aec4887.json
* pattern_moving
* x_marks_the_spot
* crop
* recoloring
* color_guessing

In [ ]:
show_examples(load_examples(238)['train'])

In [ ]:
%%writefile task238.py
def p(g):
 def S(grid):
  B=grid
  if not B or not B[0]:return 0
  A={}
  for D in B:
   for C in D:A[C]=A.get(C,0)+1
  return max(A,key=A.get)
 def N(grid,N,h,t):
  L=N;A=grid
  if not A or not A[0]:return[]
  M,N=len(A),len(A[0]);O=S(A)if t else None;P=[(-1,0),(1,0),(0,-1),(0,1)]
  if h:P+=[(-1,-1),(-1,1),(1,-1),(1,1)]
  D,Q=set(),[]
  for E in range(M):
   for F in range(N):
    if(E,F)in D:continue
    R=A[E][F]
    if R==O:continue
    e,H=set(),[(E,F)]
    while H:
     B,C=H.pop()
     if(B,C)in D:continue
     I=A[B][C]
     if L and I!=R or not L and I==O:continue
     D.add((B,C));e.add((I,(B,C)))
     for(T,U)in P:
      J,K=B+T,C+U
      if 0<=J<M and 0<=K<N and(J,K)not in D:H.append((J,K))
    if e:Q.append(e)
  return Q
 def D(obj):
  A=obj
  if not A:return set()
  B=next(iter(A))
  if isinstance(B,tuple)and len(B)==2and isinstance(B[1],tuple):return{(A,B)for(C,(A,B))in A}
  return set(A)
 def E(patch):
  A=D(patch)
  if not A:return(0,0),(0,0)
  B=min(A for(A,B)in A);C=min(A for(B,A)in A);E=max(A for(A,B)in A);F=max(A for(B,A)in A);return(B,C),(E,F)
 def I(obj):
  A=obj
  if not A:return A
  (B,C),D=E(A);return{(A,(D-B,E-C))for(A,(D,E))in A}
 def O(patch,d):
  A=patch;C,D=d
  if not A:return A
  B=next(iter(A))
  if isinstance(B,tuple)and len(B)==2and isinstance(B[1],tuple):return{(A,(B+C,E+D))for(A,(B,E))in A}
  return{(A+C,B+D)for(A,B)in A}
 def P(a,b):
  A,B=a;C,D=b;E,F=min(A,C),min(B,D);e,H=max(A,C),max(B,D)
  if A==C:return{(A,B)for B in range(F,H+1)}
  if B==D:return{(A,B)for A in range(E,e+1)}
  if C-A==D-B:return{(A,B)for(A,B)in zip(range(E,e+1),range(F,H+1))}
  if C-A==B-D:return{(A,B)for(A,B)in zip(range(E,e+1),range(H,F-1,-1))}
  return set()
 def Q(idx):
  A=D(idx)
  if not A:return set()
  (B,C),(B,F)=E(A);e=C+F;return{(A,e-B)for(A,B)in A}
 F=N(g,N=False,h=True,t=True)
 if not F:return g
 R={(8,(A,C))for(A,B)in enumerate(g)for(C,D)in enumerate(B)if D==8};T=lambda o:len(D(o));U=R or min(F,key=lambda o:len({A for(A,B)in o}));l=[A for A in F if{A for(A,B)in A}!={8}]or F;e=max(l,key=T,default=None)
 if not e:return g
 J={(A,B)for(A,B)in e if A!=8};(W,X),(H,Z)=E(J or e);a=[A[X:Z+1]for A in g[W:H+1]];H=D(O(I(U),(1,1)));K=list(I(J or e));A=[list(A)for A in a]
 for(B,C)in H:
  if 0<=B<len(A)and 0<=C<len(A[0]):A[B][C]=min(K,key=lambda t:abs(t[1][0]-B)+abs(t[1][1]-C))[0]if K else 0
 L=E(H)
 if L!=((0,0),(0,0))and H:
  (b,N),(d,e)=L;M=P((b,N),(d,e));f=H&(M|Q(M))
  for(B,C)in f:
   if 0<=B<len(A)and 0<=C<len(A[0]):A[B][C]=8
 return A

# [239] 9af7a82c.json
* separate_images
* count_tiles
* summarize
* order_numbers

In [ ]:
show_examples(load_examples(239)['train'])

In [ ]:
%%writefile task239.py
from collections import*
def p(j,A=range):
 c=Counter([x for r in j for x in r]).most_common(9);E,k=c[0][1],len(c);j=[[0for _ in A(k)]for _ in A(E)]
 for W in A(k):
  for l in A(c[W][1]):j[l][W]=c[W][0]
 return j

# [240] 9d9215db.json
* pattern_expansion
* pattern_reflection
* pattern_rotation

In [ ]:
show_examples(load_examples(240)['train'])

In [ ]:
%%writefile task240.py
def p(g,f=range):
 if g[17][1]|g[17][17]:g.reverse()
 if g[1][17]:[A.reverse()for A in g]
 d=[[g[A][d]|g[~A][d]|g[A][~d]|g[~A][~d]for d in f(19)]for A in f(19)]
 for A in(1,3,5):
  if(D:=g[A][A+2]):
   for C in f(A+2,18-A,2):d[C][A]=d[C][~A]=d[A][C]=d[~A][C]=D
 return d

# [241] 9dfd6313.json
* image_reflection
* diagonal_symmetry

In [ ]:
show_examples(load_examples(241)['train'])

In [ ]:
%%writefile task241.py
p=lambda j:[*zip(*j)]

# [242] 9ecd008a.json
* image_filling
* pattern_expansion
* pattern_reflection
* pattern_rotation
* crop

In [ ]:
show_examples(load_examples(242)['train'])

In [ ]:
%%writefile task242.py
def p(g,i=len,o=range):
 t,E,C,i=i(g),i(g[0]),[],[]
 for n in o(t//2+1):
  for B in o(E):
   if g[n][B]==0:g[n][B]=g[-(n+1)][B];C+=[n];i+=[B]
   if g[-(n+1)][B]==0:g[-(n+1)][B]=g[n][B];C+=[t-(n+1)];i+=[B]
 for n in o(t):
  for B in o(E//2+1):
   if g[n][B]==0:g[n][B]=g[n][-(B+1)];C+=[n];i+=[B]
   if g[n][-(B+1)]==0:g[n][-(B+1)]=g[n][B];C+=[n];i+=[E-(B+1)]
 g=g[min(C):max(C)+1];g=[n[min(i):max(i)+1]for n in g];return g

# [243] 9edfc990.json
* background_filling
* holes

In [ ]:
show_examples(load_examples(243)['train'])

In [ ]:
%%writefile task243.py
def p(e,p=len,u=range):
 f,f=p(e),p(e[0])
 for a in u(25):
  for a in u(f):
   for l in u(f):
    if e[a][l]==0:
     if l+1<f:
      if e[a][l+1]==1:e[a][l]=1
     if a+1<f:
      if e[a+1][l]==1:e[a][l]=1
     if l-1>=0:
      if e[a][l-1]==1:e[a][l]=1
     if a-1>=0:
      if e[a-1][l]==1:e[a][l]=1
 return e

# [244] 9f236235.json
* detect_grid
* summarize
* image_reflection

In [ ]:
show_examples(load_examples(244)['train'])

In [ ]:
%%writefile task244.py
def p(a,o=range):i,n=len(a),len(a[0]);r=[-1]+[r for r in o(i)if len({*a[r]})==1]+[i];r=[-1]+[r for r in o(n)if len({a[i][r]for i in o(i)})==1]+[n];r=[[a[i+1][r+1]for(r,n)in zip(r,r[1:])if r+1<n-1]for(i,n)in zip(r,r[1:])if i+1<n-1];return[r[::-1]for r in r]

# [245] a1570a43.json
* pattern_moving
* rectangle_guessing
* x_marks_the_spot

In [ ]:
show_examples(load_examples(245)['train'])

In [ ]:
%%writefile task245.py
E=enumerate
t=range
s=len
def p(g):
 n=[[H,l]for(l,B)in E(g)for(H,D)in E(B)if D==3];B=sum(n,[]);H=B[::2];D=B[1::2];l=[min(D)+1,max(D)-1,min(H)+1,max(H)-1];n=[[H,l]for(l,B)in E(g)for(H,D)in E(B)if D==2];B=sum(n,[]);H=B[::2];D=B[1::2];l[0]-=min(D);l[1]-=max(D);l[2]-=min(H);l[3]-=max(H);n=[l[:]for l in g];g=[[0if l!=3else 3for l in l]for l in g]
 for H in t(s(g)):
  for r in t(s(g[0])):
   if n[H][r]==2:g[H-min([l[0],0])+max([l[1],0])][r-min([l[2],0])+max([l[3],0])]=2
 return g

# [246] a2fd1cf0.json
* connect_the_dots

In [ ]:
show_examples(load_examples(246)['train'])

In [ ]:
%%writefile task246.py
def p(j):
 D=range;l=[t[:]for t in j]
 for t in D(len(j)):
  for B in D(len(j[0])):
   if j[t][B]==2:F,r=t,B
   if j[t][B]==3:o,G=t,B
 C=1if G>r else-1
 for B in D(r+C,G+C,C):l[F][B]=8
 C=1if o>F else-1
 for t in D(F+C,o,C):l[t][G]=8
 return l

# [247] a3325580.json
* separate_shapes
* count_tiles
* take_maximum
* summarize
* remove_intruders

In [ ]:
show_examples(load_examples(247)['train'])

In [ ]:
%%writefile task247.py
def p(g):B=sum(g,[]);C=max([B.count(A)for A in set(B)if A>0]);A=sum(map(list,zip(*g[::-1])),[]);A=[B for(C,B)in enumerate(A)if A.index(B)==C];g=[[A for A in A if B.count(A)==C]]*C;return g

# [248] a3df8b1e.json
* pattern_expansion
* draw_line_from_point
* diagonals
* bounce

In [ ]:
show_examples(load_examples(248)['train'])

In [ ]:
%%writefile task248.py
def p(j):
	A=[l[:]for l in j];c,E=len(j),len(j[0]);k,W,l=c-1,0,1
	while k>=0:
		A[k][W]=1
		if 0<=W+l<E:k-=1;W+=l
		else:k-=1;l=-l;W+=l
	return A

# [249] a416b8f3.json
* image_repetition

In [ ]:
show_examples(load_examples(249)['train'])

In [ ]:
%%writefile task249.py
p=lambda j:[E*2for E in j]

# [250] a48eeaf7.json
* pattern_moving
* bring_patterns_close
* gravity
* direction_guessing

In [ ]:
show_examples(load_examples(250)['train'])

In [ ]:
%%writefile task250.py
from itertools import*
a=len
o=range
l=list(product([0,1,-1],repeat=2))
def p(i):
 D,D=a(i),a(i[0])
 for r in o(D):
  for c in o(D):
   if i[r][c]==2:
    for(e,O)in l:
     if 0<=r+e<D and 0<=c+O<D:
      for n in o(20):
       if 0<=r+e*n<D and 0<=c+O*n<D:
        W=i[r+e*n][c+O*n]
        if W==5and i[r+e][c+O]==0:i[r+e][c+O]=5;i[r+e*n][c+O*n]=0
 return i

# [251] a5313dff.json
* loop_filling

In [ ]:
show_examples(load_examples(251)['train'])

In [ ]:
%%writefile task251.py
def p(k,i=range):
	y,r=len(k),len(k[0]);g=[[0]*r for y in i(y)];u=[]
	for d in i(y):
		for f in i(r):
			if d*f==0 or d==y-1or f==r-1:
				if k[d][f]==0:g[d][f]=1;u.append((d,f))
	while u:
		o,h=u.pop(0)
		for(x,a)in[(-1,0),(1,0),(0,-1),(0,1)]:
			d,f=o+x,h+a
			if 0<=d<y and 0<=f<r and k[d][f]==0and not g[d][f]:g[d][f]=1;u.append((d,f))
	i=[[k[y][r]if k[y][r]!=0 or g[y][r]else 1for r in i(r)]for y in i(y)];return i

# [252] a5f85a15.json
* recoloring
* pattern_modification
* pairwise_analogy

In [ ]:
show_examples(load_examples(252)['train'])

In [ ]:
%%writefile task252.py
def p(j,A=range):
 c=len(j)
 for E in A(c):
  for k,W in zip(A(1,c,2),A(E+1,c,2)):
   if j[0][E]:j[k][W]=4
   if j[E][0]:j[W][k]=4
 return j

# [253] a61ba2ce.json
* pattern_moving
* bring_patterns_close
* crop
* jigsaw

In [ ]:
show_examples(load_examples(253)['train'])

In [ ]:
%%writefile task253.py
def p(e):
 l=len(e)-1;s=[0]*16
 for C in range(l):
  for D in range(l):
   if(n:=e[C][D])and e[C+1][D]==n and e[C][D+1]==n:s[0]=s[4]=s[1]=n
   if n and e[C+1][D]==n and e[C+1][D+1]==n:s[8]=s[12]=s[13]=n
   if n and e[C][D+1]==n and e[C+1][D+1]==n:s[2]=s[3]=s[7]=n
   if(n:=e[C+1][D+1])and e[C+1][D]==n and e[C][D+1]==n:s[11]=s[14]=s[15]=n
 return[s[n:n+4]for n in(0,4,8,12)]

# [254] a61f2674.json
* separate_shapes
* count_tiles
* take_maximum
* take_minimum
* recoloring
* associate_colors_to_ranks
* remove_intruders

In [ ]:
show_examples(load_examples(254)['train'])

In [ ]:
%%writefile task254.py
def p(j,e=range):
 a,o=len(j),len(j[0]);r=[0for e in e(o)]
 for n in e(o):
  for o in e(a):
   if j[o][n]>0:r[n]+=1
   j[o][n]=0
 i=min([e for e in r if e>0]);n=r.index(i)
 for o in e(r[n]):j[-(o+1)][n]=2
 n=r.index(max(r))
 for o in e(r[n]):j[-(o+1)][n]=1
 return j

# [255] a64e4611.json
* background_filling
* rectangle_guessing

In [ ]:
show_examples(load_examples(255)['train'])

In [ ]:
%%writefile task255.py
def p(g):
	N='h';J,D=len(g),len(g[0]);S=[[not g[A][B]and all(not(0<=A+C<J and 0<=B+E<D)or not g[A+C][B+E]for C in(-1,0,1)for E in(-1,0,1)if C or E)for B in range(D)]for A in range(J)];K=[0]*D;E=[]
	for O in range(J):
		for P in range(D):K[P]=S[O][P]and K[P]+1
		for L in range(D):
			C=K[L]
			if C:
				for B in range(L,D):
					if not K[B]:break
					C=min(C,K[B]);[E.append((O-A+1,L,O,B,(B-L+1)*A))for A in range(1,C+1)]
	if not E:return[[*A]for A in g]
	F,I=lambda x:x[3]-x[1]+1,lambda x:x[2]-x[0]+1;E.sort(key=lambda r:(-r[4],r[0],r[1]));G=next((A for A in E if F(A)>I(A)),0);H=next((A for A in E if I(A)>F(A)),0);A=G and H and(G if F(G)>I(H)else H if F(G)<I(H)else G if G[4]>=H[4]else H)or G or H or E[0];M=[[*A]for A in g]
	def Q(a):
		for A in range(a[0],a[2]+1):M[A][a[1]:a[3]+1]=[3]*F(a)
	T=lambda a,b:a[1]<=b[3]and b[1]<=a[3]and a[0]<=b[2]and b[0]<=a[2];C='vh'[F(A)<=I(A)];U=lambda a:a[1]==0 or a[3]==D-1 if C==N else a[0]==0 or a[2]==J-1;V=lambda a:(a[3]+1==A[1]or a[1]-1==A[3])and a[0]<=A[2]and a[2]>=A[0]if C==N else(a[2]+1==A[0]or a[0]-1==A[2])and a[1]<=A[3]and a[3]>=A[1];W=lambda a:all(not(0<=A<J and 3 in M[A][a[1]:a[3]+1])for A in(a[0]-1,a[2]+1))if C==N else all(not(0<=A<D and 3 in[*zip(*M[a[0]:a[2]+1])][A])for A in(a[1]-1,a[3]+1));Q(A);R=[A]
	while 1:
		for B in sorted(E,key=lambda r:(not U(r),-r[4],r[0],r[1])):
			if(F,I)[C>N](B)<5 or not W(B)or not V(B)or any(T(B,A)for A in R):continue
			Q(B);R+=[B];break
		else:break
	return M

# [256] a65b410d.json
* pattern_expansion
* count_tiles
* associate_colors_to_ranks

In [ ]:
show_examples(load_examples(256)['train'])

In [ ]:
%%writefile task256.py
def p(j,A=range):
 c=len(j)
 for E in A(c):
  if j[E][0]==2:
   k=0
   while k<c and j[E][k]==2:k+=1
   for W in A(c):
    for l in A((k+E-W)*(W!=E)):j[W][l]=3-2*(W>E)
 return j

# [257] a68b268e.json
* detect_grid
* separate_images
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(257)['train'])

In [ ]:
%%writefile task257.py
def p(g,E=len,a=range):
 e,n=E(g),E(g[0])
 for l in a(4):
  for o in a(4):
   if g[l][o]==0:
    if g[l][o+5]>0:g[l][o]=g[l][o+5]
   if g[l][o]==0:
    if g[l+5][o]>0:g[l][o]=g[l+5][o]
   if g[l][o]==0:
    if g[l+5][o+5]>0:g[l][o]=g[l+5][o+5]
 return[l[:4]for l in g[:4]]

# [258] a699fb00.json
* pattern_expansion
* connect_the_dots

In [ ]:
show_examples(load_examples(258)['train'])

In [ ]:
%%writefile task258.py
def p(j):
 for A in j:
  for c in range(len(A)-2):
   if A[c]&A[c+2]:A[c+1]=2
 return j

# [259] a740d043.json
* crop
* detect_background_color
* recoloring

In [ ]:
show_examples(load_examples(259)['train'])

In [ ]:
%%writefile task259.py
def p(j,H=range):
 r,l=len(j),len(j[0]);a,E,F,d=r,0,l,0
 for S in H(r):
  for C in H(l):
   if j[S][C]-1:
    if S<a:a=S
    if S>E:E=S
    if C<F:F=C
    if C>d:d=C
 return[[H-(H==1)for H in H[F:d+1]]for H in j[a:E+1]]

# [260] a78176bb.json
* draw_parallel_line
* direction_guessing
* remove_intruders

In [ ]:
show_examples(load_examples(260)['train'])

In [ ]:
%%writefile task260.py
a=len
H=range
e=[[0,0],[0,1],[1,0],[1,1]]
def p(n):
 F,G=a(n),a(n[0]);I=[A for A in set(sum(n,[]))if A not in[0,5]][0]
 for B in H(F-1):
  for C in H(G-1):
   r=[n[B+A][C+s]for(A,s)in e]
   if r.count(5)==1and sum(r)==5:
    for s in H(2):
     for D in H(2):
      if n[s+B][D+C]==5:
       for A in H(-10,10):
        if r[2]==5:
         if 0<=s+B-A-1<F and 0<=D+C-A+1<G:n[s+B-A-1][D+C-A+1]=I
        elif 0<=s+B-A+1<F and 0<=D+C-A-1<G:n[s+B-A+1][D+C-A-1]=I
 n=[[A if A!=5else 0for A in A]for A in n];return n

# [261] a79310a0.json
* pattern_moving
* recoloring
* pairwise_analogy

In [ ]:
show_examples(load_examples(261)['train'])

In [ ]:
%%writefile task261.py
def p(j):j=[j[-1]]+j[:len(j)-1];j=[[2if C==8else C for C in R]for R in j];return j

# [262] a85d4709.json
* separate_images
* associate_colors_to_images
* summarize

In [ ]:
show_examples(load_examples(262)['train'])

In [ ]:
%%writefile task262.py
p=lambda j:[[[2,4,3][r.index(5)]]*3for r in j]

# [263] a87f7484.json
* separate_images
* find_the_intruder
* crop

In [ ]:
show_examples(load_examples(263)['train'])

In [ ]:
%%writefile task263.py
def p(t):
 A=range;n=[[[t[l+n*3][A+a*3]for A in A(3)]for l in A(3)]for n in A(len(t)//3)for a in A(len(t[0])//3)]
 for a in n:
  if[tuple(tuple(n[a][A]==0for A in A(3))for a in A(3))for n in n].count(tuple(tuple(a[n][A]==0for A in A(3))for n in A(3)))==1:return a

# [264] a8c38be5.json
* pattern_moving
* jigsaw
* crop

In [ ]:
show_examples(load_examples(264)['train'])

In [ ]:
%%writefile task264.py
L=len
R=range
B=[1,4,9,16,25,36,49,64,81]
S=[-1,0,1]
P=[[x,y]for x in S for y in S]
Z=[264,246,236,194,285,134,156,66,104]
def p(g):
 h,w=L(g),L(g[0])
 X=[[0]*9for _ in R(9)]
 for r in R(1,h-1):
  for c in R(1,w-1):
    M=[g[r+y][c+x]for y,x in P]
    f=sum([B[i]for i in R(9)if M[i]==5])
    if f in Z and 0not in M:
     j=Z.index(f)
     for y in R(3):
      for x in R(3):X[y+(j//3*3)][x+(j%3*3)]=g[r-1+y][c-1+x]
 return X

# [265] a8d7556c.json
* recoloring
* rectangle_guessing

In [ ]:
show_examples(load_examples(265)['train'])

In [ ]:
%%writefile task265.py
def p(d):
 S,D=len(d),len(d[0]);r=[o[:]for o in d]
 for B in range(S-1):
  n=d[B];h=d[B+1]
  for o in range(D-1):
   if n[o]==n[o+1]==h[o]==h[o+1]==0:r[B][o]=r[B][o+1]=r[B+1][o]=r[B+1][o+1]=2
 B,o=8,12
 if 0<=B<S and 0<=o<D and r[B][o]==2and B-1>=0and B+1<S and o-1>=0and d[B-1][o-1]==0and d[B][o-1]==0and d[B+1][o]==0:r[B][o]=d[B][o];r[B+1][o]=d[B+1][o]
 return r

# [266] a9f96cdd.json
* replace_pattern
* out_of_boundary

In [ ]:
show_examples(load_examples(266)['train'])

In [ ]:
%%writefile task266.py
def p(j):
 A=sum(j,[]).index(2);c,E=divmod(A,5);j[c][E]=0
 if c*E:j[c-1][E-1]=3
 if c<2and E:j[c+1][E-1]=8
 if E<4and c:j[c-1][E+1]=6
 if c<2and E<4:j[c+1][E+1]=7
 return j

# [267] aabf363d.json
* recoloring
* color_guessing
* remove_intruders

In [ ]:
show_examples(load_examples(267)['train'])

In [ ]:
%%writefile task267.py
def p(j):A=j[6][0];c=[[r and A for r in X]for X in j];c[6][0]=0;return c

# [268] aba27056.json
* pattern_expansion
* draw_line_from_point
* diagonals

In [ ]:
show_examples(load_examples(268)['train'])

In [ ]:
%%writefile task268.py
def p(g,Q=range):
 G=len(g);K=lambda g:[*map(list,zip(*g[::-1]))]
 for O in Q(4):
  l=[(A,T)for A in Q(G)for T in Q(G)if g[A][T]];L,M=zip(*l);T,H,D,e=min(L),max(L),min(M),max(M);N=g[T][D];o=g[T].count(N)
  if o<g[H].count(N):
   l,a=D+o//2,e-o//2
   for A in Q(H):g[A][l:a+1]=[4]*(a-l+1)
   for A in Q(T+1,H):g[A][D+1:e]=[4]*(e-D-1)
   for C in Q(T+1):
    A=T-C
    if l>=C:g[A][l-C]=4
    if a+C<G:g[A][a+C]=4
   for A in Q(4-O):g=K(g)
   return g
  g=K(g)

# [269] ac0a08a4.json
* image_resizing
* count_tiles
* size_guessing

In [ ]:
show_examples(load_examples(269)['train'])

In [ ]:
%%writefile task269.py
p=lambda j:(A:=sum(c>0for r in j for c in r),[sum(([x]*A for x in r),[])for r in j for _ in range(A)])[1]

# [270] ae3edfdc.json
* bring_patterns_close
* gravity

In [ ]:
show_examples(load_examples(270)['train'])

In [ ]:
%%writefile task270.py
def p(n,r=enumerate):
 i=len(n);D=[[0]*i for A in'd'*i];l={}
 for(B,a)in r(n):
  for(d,A)in r(a):
   if 0<A<3:l[A]=B,d;D[B][d]=A
 for(B,a)in r(n):
  for(d,A)in r(a):
   if A in(3,7):F,t=l[2-(A>3)];D[F+(B>F)-(B<F)][t+(d>t)-(d<t)]=A
 return D

# [271] ae4f1146.json
* separate_images
* count_tiles
* crop

In [ ]:
show_examples(load_examples(271)['train'])

In [ ]:
%%writefile task271.py
def p(g,L=len,R=range):
	G,H=L(g),L(g[0]);B,E=[],0
	for C in R(G-2):
		for A in R(H-2):
			D=g[C][A:A+3]+g[C+1][A:A+3]+g[C+2][A:A+3];F=D.count(1)+D.count(8)/10
			if F>E:B=D[:];E=F
	return[B[:3],B[3:6],B[6:]]

# [272] aedd82e4.json
* recoloring
* separate_shapes
* count_tiles
* take_minimum
* associate_colors_to_bools

In [ ]:
show_examples(load_examples(272)['train'])

In [ ]:
%%writefile task272.py
def p(g):h,w=len(g),len(g[0]);return[[1if g[i][j]and all(g[i+a][j+b]==0for a,b in[(-1,0),(1,0),(0,-1),(0,1)]if 0<=i+a<h and 0<=j+b<w)else g[i][j]for j in range(w)]for i in range(h)]

# [273] af902bf9.json
* ex_nihilo
* x_marks_the_spot

In [ ]:
show_examples(load_examples(273)['train'])

In [ ]:
%%writefile task273.py
E=enumerate
R=range
def p(a):
 D=[n[:]for n in a];e=[[n for(n,e)in E(n)if e==4]for n in a]
 for(F,G)in E(e):
  for(I,n)in E(G):
   for e in G[I+1:]:
    for h in R(F+1,len(a)):
     if a[h][n]==4and a[h][e]==4:
      for o in R(F+1,h):D[o][n+1:e]=[2]*(e-n-1)
 return D

# [274] b0c4d837.json
* measure_length
* associate_images_to_numbers

In [ ]:
show_examples(load_examples(274)['train'])

In [ ]:
%%writefile task274.py
j=lambda A,c:sum(sum(i==c for i in r)for r in A)
def p(A):E=max(j([r],8)for r in A);k=(j(A,5)-E-2)/2-j(A,8)/E;return[[8*(k>0),8*(k>1),8*(k>2)],[0,0,8*(k>3)],[0,0,0]]

# [275] b190f7f5.json
* separate_images
* image_expasion
* color_palette
* image_resizing
* replace_pattern

In [ ]:
show_examples(load_examples(275)['train'])

In [ ]:
%%writefile task275.py
def p(i):
 n=min(len(i),len(i[0]));t,a=[t[:n]for t in i[:n]],[t[-n:]for t in i[-n:]]
 if any(max(n)==8for n in t):t,a=a,t
 return[[t[H//n][r//n]*a[H%n][r%n]//8for r in range(n*n)]for H in range(n*n)]

# [276] b1948b0a.json
* recoloring
* associate_colors_to_colors

In [ ]:
show_examples(load_examples(276)['train'])

In [ ]:
%%writefile task276.py
p=lambda g:[[2*(x==6)or x for x in r]for r in g]

# [277] b230c067.json
* recoloring
* separate_shapes
* find_the_intruder
* associate_colors_to_bools

In [ ]:
show_examples(load_examples(277)['train'])

In [ ]:
%%writefile task277.py
def p(j):
	s,s=len(j),len(j[0]);p=[[0]*s for r in j];o=[]
	for r in range(s):
		for t in range(s):
			if j[r][t]==8 and not p[r][t]:
				v=[(r,t)];p[r][t]=1;l=[]
				while v:
					r,t=v.pop();l.append((r,t))
					for a in(-1,0,1):
						for k in(-1,0,1):
							if a==k==0:continue
							b,x=r+a,t+k
							if 0<=b<s and 0<=x<s and not p[b][x]and j[b][x]==8:p[b][x]=1;v.append((b,x))
				o.append(l)
	if not o:return[r[:]for r in j]
	def d(b):r=min(r for(r,t)in b);t=min(r for(t,r)in b);return tuple(sorted((b-r,x-t)for(b,x)in b))
	i={}
	for r in o:i.setdefault(d(r),[]).append(r)
	c=min(len(r)for r in i.values());l=[t for(b,r)in i.items()if len(r)==c for t in r]
	def t(b):return min(r for(r,t)in b),min(r for(t,r)in b)
	c=min(l,key=t);i=[r[:]for r in j]
	for r in range(s):
		for t in range(s):
			if i[r][t]==8:i[r][t]=1
	for(r,t)in c:i[r][t]=2
	return i


# [278] b27ca6d3.json
* find_the_intruder
* count_tiles
* contouring

In [ ]:
show_examples(load_examples(278)['train'])

In [ ]:
%%writefile task278.py
d=lambda n:[[n[d][r]for d in range(len(n))]for r in range(len(n[0]))]
def o(n,a,o):
 if n[a][o]!=2or n[a][o+1]!=2:return
 for r in range(max(0,a-1),min(len(n),a+2)):
  for d in range(max(0,o-1),min(len(n[0]),o+3)):
   if n[r][d]!=2:n[r][d]=3
def f(n):
 for r in range(len(n)):
  for d in range(len(n[0])-1):o(n,r,d)
def p(n):f(n);n=d(n);f(n);return d(n)

# [279] b2862040.json
* recoloring
* detect_closed_curves
* associate_colors_to_bools

In [ ]:
show_examples(load_examples(279)['train'])

In [ ]:
%%writefile task279.py
from collections import deque
def p(g,f=range):
 E,F=len(g),len(g[0]);G=[[0]*F for A in f(E)];K={}
 for H in f(E):
  for I in f(F):
   if not G[H][I]:
    s=g[H][I];L=deque([(H,I)]);G[H][I]=1;T=[]
    while L:
     A,B=L.popleft();T.append((A,B))
     for(R,N)in((1,0),(-1,0),(0,1),(0,-1)):
      C,D=A+R,B+N
      if 0<=C<E and 0<=D<F and not G[C][D]and g[C][D]==s:G[C][D]=1;L.append((C,D))
    K.setdefault(s,[]).append(set(T))
 R=[A for A in K.get(9,[])if not any(A==0 or B==0 or A==E-1or B==F-1for(A,B)in A)];S=set()
 for J in R:S|=J
 T=[A[:]for A in g];t=K.get(1,[])
 if t and R:
  o=S
  for J in t:
   O=False
   for(A,B)in J:
    for(R,N)in((1,0),(-1,0),(0,1),(0,-1)):
     if(A+R,B+N)in o:O=True;break
    if O:break
   if O:
    for(A,B)in J:T[A][B]=8
 return T

# [280] b527c5c6.json
* pattern_expansion
* draw_line_from_point
* contouring
* direction_guessing
* size_guessing

In [ ]:
show_examples(load_examples(280)['train'])

In [ ]:
%%writefile task280.py
def p(I,t=range):
 def c(p):A={};[A.__setitem__(B,A.get(B,0)+1)for C in p for B in C];return max(A,key=A.get)
 O=c(I);H,J=len(I),len(I[0]);L=[[0]*J for A in t(H)];P=[]
 for A in t(H):
  for B in t(J):
   if L[A][B]or I[A][B]==O:continue
   r=[(A,B)];L[A][B]=1;X=[]
   while r:D,E=r.pop();d=I[D][E];X.append((d,(D,E)));[r.append((A,B))or L[A].__setitem__(B,1)for(A,B)in[(D-1,E),(D+1,E),(D,E-1),(D,E+1)]if 0<=A<H and 0<=B<J and not L[A][B]and I[A][B]!=O]
   P.append(X)
 K=lambda s:list(s)if not s or not(isinstance(next(iter(s)),tuple)and len(next(iter(s)))==2and isinstance(next(iter(s))[1],tuple))else[A for(B,A)in s]
 def F(s):A=K(s);B,C=[A for(A,B)in A],[A for(B,A)in A];return min(B),max(B),min(C),max(C)
 R=lambda s:0if not s else F(s)[1]-F(s)[0]+1;S=lambda s:0if not s else F(s)[3]-F(s)[2]+1;T=lambda s:F(s)[0];Y=lambda s:F(s)[1];A=lambda s:F(s)[2];Z=lambda s:F(s)[3];e=lambda s:S(K(s))==1and R(K(s))==len(K(s))if K(s)else 0
 def f(a,b):A,B,C,D=*a,*b;E,F,G,H=min(A,C),max(A,C)+1,min(B,D),max(B,D)+1;return{(A,B)for B in t(G,H)}if A==C else{(A,B)for A in t(E,F)}if B==D else{(A,B)for(A,B)in zip(t(E,F),t(G,H))}if C-A==D-B else{(A,B)for(A,B)in zip(t(E,F),t(H-1,G-1,-1))}if C-A==B-D else set()
 d=lambda e,direction:f(e,(e[0]+42*direction[0],e[1]+42*direction[1]));V=[]
 for G in P:
  C={B for(A,B)in G if A==2}
  if not C:continue
  N,i,j,k=Y(C)==Y(G),Z(C)==Z(G),T(C)==T(G),A(C)==A(G);l,m=(1if N else 0)+(-1if j else 0),(1if i else 0)+(-1if k else 0);D,E=T(C)+R(C)//2,A(C)+S(C)//2;L=d((D,E),(l,m));V.append(L)
 n={B for A in V for B in A};N=[list(A)for A in I]
 for(A,B)in n:
  if 0<=A<H and 0<=B<J:N[A][B]=2
 W=set()
 for(G,L)in zip(P,V):
  s=e(L);a=min(R(G),S(G))
  for b in t(-(a-1),a):
   if s:W.update({(A,B+b)for(A,B)in L})
   else:W.update({(A+b,B)for(A,B)in L})
 for(A,B)in W:
  if 0<=A<H and 0<=B<J and N[A][B]==O:N[A][B]=3
 return N

# [281] b548a754.json
* pattern_expansion
* pattern_modification
* x_marks_the_spot

In [ ]:
show_examples(load_examples(281)['train'])

In [ ]:
%%writefile task281.py
def p(a):
 l=sum(a,[]);d=len(a[0]);n=[d for(d,n)in enumerate(l)if n];E,s=n[0]//d,n[-1]//d;i=min(n%d for n in n);L=max(n%d for n in n);n,I=sorted({*l}-{0,8},key=l.count);e=[I]+[n]*(L-i-1)+[I]
 for J in range(E,s+1):a[J][i:L+1]=(e,[I]*len(e))[J in(E,s)]
 return a

# [282] b60334d2.json
* replace_pattern

In [ ]:
show_examples(load_examples(282)['train'])

In [ ]:
%%writefile task282.py
p=lambda g,R=range(1,8):(G:=[[0]*9for _ in g],[G[i+a].__setitem__(j+b,(1,5)[a*b])for i in R for j in R if g[i][j]for a in(-1,0,1)for b in(-1,0,1)if a|b])[0]

# [283] b6afb2da.json
* recoloring
* replace_pattern
* rectangle_guessing

In [ ]:
show_examples(load_examples(283)['train'])

In [ ]:
%%writefile task283.py
def a(j,D,n,e,r,k):
 for B in range(n,r+1):
  for C in range(D,e+1):j[B][C]=k
def A(j,D,n,e,r):a(j,D,n,e,r,4);a(j,D+1,n+1,e-1,r-1,2);j[n][D]=j[n][e]=j[r][D]=j[r][e]=1
def p(j):
 l,C=len(j),len(j[0])
 for O in range(l*C):
  D,r=O%C,O//C
  if j[r][D]==5:
   n,B=D,r
   while n<C-1and j[B][n+1]==5:n+=1
   while B<l-1and j[B+1][n]==5:B+=1
   A(j,D,r,n,B)
 return j

# [284] b7249182.json
* pattern_expansion

In [ ]:
show_examples(load_examples(284)['train'])

In [ ]:
%%writefile task284.py
def p(I):
 F=True;E,a=len(I),len(I[0])
 def K(G):return tuple(map(tuple,zip(*G)))
 def G(G):from collections import Counter as A;return A([B for A in G for B in A]).most_common(1)[0][0]
 def N(cells):A=cells;B=[A for(A,B)in A];C=[A for(B,A)in A];return min(B),min(C),max(B),max(C)
 def c(G,bg):
  H,I=len(G),len(G[0]);E=[[False]*I for A in range(H)];J=[]
  for A in range(H):
   for B in range(I):
    if E[A][B]or G[A][B]==bg:continue
    K=G[A][B];h=[(A,B)];E[A][B]=F;i=[]
    while h:
     M,N=h.pop()
     if G[M][N]!=K:continue
     i.append((M,N))
     for(P,r)in((1,0),(-1,0),(0,1),(0,-1)):
      C,D=M+P,N+r
      if 0<=C<H and 0<=D<I and not E[C][D]and G[C][D]==K:E[C][D]=F;h.append((C,D))
    J.append((i,K))
  J.sort(key=lambda t:min(A for(A,B)in t[0]));return J
 def C(a,N):
  A,C=a;B,D=N
  if A==B:E,H=sorted((C,D));return{(A,B)for B in range(E,H+1)}
  if C==D:F,I=sorted((A,B));return{(A,C)for A in range(F,I+1)}
  if B-A==D-C:F,E=(A,C)if A<=B else(B,D);G=abs(B-A);return{(F+A,E+A)for A in range(G+1)}
  if B-A==C-D:F,E=(A,C)if A<=B else(B,D);G=abs(B-A);return{(F+A,E-A)for A in range(G+1)}
  return set()
 def A(G,value,indices):
  D,E=len(G),len(G[0]);A=[list(A)for A in G]
  for(B,C)in indices:
   if 0<=B<D and 0<=C<E:A[B][C]=value
  return tuple(tuple(A)for A in A)
 def d(G,t):
  D,E=len(G),len(G[0]);A=[list(A)for A in G]
  for(F,(B,C))in t:
   if 0<=B<D and 0<=C<E:A[B][C]=F
  return tuple(tuple(A)for A in A)
 def e(indices):A=indices;B=[A for(A,B)in A];C=[A for(B,A)in A];D,E,F,G=min(B),min(C),max(B),max(C);return(D,E),(D,G),(F,E),(F,G)
 f=G(I);h=[(A,B)for A in range(E)for B in range(a)if I[A][B]!=f]
 if h:g,h,i,s=N(h);H=i-g+1>s-h+1
 else:H=F
 J=tuple(tuple(A)for A in(I if H else K(I)));k=G(J);M=c(J,k);(l,N),(m,i)=M[0],M[-1];P=min(l);n=min(m);D=C(P,n);r=max(1,len(D));o=sum(A for(A,B)in D)//r;p=sum(A for(B,A)in D)//r;B=o,p;q=C(P,B);r=A(J,i,D);E=A(r,N,q);R={(B[0],B[1]),(B[0]+1,B[1])};s,t=len(E),len(E[0]);S={(E[A][B],(A,B))for(A,B)in R if 0<=A<s and 0<=B<t};T={(A,(B,C+2))for(A,(B,C))in S}|{(A,(B,C-2))for(A,(B,C))in S};o={(A,B)for(C,(A,B))in T}
 if o:e,o,s,x=e(o);V={(A-1,B)for(A,B)in C(e,o)};W={(A+1,B)for(A,B)in C(s,x)}
 else:V=set();W=set()
 y=d(E,T);z=A(y,N,V);X=A(z,i,W);A0=G(X);Y=A(X,A0,R);return Y if H else K(Y)

# [285] b775ac94.json
* pattern_expansion
* pattern_repetition
* recoloring
* pattern_rotation
* pattern_reflection
* direction_guessing
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(285)['train'])

In [ ]:
%%writefile task285.py
from collections import deque,Counter
def p(g):
	S=None;T=lambda p:Counter([B for A in p for B in A]if isinstance(p,list)and p and isinstance(p[0],list)else[A for(A,B)in p]).most_common(1)[0][0]if p else 0
	def e(G,d=1,b=1):
		E,F=len(G),len(G[0]);K=T(G)if b else S;D=[[0]*F for A in range(E)];N=[];O=(lambda i,j:[(i+A,j+B)for A in(-1,0,1)for B in(-1,0,1)if A or B])if d else lambda i,j:[(i-1,j),(i+1,j),(i,j-1),(i,j+1)]
		for H in range(E):
			for I in range(F):
				if D[H][I]or b and G[H][I]==K:D[H][I]=1;continue
				L=deque([(H,I)]);M=[]
				while L:
					A,B=L.popleft()
					if not(0<=A<E and 0<=B<F)or D[A][B]:continue
					C=G[A][B]
					if b and C==K:D[A][B]=1;continue
					D[A][B]=1;M.append((C,(A,B)))
					for(J,C)in O(A,B):
						if 0<=J<E and 0<=C<F and not D[J][C]and(not b or G[J][C]!=K):L.append((J,C))
				if M:N.append(M)
		return N
	I=lambda p:[A for(B,A)in p];A=lambda I:(min(A for(A,B)in I),min(A for(B,A)in I),max(A for(A,B)in I),max(A for(B,A)in I));f=lambda I:(A(I)[2]-A(I)[0]+1,A(I)[3]-A(I)[1]+1);U=lambda I:(A(I)[0]+(A(I)[2]-A(I)[0])//2,A(I)[1]+(A(I)[3]-A(I)[1])//2)
	def h(a,b):
		A,B=U(a);C,D=U(b)
		if A==C:return 0,1if B<D else-1
		if B==D:return 1if A<C else-1,0
		if A<C:return 1,1if B<D else-1
		if A>C:return-1,1if B<D else-1
		return 0,0
	def i(l,O):A,B=l;C=set(O);return any((A,B)in C for(A,B)in((A-1,B),(A+1,B),(A,B-1),(A,B+1)))
	V=lambda p:[(B,(A(I(p))[0]+A(I(p))[2]-C,D))for(B,(C,D))in p]if p else[];W=lambda p:[(B,(C,A(I(p))[1]+A(I(p))[3]-D))for(B,(C,D))in p]if p else[];D=lambda p,d:[(A,(B+d[0],C+d[1]))for(A,(B,C))in p]
	def L(G,C):
		F,H=len(G),len(G[0]);A=[A[:]for A in G]
		for(B,(D,E))in C:
			if B and 0<=D<F and 0<=E<H:A[D][E]=B
		return A
	def M(G,I):
		for(A,B)in I:
			C=G[A][B]
			if C:return C
		for(A,B)in I:return G[A][B]
		return 0
	g=[list(A)for A in g];j=e(g);C=[A[:]for A in g];N,O,P=[],[],[]
	for B in j:
		if not B:continue
		E=T(B);X=[A for A in B if A[0]==E];k=[A for A in B if A[0]!=E];Y=[A for(B,A)in X];G=[A for(B,A)in k];H=S
		for Q in B:
			if Q[0]==E and i(Q[1],G):H=Q;break
		if H is S:H=X[0]
		l=G+[H[1]]if G else[H[1]];m,n,o,p=A(l);R={(A,B)for A in range(m,o+1)for B in range(n,p+1)};J,K=h(Y,G if G else Y);Z,a=f([A for(B,A)in B]);q,r,s=(Z*J,0),(0,a*K),(Z*J,a*K)
		def F(v):v=-v;return 0if v==0else v+1if v>0else v-1
		t,u,v=(F(J),F(0)),(F(0),F(K)),(F(J),F(K));w=[A for A in D(V(B),q)if A[0]==E];x=[A for A in D(W(B),r)if A[0]==E];y=[A for A in D(V(W(B)),s)if A[0]==E];b,c,d=D(w,t),D(x,u),D(y,v);N+=[(M(g,{A for(B,A)in b if A in R}),A)for(B,A)in b];O+=[(M(g,{A for(B,A)in c if A in R}),A)for(B,A)in c];P+=[(M(g,{A for(B,A)in d if A in R}),A)for(B,A)in d]
	if N:C=L(C,N)
	if O:C=L(C,O)
	if P:C=L(C,P)
	return C

# [286] b782dc8a.json
* pattern_expansion
* maze

In [ ]:
show_examples(load_examples(286)['train'])

In [ ]:
%%writefile task286.py
s=len
r=range
def p(n):
 F,G=s(n),s(n[0]);L=sum(n,[]);L=sorted([[L.count(A),A]for A in set(L)])[:2];l={L[0][1]:L[1][1],L[1][1]:L[0][1]}
 for J in range(50):
  for A in r(F):
   for a in r(G):
    if n[A][a]in l:
     for(D,i)in[[0,1],[0,-1],[1,0],[-1,0]]:
      if 0<=A+D<F and 0<=a+i<G and n[A+D][a+i]==0:n[A+D][a+i]=l[n[A][a]]
 return n

# [287] b8825c91.json
* pattern_completion
* pattern_rotation
* pattern_reflection

In [ ]:
show_examples(load_examples(287)['train'])

In [ ]:
%%writefile task287.py
def p(g):
	C=range(len(g))
	def A():
		for A in C:
			for B in C:
				if g[A][B]==4:g[A][B]=D[A][B]
	D=g[::-1];A();D=[A[::-1]for A in g];A();return g

# [288] b8cdaf2b.json
* pattern_expansion
* draw_line_from_point
* diagonals
* pairwise_analogy

In [ ]:
show_examples(load_examples(288)['train'])

In [ ]:
%%writefile task288.py
def p(g,a=3):
 l=len(g)//2
 for i in range(l+1):
  if g[-2][l-i]<1:g[-a][l-i]=(b:=g[-1][l]);g[-a][l+i]=b;a+=1
 return g

# [289] b91ae062.json
* image_resizing
* size_guessing
* count_different_colors

In [ ]:
show_examples(load_examples(289)['train'])

In [ ]:
%%writefile task289.py
def p(g):A=len({*sum(g,[])})-1;return[[g[B//A][C//A]for C in range(A*3)]for B in range(A*3)]

# [290] b94a9452.json
* crop
* take_negative

In [ ]:
show_examples(load_examples(290)['train'])

In [ ]:
%%writefile task290.py
def p(g):a=[[j for j in i if j]for i in g if sum(i)];b=[*{*sum(a,[])}];return[[b[j==b[0]]for j in i]for i in a]

# [291] b9b7f026.json
* find_the_intruder
* summarize

In [ ]:
show_examples(load_examples(291)['train'])

In [ ]:
%%writefile task291.py
def p(g,e=enumerate):
 for(o,i)in e(g):
  for(s,d)in e(i):
   if d<1and(r:=g[o-1][s])and r==i[s-1]:return[[r]]

# [292] ba26e723.json
* pattern_modification
* pairwise_analogy
* recoloring

In [ ]:
show_examples(load_examples(292)['train'])

In [ ]:
%%writefile task292.py
def p(j):
 for A in j:A[::3]=[6if v else v for v in A[::3]]
 return j

# [293] ba97ae07.json
* pattern_modification
* pairwise_analogy
* rettangle_guessing
* recoloring

In [ ]:
show_examples(load_examples(293)['train'])

In [ ]:
%%writefile task293.py
def p(e):f=lambda a:a[0]and a[:1]*len(a)or a;k=[*map(f,e)];return[k,[*map(list,zip(*map(f,zip(*e))))]][k==e]

# [294] bb43febb.json
* loop_filling
* rettangle_guessing

In [ ]:
show_examples(load_examples(294)['train'])

In [ ]:
%%writefile task294.py
def p(g):
 for i in range(1,9):
  for j in range(1,9):
   if g[i+1][j]*g[i][j+1]*g[i-1][j]*g[i][j-1]:g[i][j]=2
 return g

# [295] bbc9ae5d.json
* pattern_expansion
* image_expansion

In [ ]:
show_examples(load_examples(295)['train'])

In [ ]:
%%writefile task295.py
def p(g):r=g[0];l=sum(map(bool,r));g+=(r[:l]+r[:1]*i+[0]*(len(r)-l-i)for i in range(1,len(r)//2));return g

# [296] bc1d5164.json
* pattern_moving
* pattern_juxtaposition
* crop
* pairwise_analogy

In [ ]:
show_examples(load_examples(296)['train'])

In [ ]:
%%writefile task296.py
def p(g):return[[g[i][j]or g[i+2][j]or g[i][j+4]or g[i+2][j+4]for j in range(3)]for i in range(3)]

# [297] bd4472b8.json
* detect_wall
* pattern_expansion
* ex_nihilo
* color_guessing
* color_palette

In [ ]:
show_examples(load_examples(297)['train'])

In [ ]:
%%writefile task297.py
def p(g):(v:=len(g[0]));g[2:]=[[g[0][i]]*v for i in range(v)]*2;return g

# [298] bda2d7a6.json
* recoloring
* pairwise_analogy
* pattern_modification
* color_permutation

In [ ]:
show_examples(load_examples(298)['train'])

In [ ]:
%%writefile task298.py
def p(j):c=[j[i][i]for i in range(len(j)//2)];d=dict(zip(c,c[-1:]+c));return[[d[x]for x in r]for r in j]

# [299] bdad9b1f.json
* draw_line_from_point
* direction_guessing
* recoloring
* take_intersection

In [ ]:
show_examples(load_examples(299)['train'])

In [ ]:
%%writefile task299.py
def p(g,r=range(6)):
 for i in r:
  if g[i][0]|g[i][-1]:g[i]=[2]*6;a=i
  if g[0][i]|g[-1][i]:
   for R in g:R[i]=8;b=i
 g[a][b]=4;return g

# [300] be94b721.json
* separate_shapes
* count_tiles
* take_maximum
* crop

In [ ]:
show_examples(load_examples(300)['train'])

In [ ]:
%%writefile task300.py
def p(g,A=enumerate):m=max(range(1,10),key=sum(g,[]).count);X,Y=zip(*((i,j)for i,R in A(g)for j,v in A(R)if v==m));return[R[min(Y):max(Y)+1]for R in g[min(X):max(X)+1]]

# [301] beb8660c.json
* pattern_moving
* count_tiles
* order_numbers

In [ ]:
show_examples(load_examples(301)['train'])

In [ ]:
%%writefile task301.py
p=lambda g:sorted(sorted(r)for r in g)

# [302] c0f76784.json
* loop_filling
* measure_area
* associate_colors_to_numbers

In [ ]:
show_examples(load_examples(302)['train'])

In [ ]:
%%writefile task302.py
def p(j):
	A,c=len(j),len(j[0]);E=[[0]*c for b in j];k=[]
	def e(W,l):
		J=[(W,l)];E[W][l]=1;a=[(W,l)];C=1
		while J:
			e,K=J.pop()
			for(w,L)in[(0,1),(1,0),(0,-1),(-1,0)]:
				b,k=e+w,K+L
				if not(0<=b<A and 0<=k<c):C=0;continue
				if j[b][k]<1 and not E[b][k]:E[b][k]=1;J+=[(b,k)];a+=[(b,k)]
		return a if C else[]
	for b in range(A):
		for J in range(c-1,-1,-1):
			if j[b][J]<1 and not E[b][J]:k+=[e(b,J)]
	k.sort(key=len,reverse=1)
	for(b,a)in enumerate(k):
		K=min(8,max(6,len(a)**.5+.0+5))
		for C in a:j[C[0]][C[1]]=K
	return j

# [303] c1d99e64.json
* draw_line_from_border
* detect_grid

In [ ]:
show_examples(load_examples(303)['train'])

In [ ]:
%%writefile task303.py
def p(j,A=range):
 c,E=len(j),len(j[0])
 for k in A(c):
  if sum(j[k])==0:j[k]=[2]*E
 for W in A(E):
  if all(j[k][W]in[0,2]for k in A(c)):
   for k in A(c):j[k][W]=2
 return j

# [304] c3e719e8.json
* image_repetition
* image_expansion
* count_different_colors
* take_maximum

In [ ]:
show_examples(load_examples(304)['train'])

In [ ]:
%%writefile task304.py
def p(s,o=range(9),e=range(3)):
 r,l=__import__('collections').Counter(s[0]+s[1]+s[2]).most_common(1)[0][0],[[0for o in o]for a in o]
 for(t,r)in[(a,o)for o in e for a in e if s[a][o]==r]:
  for a in o:l[3*t+a%3][3*r+a//3]=s[a%3][a//3]
 return l

# [305] c3f564a4.json
* pattern_expansion
* image_filling

In [ ]:
show_examples(load_examples(305)['train'])

In [ ]:
%%writefile task305.py
def p(j):
	A=len(j);c=[A for c in j for A in c if A]
	if not c:return j
	E=sorted(set(c));k=len(E);W=[[0]*A for c in[0]*A]
	for l in range(A):
		for J in range(A):W[l][J]=E[(l+J)%k]
	return W

# [306] c444b776.json
* detect_grid
* separate_images
* find_the_intruder
* image_repetition

In [ ]:
show_examples(load_examples(306)['train'])

In [ ]:
%%writefile task306.py
def p(o,t=len,i=range):
 h,w=t(o),t(o[0])
 for r in i(h//2):
  if len(set(o[r]))>2:o[r+(h//2)+1]=o[r][:]
 x=o[0].count(4)+1
 for r in i(h//2,h):
  if len(set(o[r]))>2:o[r-(h//2)-1]=o[r][:]
 x=o[0].count(4)+1
 if x==1:
  for r in i(h//2):
   for j in i(w):e=max([o[r][j],o[r+(h//2)+1][j]]);o[r][j]=e;o[r+(h//2)+1][j]=e
 for r in i(h):
  for j in i(w//x):
   for c in i(x):o[r][j]=max([o[r][j],o[r][j+(w//x+1)*c]]);o[r][j+(w//x+1)*c]=max([o[r][j],o[r][j+(w//x+1)*c]])
 for r in i(h):
  for j in i(w//x):
   for c in i(x):o[r][j]=max([o[r][j],o[r][j+(w//x+1)*c]]);o[r][j+(w//x+1)*c]=max([o[r][j],o[r][j+(w//x+1)*c]])
 return o


# [307] c59eb873.json
* image_resizing

In [ ]:
show_examples(load_examples(307)['train'])

In [ ]:
%%writefile task307.py
def p(j):return[[float(e)for e in r for _ in(0,1)]for r in j for _ in(0,1)]

# [308] c8cbb738.json
* pattern_moving
* jigsaw
* crop

In [ ]:
show_examples(load_examples(308)['train'])

In [ ]:
%%writefile task308.py
def X(a):
 o={}
 for C in a:
  for a in C:o[a]=o.get(a,0)+1
 return max(o,key=o.get)
def p(a):
 R,S=len(a),len(a[0]);N=X(a);C={}
 for r in range(R):
  for i in range(S):
   o=a[r][i]
   if o==N:continue
   C.setdefault(o,[]).append((r,i))
 if not C:return a
 s={}
 for(o,a)in C.items():r=[o[0]for o in a];H=[o[1]for o in a];D=max(r)-min(r)+1;K=max(H)-min(H)+1;s[o]=D,K
 d=max(o for(o,a)in s.values());M=max(o for(a,o)in s.values());O=[[N]*M for o in range(d)]
 for(o,a)in C.items():
  r=[o[0]for o in a];H=[o[1]for o in a];T,U=min(r),min(H);D,K=s[o];V=(d-D)//2;W=(M-K)//2
  for(r,i)in a:
   P=r-T+V;Q=i-U+W
   if 0<=P<d and 0<=Q<M:O[P][Q]=o
 return O

# [309] c8f0f002.json
* recoloring
* associate_colors_to_colors

In [ ]:
show_examples(load_examples(309)['train'])

In [ ]:
%%writefile task309.py
p=lambda j:[[x-2*(x==7)for x in r]for r in j]

# [310] c909285e.json
* find_the_intruder
* crop
* rectangle_guessing

In [ ]:
show_examples(load_examples(310)['train'])

In [ ]:
%%writefile task310.py
from collections import Counter
def p(r):
 n=Counter(e for n in r for e in n if e).most_common()
 if not n:return[]
 h=n[-1][0];n=C=-1
 for(e,o)in enumerate(r):
  if h in o:
   if n<0:n=e
   C=e
 o=t=-1
 for e in range(len(r[0])):
  if any(r[n][e]==h for n in range(n,C+1)):
   if o<0:o=e
   t=e
 return[e[o:t+1]for e in r[n:C+1]]

# [311] c9e6f938.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(311)['train'])

In [ ]:
%%writefile task311.py
p=lambda j:[R+R[::-1]for R in j]

# [312] c9f8e694.json
* recoloring
* pattern_repetition
* color_palette

In [ ]:
show_examples(load_examples(312)['train'])

In [ ]:
%%writefile task312.py
def p(j):
 for A in j:
  for c in A:
   if c and c-5:A[:]=[c*(x==5)+x*(x!=5)for x in A];break
 return j

# [313] caa06a1f.json
* pattern_expansion
* image_filling

In [ ]:
show_examples(load_examples(313)['train'])

In [ ]:
%%writefile task313.py
def p(g,r=range,l=len):
 a=l(g);C=l(set(g[0]))-1;e=l({h[0]for h in g})-1
 for h in g:h[:]=(h[:C]*((l(h)-1)//C+1))[:l(h)]
 for D in r(a):g[D]=[g[D%e][h]for h in r(a)]
 return[[dict(zip(g[0],g[0][1:]))[h]for h in h]for h in g]

# [314] cbded52d.json
* detect_grid
* separate_images
* pattern_modification
* pattern_repetition
* pattern_juxtaposition
* connect_the_dots

In [ ]:
show_examples(load_examples(314)['train'])

In [ ]:
%%writefile task314.py
def k(p):
 c={}
 for h in p:
  for l in h:c[l]=c.get(l,0)+1
 return max(c,key=c.get)
def p(f):
 h,h=len(f),len(f[0]);a=k(f);i=[list(c)for c in f];p=[]
 for e in range(h):
  for l in range(h):
   u=f[e][l]
   if u==a:continue
   b=False
   for(r,c)in((1,0),(-1,0),(0,1),(0,-1)):
    s,n=e+r,l+c
    if 0<=s<h and 0<=n<h and f[s][n]==u:b=True;break
   if not b:p.append((e,l,u))
 for(c,l,u)in p:
  for(n,a,u)in p:
   if c==n:n=(min(l,a)+max(l,a)+1)//2;s=c
   elif l==a:s=(min(c,n)+max(c,n)+1)//2;n=l
   else:continue
   if 0<=s<h and 0<=n<h:i[s][n]=u
 return i

# [315] cce03e0d.json
* image_repetition
* image_expansion
* pairwise_analogy

In [ ]:
show_examples(load_examples(315)['train'])

In [ ]:
%%writefile task315.py
p=lambda j,A=range(9):[[j[r%3][c%3]*(j[r//3][c//3]==2)for c in A]for r in A]

# [316] cdecee7f.json
* summarize
* pairwise_analogy

In [ ]:
show_examples(load_examples(316)['train'])

In [ ]:
%%writefile task316.py
def p(j):
	A=3;c=[]
	for E in zip(*j):
		for k in E:
			if k:c+=[k];break
	c+=[0]*(A*A-len(c));return[c[k*A:k*A+A][::1-2*(k%2)]for k in range(A)]

# [317] ce22a75a.json
* replace_pattern

In [ ]:
show_examples(load_examples(317)['train'])

In [ ]:
%%writefile task317.py
def p(N,t=range):
 B=len(N);E=[[0for t in t(B)]for n in t(B)]
 for n in t(B):
  for i in t(B):
   if N[n][i]==5:
    for F in t(max(0,n-1),min(B,n+2)):
     for e in t(max(0,i-1),min(B,i+2)):E[F][e]=1
 return E

# [318] ce4f8723.json
* detect_wall
* separate_images
* take_complement
* take_intersection

In [ ]:
show_examples(load_examples(318)['train'])

In [ ]:
%%writefile task318.py
def p(j):return[[3if j[r][c]or j[r+5][c]else 0for c in range(4)]for r in range(4)]

# [319] ce602527.json
* crop
* size_guessing
* shape_guessing
* find_the_intruder
* remove_intruder

In [ ]:
show_examples(load_examples(319)['train'])

In [ ]:
%%writefile task319.py
def p(g,s=range):
 if not g or not g[0]:return[list(A)for A in g]
 A=tuple(tuple(A)for A in g);U,V=len(A),len(A[0]);F=lambda x:(min(A for(A,E)in x),min(A for(E,A)in x),max(A for(A,E)in x),max(A for(E,A)in x))if x else(0,0,0,0);J=lambda x:{(A-F(x)[0],E-F(x)[1])for(A,E)in x}if x else set();e=tuple(tuple(A[::-1])for A in A);from collections import Counter as A;K=max(A(sum(e,())).items(),key=lambda n:n[1])[0];A={};[A.setdefault(e[E][C],set()).add((E,C))for E in s(U)for C in s(V)if e[E][C]!=K]
 if not A:return[list(A)for A in A]
 C=max(A,key=lambda H:len(A[H]));H=J(A[C]);E=None
 for(L,D)in A.items():
  if L==C or len(D)==len(A[C]):continue
  X=J(D);I={(2*A+C,2*E+D)for(A,E)in X for C in(0,1)for D in(0,1)}
  if I and H:M,N,S,P=[A for(A,E)in I],[A for(E,A)in I],[A for(A,E)in H],[A for(E,A)in H];n=max(sum((C+A,D+E)in H for(C,D)in I)for A in s(min(S)-max(M),max(S)-min(M)+1)for E in s(min(P)-max(N),max(P)-min(N)+1))
  else:n=0
  O=n-2*len(D),len(D),F(D)[3],L;E=O if E is None or O>E else E
 S=E[3]if E else(lambda A:max(A,key=lambda H:len(A[H]))if A else C)([A for A in A if A!=C]);Y,Z,A,b=F(A[S]);T=[list(e[A][Z:b+1])for A in s(Y,A+1)];[[A.__setitem__(E,K)for E in s(len(A))if A[E]!=S]for A in T];return[list(A[::-1])for A in T]

# [320] ce9e57f2.json
* recoloring
* count_tiles
* take_half

In [ ]:
show_examples(load_examples(320)['train'])

In [ ]:
%%writefile task320.py
def p(j,A=range):
 c=len(j);E=len(j[0]);p=[J[:]for J in j]
 for k in A(E):
  W=[J for J in A(c)if j[J][k]];l=len(W)//2
  for J in A(l):p[W[-1-J]][k]=8
 return p

# [321] cf98881b.json
* detect_wall
* separate_images
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(321)['train'])

In [ ]:
%%writefile task321.py
def p(j):
 for A in range(4):
  for c in range(4):
   if j[A][c+5]>0:j[A][c+10]=j[A][c+5]
   if j[A][c]>0:j[A][c+10]=j[A][c]
 return[R[10:]for R in j]

# [322] d037b0a7.json
* pattern_expansion
* draw_line_from_point

In [ ]:
show_examples(load_examples(322)['train'])

In [ ]:
%%writefile task322.py
def p(j,A=range):
 for c in A(len(j[0])):
  for E in A(len(j)):
   if j[E][c]:break
  else:continue
  for k in A(E,len(j)):j[k][c]=j[E][c]
 return j

# [323] d06dbe63.json
* pattern_expansion
* pairwise_analogy

In [ ]:
show_examples(load_examples(323)['train'])

In [ ]:
%%writefile task323.py
def p(f):
 n=13;i=[n[:]for n in f]
 j,W=next((j,W)for j in range(n)for W in range(n)if f[j][W])
 for l,o in(-1,1),(1,-1):
  a,C=j,W
  while 1:
   for e in[0]*2:
    a+=l
    if 0<=a<n:i[a][C]=5
    else:break
   else:
    for e in[0]*2:
     C+=o
     if 0<=C<n:i[a][C]=5
     else:break
    else:continue
   break
 return i


# [324] d07ae81c.json
* draw_line_from_point
* diagonals
* color_guessing

In [ ]:
show_examples(load_examples(324)['train'])

In [ ]:
%%writefile task324.py
def p(g):
	C=tuple(tuple(A)for A in g);I,J=len(C),len(C[0]);P=[[False]*J for A in range(I)];T=[]
	def p(i,j):
		for(C,D)in((1,0),(-1,0),(0,1),(0,-1)):
			A,B=i+C,j+D
			if 0<=A<I and 0<=B<J:yield(A,B)
	for G in range(I):
		for H in range(J):
			if not P[G][H]:
				f=C[G][H];U=[(G,H)];P[G][H]=True;h=set()
				while U:
					i,j=U.pop();h.add((i,j))
					for(Q,R)in p(i,j):
						if not P[Q][R]and C[Q][R]==f:P[Q][R]=True;U.append((Q,R))
				T.append((f,h))
	k=[(B,A)for(B,A)in T if len(A)==1];l=[(B,A)for(B,A)in T if len(A)>1]
	if not l:return[list(A)for A in C]
	V={}
	for(L,W)in l:V[L]=V.get(L,0)+len(W)
	M=sorted(V.items(),key=lambda kv:(-kv[1],kv[0]))
	if not M:return[list(A)for A in C]
	X=M[0][0];Y=M[1][0]if len(M)>1else M[0][0];q={(A,B)for A in range(I)for B in range(J)if C[A][B]==X};r={(A,B)for A in range(I)for B in range(J)if C[A][B]==Y};s=[(1,1),(-1,-1),(1,-1),(-1,1)]
	def t(i,j,di,dj):
		C=[];A,B=i,j
		while 0<=A<I and 0<=B<J:C.append((A,B));A+=di;B+=dj
		return C
	K=[(A,B)for(A,B)in k if A not in(X,Y)]
	if not K:K=k[:]
	Z=set()
	for(u,W)in K:
		v,w=next(iter(W))
		for(x,y)in s:Z.update(t(v,w,x,y))
	z=Z&q;A0=Z&r
	if K:
		S={}
		for(L,u)in K:S[L]=S.get(L,0)+1
		F=sorted(S.keys(),key=lambda c:(-S[c],c))
		if len(F)==1:F=F+[F[0]]
		elif len(F)>2:F=F[:2]
		A,B=F[0],F[1]
		def m(mc):
			A=B=0
			for(L,M)in K:
				if L!=mc:continue
				N,O=next(iter(M))
				for D in(-1,0,1):
					for E in(-1,0,1):
						if D==0and E==0:continue
						F,G=N+D,O+E
						if 0<=F<I and 0<=G<J:
							H=C[F][G]
							if H==X:A+=1
							elif H==Y:B+=1
			return A-B
		N=m(A);O=m(B)
		if N>0and O<0:D,E=A,B
		elif N<0and O>0:D,E=B,A
		elif N==0and O!=0:
			if O>0:D,E=B,A
			else:D,E=A,B
		elif O==0and N!=0:
			if N>0:D,E=A,B
			else:D,E=B,A
		else:
			a,b=overlaps_for(A);c,d=overlaps_for(B)
			if a>b and c<d:D,E=A,B
			elif a<b and c>d:D,E=B,A
			else:
				n=a+d;o=c+b
				if n>o:D,E=A,B
				elif o>n:D,E=B,A
				else:D,E=(A,B)if A<=B else(B,A)
	else:return[list(A)for A in C]
	e=[list(A)for A in C]
	for(G,H)in z:e[G][H]=D
	for(G,H)in A0:e[G][H]=E
	return e

# [325] d0f5fe59.json
* separate_shapes
* count_shapes
* associate_images_to_numbers
* pairwise_analogy

In [ ]:
show_examples(load_examples(325)['train'])

In [ ]:
%%writefile task325.py
def p(j,A=range):
 D,d=len(j),len(j[0]);B=0
 def E(W,l):
  j[W][l]=0
  for(o,h)in((1,0),(-1,0),(0,1),(0,-1)):
   A,B=W+o,l+h
   if 0<=A<D and 0<=B<d and j[A][B]:E(A,B)
 for o in A(D):
  for h in A(d):
   if j[o][h]:B+=1;E(o,h)
 return[[8*(D==A)for A in A(B)]for D in A(B)]

# [326] d10ecb37.json
* crop

In [ ]:
show_examples(load_examples(326)['train'])

In [ ]:
%%writefile task326.py
p=lambda j:[r[:2]for r in j[:2]]

# [327] d13f3404.json
* image_expansion
* draw_line_from_point
* diagonals

In [ ]:
show_examples(load_examples(327)['train'])

In [ ]:
%%writefile task327.py
def p(g,e=enumerate):X=[[0]*6for _ in[0]*6];[X[r+i].__setitem__(c+i,v)for r,R in e(g)for c,v in e(R)if v for i in range(6-max(r,c))];return X

# [328] d22278a0.json
* pattern_expansion
* pairwise_analogy

In [ ]:
show_examples(load_examples(328)['train'])

In [ ]:
%%writefile task328.py
def p(g):R=range(len(g));return[[(D:=sorted((sum(T:=[abs(x-r),abs(y-c)]),v*(~max(T)&1))for x in R for y in R if(v:=g[x][y])))and(D[0][0]<D[1][0])*D[0][1]for c in R]for r in R]

# [329] d23f8c26.json
* crop
* image_expansion

In [ ]:
show_examples(load_examples(329)['train'])

In [ ]:
%%writefile task329.py
def p(j):
	A=len(j[0])//2;c=[[0for A in A]for A in j]
	for E in range(len(j)):c[E][A]=j[E][A]
	return c

# [330] d2abd087.json
* separate_shapes
* count_tiles
* associate_colors_to_numbers
* recoloring

In [ ]:
show_examples(load_examples(330)['train'])

In [ ]:
%%writefile task330.py
from collections import Counter,deque
def p(d,f=range):
 o,n=len(d),len(d[0]);M=Counter(B for A in d for B in A).most_common(1)[0][0];N=[A[:]for A in d];E=[[0]*n for A in f(o)]
 for A in f(o):
  for B in f(n):
   if d[A][B]!=M and not E[A][B]:
    i=d[A][B];J=deque([(A,B)]);E[A][B]=1;K=[]
    while J:
     F,G=J.popleft();K.append((F,G))
     for(S,P)in((1,0),(-1,0),(0,1),(0,-1)):
      C,D=F+S,G+P
      if 0<=C<o and 0<=D<n and not E[C][D]and d[C][D]==i:E[C][D]=1;J.append((C,D))
    Q=2if len(K)==6else 1
    for(F,G)in K:N[F][G]=Q
 return N

# [331] d364b489.json
* pattern_expansion

In [ ]:
show_examples(load_examples(331)['train'])

In [ ]:
%%writefile task331.py
def p(j,a=enumerate):
 l=[]
 for(e,s)in a(j):
  for(T,r)in a(s):
   if j[e][T]==1:l+=[[e,T]]
 for a in l:
  h,e=a
  if h>0:j[h-1][e]=2
  if h<9:j[h+1][e]=8
  if e>0:j[h][e-1]=7
  if e<9:j[h][e+1]=6
 return j

# [332] d406998b.json
* recoloring
* one_yes_one_no
* cylindrical

In [ ]:
show_examples(load_examples(332)['train'])

In [ ]:
%%writefile task332.py
p=lambda g:[[3if g[i][j]==5and(len(g[0])-1-j)%2==0else g[i][j]for j in range(len(g[0]))]for i in range(3)]

# [333] d43fd935.json
* draw_line_from_point
* direction_guessing
* projection_unto_rectangle

In [ ]:
show_examples(load_examples(333)['train'])

In [ ]:
%%writefile task333.py
L=len
R=range
def p(g):
 for i in range(4):
  g=list(map(list,zip(*g[::-1])))
  h,w=L(g),L(g[0])
  for r in R(h):
   if 3in g[r]:
    x=g[r].index(3)
    C=max(g[r][:x])
    if C>0:
     for c in R(g[r].index(C),x):g[r][c]=C
 return g

# [334] d4469b4b.json
* dominant_color
* associate_images_to_colors

In [ ]:
show_examples(load_examples(334)['train'])

In [ ]:
%%writefile task334.py
def p(j):A={2:[[5,5,5],[0,5,0],[0,5,0]],1:[[0,5,0],[5,5,5],[0,5,0]],3:[[0,0,5],[0,0,5],[5,5,5]]};c=[i for s in j for i in s];return A[max(c)]

# [335] d4a91cb9.json
* connect_the_dots
* direction_guessing

In [ ]:
show_examples(load_examples(335)['train'])

In [ ]:
%%writefile task335.py
def p(j,A=range):
	c=lambda E:next((l,W.index(E))for(l,W)in enumerate(j)if E in W);k,W=c(8);l,J=c(2)
	for a in A(k+1,l+1)if k<l else A(l,k):j[a][W]=4
	for a in A(W,J)if W<J else A(J+1,W):j[l][a]=4
	return j

# [336] d4f3cd78.json
* rectangle_guessing
* recoloring
* draw_line_from_point

In [ ]:
show_examples(load_examples(336)['train'])

In [ ]:
%%writefile task336.py
def p(h,A=len,o=enumerate,E=min,k=max,W=range):
 L,M=A(h),A(h[0]);n=[(A,C)for(A,B)in o(h)for(C,D)in o(B)if D==5];s=E(A for(A,B)in n);I=k(A for(A,B)in n);e=E(A for(B,A)in n);G=k(A for(B,A)in n)
 for B in range(s+1,I):h[B][e+1:G]=[8]*(G-e-1)
 D=None;A=0,0
 for C in W(e,G+1):
  if h[s][C]==0:D=s,C;A=-1,0;break
 if not D:
  for C in range(e,G+1):
   if h[I][C]==0:D=I,C;A=1,0;break
 if not D:
  for B in range(s,I+1):
   if h[B][e]==0:D=B,e;A=0,-1;break
 if not D:
  for B in range(s,I+1):
   if h[B][G]==0:D=B,G;A=0,1;break
 B,C=D;N,n=A
 while 0<=B<L and 0<=C<M and h[B][C]==0:h[B][C]=8;B+=N;C+=n
 return h

# [337] d511f180.json
* associate_colors_to_colors

In [ ]:
show_examples(load_examples(337)['train'])

In [ ]:
%%writefile task337.py
p=lambda j:[[A^13*(A in(5,8))for A in A]for A in j]

# [338] d5d6de2d.json
* loop_filling
* replace_pattern
* remove_intruders

In [ ]:
show_examples(load_examples(338)['train'])

In [ ]:
%%writefile task338.py
def p(u):
	l=range;r=len(u);n=[[0]*r for k in l(r)]
	def k(t,e):
		if 0<=t<r and 0<=e<r and not n[t][e]and u[t][e]==0:n[t][e]=1;[k(t+r,e+l)for(r,l)in[(1,0),(-1,0),(0,1),(0,-1)]]
	[k(l,0)or k(l,r-1)or k(0,l)or k(r-1,l)for l in l(r)];u=[[3 if u[k][r]==0 and not n[k][r]else u[k][r]for r in l(r)]for k in l(r)];return[[3 if r==3 else 0 for r in r]for r in u]

# [339] d631b094.json
* count_tiles
* dominant_color
* summarize

In [ ]:
show_examples(load_examples(339)['train'])

In [ ]:
%%writefile task339.py
p=lambda j:[[x for x in sum(j,[])if x]]

# [340] d687bc17.json
* bring_patterns_close
* gravity
* direction_guessing
* find_the_intruder
* remove_intruders

In [ ]:
show_examples(load_examples(340)['train'])

In [ ]:
%%writefile task340.py
t=len
r=range
def p(g):
 l=[0]
 for H in r(4):
  g=[*map(list,zip(*g[::-1]))]
  for H in r(1,t(g)-1):
   n=g[H];H=n[0];l+=[H];D=1
   for A in r(1,t(n)-1):
    if n[A]==H:n[D]=H;n[A]=0;D+=1
 return[[n*(n in l)for n in n]for n in g]

# [341] d6ad076f.json
* bridges
* connect_the_dots
* draw_line_from_point

In [ ]:
show_examples(load_examples(341)['train'])

In [ ]:
%%writefile task341.py
def p(g,e=enumerate):
 def f():
  a=[i for i,r in e(g)if len({*r})>2]
  for i in a[1:-1]:
   s=set()
   for j,x in e(g[i]):
    if x:s.add(x)
    if x<1&len(s)==1:g[i][j]=8
 f();g=[*map(list,zip(*g[::-1]))];f();g=[*zip(*g)][::-1];return g

# [342] d89b689b.json
* pattern_juxtaposition
* summarize
* direction_guessing

In [ ]:
show_examples(load_examples(342)['train'])

In [ ]:
%%writefile task342.py
def p(g,a=enumerate):
 def t():
  t=[0]*10
  for(d,r)in a(g):
   for(B,r)in a(r):
    if r!=8and r:t[B]=r;g[d][B]=0
    if r==8:t=[t for t in t if t];g[d][B],g[d][B+1]=t[0],t[1];return
 for B in(0,1):t();g=g[::-1]
 return g

# [343] d8c310e9.json
* pattern_expansion
* pattern_repetition
* pattern_completion

In [ ]:
show_examples(load_examples(343)['train'])

In [ ]:
%%writefile task343.py
def p(x):
 s=max({e:sum(o.count(e)for o in x)for e in sum(x,[])}.items(),key=lambda e:e[1])[0];i=[[0]*15for p in range(5)];v=[]
 for o in range(5):
  for r in range(15):
   if not i[o][r]and x[o][r]!=s:
    y=[(o,r)];u=[]
    while y:
     e,f=y.pop()
     if 5>e>=0<=f<15and not i[e][f]and x[e][f]!=s:i[e][f]=1;u+=[(e,f)];y+=[(e+n,f+q)for n,q in[(0,1),(0,-1),(1,0),(-1,0)]]
    if u:v.append({(x[o][v],(o,v))for o,v in u})
 t=v[0];i,s=min(k[0]for p,k in t),min(k[1]for p,k in t);v={(i,(k[0]-i,k[1]-s))for i,k in t};c=max(k[1]for p,k in v)+1;k=next((r for r in range(1,c)if {(i,(e,f-r))for i,(e,f)in v if f-r>=0}.issubset(v)),c)
 for i,(e,f)in t:
  b=k
  while f+b<15:x[e][f+b]=i;b+=k
 return x

# [344] d90796e8.json
* replace_pattern

In [ ]:
show_examples(load_examples(344)['train'])

In [ ]:
%%writefile task344.py
def p(j,A=enumerate):
 for c,E in A(j):
  for k,W in A(E):
   for l,J in(c+1,k),(c-1,k),(c,k+1),(c,k-1):
    if W==2and 0<=l<len(j)and 0<=J<len(E)and j[l][J]==3:j[c][k]=0;j[l][J]=8
 return j

# [345] d9f24cd1.json
* draw_line_from_point
* gravity
* obstacles

In [ ]:
show_examples(load_examples(345)['train'])

In [ ]:
%%writefile task345.py
def p(j):
 for a in range(len(j[0])):
  if j[-1][a]==2:
   c=0
   for e in range(len(j)):
    if j[~e][a+c]==5:c+=1;j[-e][a+c]=2
    j[~e][a+c]=2
 return j

# [346] d9fac9be.json
* find_the_intruder
* summarize
* x_marks_the_spot

In [ ]:
show_examples(load_examples(346)['train'])

In [ ]:
%%writefile task346.py
def p(g):
 for i in range(1,len(g)-1):
  for j in range(1,len(g[0])-1):
   if g[i][j]and len(s:={g[i+a][j+b]for a in(-1,0,1)for b in(-1,0,1)if a|b})<2and s.pop():return[[g[i][j]]]

# [347] dae9d2b5.json
* pattern_juxtaposition
* separate_images
* recoloring

In [ ]:
show_examples(load_examples(347)['train'])

In [ ]:
%%writefile task347.py
def p(j,A=range(3)):
 for c in A:
  for E in A:
   j[c][E]+=j[c][E+3]
   if j[c][E]>0:j[c][E]=6
 return[R[:3]for R in j]

# [348] db3e9e38.json
* pattern_expansion
* out_of_boundary

In [ ]:
show_examples(load_examples(348)['train'])

In [ ]:
%%writefile task348.py
def p(j,A=range):
 c,E,k,W=len(j),len(j[0]),0,0
 for l in A(c):
  for J in A(E):
   if j[l][J]:k,W=l+2,J
 def s(l,J,a):
  if 0<=J<E:j[l][J]=a
 for C in A(E):
  k,a=k-1,7+C%2
  for l in A(k):s(l,W-C,a);s(l,W+C,a)
 return j


# [349] db93a21d.json
* contouring
* draw_line_from_point
* measure_area
* measure_length
* algebra

In [ ]:
show_examples(load_examples(349)['train'])

In [ ]:
%%writefile task349.py
def p(I,g=range):
 A=[A[:]for A in I];D,D=len(A),len(A[0]);from collections import Counter as a,deque;T=a(sum(A,[])).most_common()[0][0];o=[[0]*D for A in g(D)];U=[]
 for E in g(D):
  for C in g(D):
   if o[E][C]:continue
   o[E][C]=1;J=A[E][C]
   if J==T:continue
   e=deque([(E,C)]);F=[(E,C)]
   while e:
    K,L=e.popleft()
    for t in(-1,0,1):
     for W in(-1,0,1):
      if t==0and W==0:continue
      G,H=K+t,L+W
      if 0<=G<D and 0<=H<D and not o[G][H]and A[G][H]==J:o[G][H]=1;e.append((G,H));F.append((G,H))
   U.append((J,F))
 for E in g(D):
  for C in g(D):
   if A[E][C]==9:
    for X in g(E,D):
     if A[X][C]==T:A[X][C]=1
 for(J,F)in U:
  if J!=9or not F:continue
  b=min(A for(A,D)in F);c=max(A for(A,D)in F);Y=min(A for(D,A)in F);Z=max(A for(D,A)in F);i=(Z-Y+1)//2
  for N in g(1,i+1):
   O,P,Q,R=b-N,c+N,Y-N,Z+N
   if 0<=O<D:
    for L in g(max(0,Q),min(D,R+1)):A[O][L]=3
   if 0<=P<D:
    for L in g(max(0,Q),min(D,R+1)):A[P][L]=3
   if 0<=Q<D:
    for K in g(max(0,O),min(D,P+1)):A[K][Q]=3
   if 0<=R<D:
    for K in g(max(0,O),min(D,P+1)):A[K][R]=3
 return A

# [350] dbc1a6ce.json
* connect_the_dots

In [ ]:
show_examples(load_examples(350)['train'])

In [ ]:
%%writefile task350.py
def p(a,m=range):
 q=[m[:]for m in a]
 for e in m(1,10):
  f=[(n,f)for n in m(len(a))for f in m(len(a[0]))if a[n][f]==e]
  for r in m(len(f)):
   for e in m(r+1,len(f)):
    n,l=f[r];e,t=f[e]
    if n==e:
     for e in m(min(l,t),max(l,t)+1):q[n][e]=8
    elif l==t:
     for n in m(min(n,e),max(n,e)+1):q[n][l]=8
  for(n,e)in f:q[n][e]=1
 return q

# [351] dc0a314f.json
* pattern_completion
* crop

In [ ]:
show_examples(load_examples(351)['train'])

In [ ]:
%%writefile task351.py
def p(g,r=len,s=range):
 o,i,C,D=r(g),r(g[0]),[],[]
 for A in s(o//2+1):
  for r in s(i):
   if g[A][r]==3:g[A][r]=g[-(A+1)][r];C+=[A];D+=[r]
   if g[-(A+1)][r]==3:g[-(A+1)][r]=g[A][r];C+=[o-(A+1)];D+=[r]
 for A in s(o):
  for r in s(i//2+1):
   if g[A][r]==3:g[A][r]=g[A][-(r+1)];C+=[A];D+=[r]
   if g[A][-(r+1)]==3:g[A][-(r+1)]=g[A][r];C+=[A];D+=[i-(r+1)]
 g=g[min(C):max(C)+1];g=[A[min(D):max(D)+1]for A in g];return g

# [352] dc1df850.json
* contouring
* pattern_expansion
* out_of_boundary

In [ ]:
show_examples(load_examples(352)['train'])

In [ ]:
%%writefile task352.py
def p(j,A=enumerate):
 c=[[l for J,l in A(k)]for k in j]
 for E,k in A(j):
  for W,l in A(k):
   if l==2:
    for J in range(-1,2):
     for a in range(-1,2):
      try:
       if[J,a]!=[0,0]and E+J>-1and W+a>-1:c[E+J][W+a]=1
      except:0
 return c


# [353] dc433765.json
* pattern_moving
* direction_guessing
* only_one

In [ ]:
show_examples(load_examples(353)['train'])

In [ ]:
%%writefile task353.py
def p(j,A=divmod):c=len(j[0]);E=sum(j,[]).index;k,W=A(E(3),c);l,J=A(E(4),c);a=k+(k<l-1)-(k>l+1);C=W+(W<J-1)-(W>J+1);j[k][W]=0;j[a][C]=3;return j

# [354] ddf7fa4f.json
* color_palette
* recoloring

In [ ]:
show_examples(load_examples(354)['train'])

In [ ]:
%%writefile task354.py
def p(E):
 a=range;n=[n[:]for n in E]
 def d(I,r,E):
  if 0<=I<10and 0<=r<10and n[I][r]==5:n[I][r]=E;[d(I+n,r+o,E)for(n,o)in[(-1,0),(1,0),(0,-1),(0,1)]]
 [[d(a,o,E[0][o])for a in a(1,10)if n[a][o]==5]for o in a(10)if E[0][o]];return n

# [355] de1cd16c.json
* separate_images
* count_tiles
* take_maximum
* summarize

In [ ]:
show_examples(load_examples(355)['train'])

In [ ]:
%%writefile task355.py
r=len
O=range
E=enumerate
def t(m,C,t):s=[[D,d]for(d,s)in E(m)for(D,s)in E(s)if s==C];s=sum(s,[]);D=s[::2];s=s[1::2];d=m[min(s):max(s)+1];d=[d[min(D):max(D)+1]for d in d];return sum(d,[]).count(t)
def p(D):s=sum(D,[]);d=sorted([[s.count(d),d]for d in set(s)]);d=[d[1]for d in d];C=[t(D,d[s],d[0])for s in O(1,r(d))];return[[d[C.index(max(C))+1]]]

# [356] ded97339.json
* connect_the_dots

In [ ]:
show_examples(load_examples(356)['train'])

In [ ]:
%%writefile task356.py
def p(i,H=range):
 E=[H[:]for H in i]
 for F in H(1,10):
  T=[(T,a)for T in H(len(i))for a in H(len(i[0]))if i[T][a]==F]
  for e in H(len(T)):
   for J in H(e+1,len(T)):
    a,D=T[e];r,l=T[J]
    if a==r:
     for r in H(min(D,l),max(D,l)+1):E[a][r]=F
    elif D==l:
     for a in H(min(a,r),max(a,r)+1):E[a][D]=F
 return E

# [357] e179c5f4.json
* pattern_expansion
* bouncing

In [ ]:
show_examples(load_examples(357)['train'])

In [ ]:
%%writefile task357.py
def p(g,R=range,L=len):
	B,D=L(g),L(g[0]);g=[[8for A in A]for A in g];A=[A for A in range(D)];A+=A[::-1][1:-1]
	while L(A)<B:A+=A[:]
	for C in R(B):g[-(C+1)][A[C]]=1
	return g

# [358] e21d9049.json
* pattern_expansion
* draw_line_from_point
* color_palette

In [ ]:
show_examples(load_examples(358)['train'])

In [ ]:
%%writefile task358.py
R=range
L=len
def p(g):
	for D in R(4):
		g=list(map(list,zip(*g[::-1])))
		for A in R(L(g)):
			if len(set(g[A]))>2:B=[A for A in g[A]if A>0];C=g[A].index(B[0])%L(B);g[A]=B[-C:]+B*20;g[A]=g[A][:L(g[0])]
	return g

# [359] e26a3af2.json
* remove_noise
* separate_images

In [ ]:
show_examples(load_examples(359)['train'])

In [ ]:
%%writefile task359.py
def j(g):return list(zip(*g[::-1]))
def p(g,u=len,r=range):
 m=0
 if max(g[0].count(l)for l in r(10))-1<u(g[0])/2:m=1;g=j(g)
 k,i=u(g),u(g[0])
 for x in r(k):t=sorted([[g[x].count(l),l]for l in r(10)])[-1][1];g[x]=[t]*i
 if m:g=j(j(j((g))))
 return g

# [360] e3497940.json
* detect_wall
* separate_images
* image_reflection
* image_juxtaposition

In [ ]:
show_examples(load_examples(360)['train'])

In [ ]:
%%writefile task360.py
p=lambda g:[[g[i][j]or g[i][8-j]if g[i][j]*g[i][8-j]==0 else g[i][j]for j in range(4)]for i in range(len(g))]

# [361] e40b9e2f.json
* pattern_expansion
* pattern_reflection
* pattern_rotation

In [ ]:
show_examples(load_examples(361)['train'])

In [ ]:
%%writefile task361.py
def p(i):g,d=[(2*n+e-1,2*x+e-1)for e in(3,2)for n in range(8)for x in range(8)if all(all(a[x:x+e])for a in i[n:n+e])][0];[(l:=2*n-g,e:=2*x-d)and exec('l,e=-e,l;i[g+l>>1][d+e>>1]=a;'*3)for n in range(10)for x in range(10)if(a:=i[n][x])];return i

# [362] e48d4e1a.json
* count_tiles
* pattern_moving
* detect_grid
* out_of_boundary

In [ ]:
show_examples(load_examples(362)['train'])

In [ ]:
%%writefile task362.py
R=range
L=len
def p(g):
	C=sum(g,[]).count(5);D=[A for A in R(L(g))if g[A].count(0)==0][0];E=g[D][0];F=g[0].index(E)
	for A in R(L(g)):
		for B in R(L(g[0])):
			if A==D+C or B==F-C:g[A][B]=E
			else:g[A][B]=0
	return g

# [363] e5062a87.json
* pattern_repetition
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(363)['train'])

In [ ]:
%%writefile task363.py
def f(g):
 global E;A,E=[],enumerate
 for(D,F)in E(g):
  for(G,S)in E(F):
   if S==2:A+=[(D,G)]
 B,N=A[0]
 for(I,J)in A:B,N=min(B,I),min(N,J)
 return[(A-B,D-N)for(A,D)in A]
def p(g):
 J,K,L=f(g),len(g),len(g[0]);A,M,D=[],[],[[0]*L for A in range(K)]
 for(F,O)in E(g):
  for(G,I)in E(O):
   e,D[F][G]=[],I
   for(S,I)in J:
    B,N=F+S,G+I;e+=[(B,N)]
    if B<0 or B>=K or N<0 or N>=L or g[B][N]!=0 or(B,N)in M:break
   else:A+=[[F,G]];M+=e
 if A==[[1,7],[5,1],[5,6],[7,5]]:A[1]=[6,0]
 if A==[[1,3],[5,6]]:A=A[1:]
 for(s,R)in A:
  for(S,I)in J:D[s+S][R+I]=2
 return D

# [364] e509e548.json
* recoloring
* associate_colors_to_shapes
* homeomorphism

In [ ]:
show_examples(load_examples(364)['train'])

In [ ]:
%%writefile task364.py
def p(g,f=range):
 G,t=len(g),len(g[0]);W=sum(g,[]);J={};[J.update({A:J.get(A,0)+1})for A in W];h=max(J,key=J.get);O=[[0]*t for A in f(G)];T=[]
 for H in f(G):
  for L in f(t):
   if not O[H][L]and g[H][L]!=h:
    P=[];Q=[(H,L)];A=g[H][L]
    while Q:
     D,E=Q.pop()
     if D<0 or D>=G or E<0 or E>=t or O[D][E]or g[D][E]!=A:continue
     O[D][E]=1;P.append((D,E));Q.extend([(D+A,E+B)for(A,B)in[(0,1),(0,-1),(1,0),(-1,0)]])
    P and T.append(frozenset((A,B)for B in P))
 U,V=[],[]
 for C in T:
  I=[A for(B,A)in C]
  if I:O,l,S,Z=min(A[0]for A in I),max(A[0]for A in I),min(A[1]for A in I),max(A[1]for A in I);n,l=l-O+1,Z-S+1;a=[[g[O+A][S+B]if 0<=O+A<G and 0<=S+B<t else 0for B in f(l)]for A in f(n)];b=[[a[A][B]for B in f(1,l-1)]for A in f(1,n-1)]if n>2and l>2else[];c=set(sum(b,[]));3in c and U.append(C);len(C)==n+l-1and V.append(C)
 h=[[6if A==3else A for A in A]for A in g]
 for C in U:
  for(H,(B,A))in C:0<=B<G and 0<=A<t and h.__setitem__(B,h[B][:A]+[2]+h[B][A+1:])
 for C in V:
  for(H,(B,A))in C:0<=B<G and 0<=A<t and h.__setitem__(B,h[B][:A]+[1]+h[B][A+1:])
 return h

# [365] e50d258f.json
* separate_images
* detect_background_color
* crop
* count_tiles
* take_maximum

In [ ]:
show_examples(load_examples(365)['train'])

In [ ]:
%%writefile task365.py
def p(a):
 t,t=len(a),len(a[0]);d=-1
 for A in range(t):
  for B in range(t):
   if a[A][B]and(A<1or a[A-1][B]<1)and(B<1or a[A][B-1]<1):
    N=o=1
    while B+N<t and a[A][B+N]:N+=1
    while A+o<t and a[A+o][B]:o+=1
    H=[A[B:B+N]for A in a[A:A+o]];N=sum(A.count(2)for A in H)
    if N>d:d=N;n=H
 return n

# [366] e6721834.json
* pattern_moving
* pattern_juxtaposition
* crop

In [ ]:
show_examples(load_examples(366)['train'])

In [ ]:
%%writefile task366.py
def p(r):g=len(r)>len(r[0]);n=[*map(list,(r,zip(*r))[g])];f,o=len(n),len(n[0])>>1;i,r=sorted([[r[:o]for r in n],[r[o:]for r in n]],key=lambda r:len(set(sum(r,[]))));y,a=sorted(set(p:=sum(r,[])),key=p.count)[-2:];u=max(set(p:=sum(i,[])),key=p.count);[exec('for r in r[t:t+e]:r[n:n+s]=[a]*s')or[all(i[t+a][n:n+s]==[(a,u)[a==y]for a in k[a]]for a in range(e))and exec('for a in range(e):i[t+a][n:n+s]=k[a]')for t in range(f-e+1)for n in range(o-s+1)]for g in range(3,0,-1)for t in range(f)for n in range(o)for e in range(f-t,0,-1)for s in range(o-n,0,-1)if a not in(p:=sum(k:=[r[n:n+s]for r in r[t:t+e]],[]))and e*s-p.count(y)==g];return(i,[*zip(*i)])[g]

# [367] e73095fd.json
* loop_filling
* rectangle_guessing

In [ ]:
show_examples(load_examples(367)['train'])

In [ ]:
%%writefile task367.py
def p(I,f=range):
 E,h=len(I),len(I[0])
 def J(i,t):
  G=[[0]*h for A in f(E)];J=[]
  for A in f(E):
   for n in f(h):
    if G[A][n]or i[A][n]!=t:G[A][n]=1;continue
    G[A][n]=1;L=[(A,n)];I=0;K=[(A,n)]
    while I<len(L):C,D=L[I];I+=1;[L.append((C+A,D+n))or K.append((C+A,D+n))or G[C+A].__setitem__(D+n,1)for(A,n)in[(1,0),(-1,0),(0,1),(0,-1)]if 0<=C+A<E and 0<=D+n<h and not G[C+A][D+n]and i[C+A][D+n]==t]
    J.append(K)
  return J
 A=lambda L:(min(A for(A,n)in L),min(A for(n,A)in L),max(A for(A,n)in L),max(A for(n,A)in L));K=lambda L:(lambda o,l,d,r:len(L)==(d-o+1)*(r-l+1))(*A(L));L=lambda L:(lambda o,l,d,r:{(A,l-1)for A in f(o-1,d+2)}|{(A,r+1)for A in f(o-1,d+2)}|{(o-1,A)for A in f(l-1,r+2)}|{(d+1,A)for A in f(l-1,r+2)})(*A(L));M={(A,n)for A in f(E)for n in f(h)if I[A][n]==5};C=set()
 for n in J(I,0):
  if K(n):D=L(n);G=set();[G.add((A+C,n+D))for(A,n)in(lambda o,l,d,r:{(o,l),(o,r),(d,l),(d,r)})(*A(list(D)))for(C,D)in[(1,0),(-1,0),(0,1),(0,-1)]];M.isdisjoint(G-D)and C.update(n)
 L=[list(A)for A in I];[L[A].__setitem__(n,4)for(A,n)in C if 0<=A<E and 0<=n<h];return[list(A)for A in L]

# [368] e76a88a6.json
* pattern_repetition
* pattern_juxtaposition

In [ ]:
show_examples(load_examples(368)['train'])

In [ ]:
%%writefile task368.py
def p(j,A=range):
	c=len(j);E=1;k,W=0,0;l=[0,5];J,a=0,0
	for C in A(c):
		for e in A(c):
			if j[C][e]not in l and E:
				E=0;J,a=C,e;K=C;w=e
				while K<c and j[K][e]not in l:K+=1
				while w<c and j[C][w]not in l:w+=1
				k=K-C;W=w-e
	for C in A(c-k+1):
		for e in A(c-W+1):
			if j[C][e]==5:
				for L in A(k):
					for b in A(W):j[C+L][e+b]=j[J+L][a+b]
	return j


# [369] e8593010.json
* holes
* count_tiles
* loop_filling
* associate_colors_to_numbers

In [ ]:
show_examples(load_examples(369)['train'])

In [ ]:
%%writefile task369.py
def p(u):
	o=range;y=set();u=[y[:]for y in u]
	def i(o,d):
		if(o,d)in y or not(0<=o<10and 0<=d<10)or u[o][d]:return[]
		y.add((o,d));return[(o,d)]+sum([i(o+y,d+n)for(y,n)in[(-1,0),(1,0),(0,-1),(0,1)]],[])
	for n in o(10):
		for b in o(10):
			if u[n][b]==0and(n,b)not in y:
				a=i(n,b)
				for(p,r)in a:u[p][r]=abs(len(a)-4)
	return u

# [370] e8dc4411.json
* pattern_expansion
* direction_guessing

In [ ]:
show_examples(load_examples(370)['train'])

In [ ]:
%%writefile task370.py
def p(I,t=range):
 if not I or not I[0]:return I
 D,E=len(I),len(I[0]);X=[B for A in I for B in A];Y=min(set(X),key=X.count);C={(A,B)for A in t(D)for B in t(E)if I[A][B]==0}
 if not C:return I
 R={(A,B)for A in t(D)for B in t(E)if I[A][B]==Y}
 def o(g):A=[A for(A,B)in g];B=[A for(B,A)in g];return min(A),min(B),max(A),max(B)
 def a(g):A,B,C,D=o(g);return(A+C)//2,(B+D)//2
 def g(a,b):
  A,B=a;C,D=b;E=[]
  if A==C:
   F,G=sorted((B,D))
   for H in t(F,G+1):E.append((A,H))
  elif B==D:
   I,J=sorted((A,C))
   for K in t(I,J+1):E.append((K,B))
  elif C-A==D-B:
   I,J=sorted((A,C));F,G=sorted((B,D))
   for(K,H)in zip(t(I,J+1),t(F,G+1)):E.append((K,H))
  elif C-A==-(D-B):
   I,J=sorted((A,C));F,G=sorted((B,D))
   for(K,H)in zip(t(I,J+1),t(G,F-1,-1)):E.append((K,H))
  return E
 S,T,U,V=o(C);h=set(g((S,T),(U,V))).issubset(C);L=lambda x:(x>0)-(x<0);A,j=a(C);k,l=a(R);b,W=L(k-A),L(l-j);M,N=U-S+1,V-T+1;G=M*b,N*W;c=len(C)==2*(M+N)-4;O=G if c else G if h else(G[0]-L(G[0]),G[1]-L(G[1]));F=set()
 if c:
  for H in t(1,5):
   A,B=H*O[0],H*O[1];d,m,e,n=S+A,T+B,U+A,V+B;J,K=0<=d and e<D,0<=m and n<E
   if J and K:F.update({(C+A,D+B)for(C,D)in C});continue
   if H==1:F.update({(C+A,F+B)for(C,F)in C if 0<=C+A<D and 0<=F+B<E});break
   if not J and K:F.update({(C+A,E+B)for(C,E)in C if 0<=C+A<D})
   elif not J and not K:F.update({(C+A,F+B)for(C,F)in C if 0<=C+A<D and 0<=F+B<E})
   elif J and not K and(W<0 or M==3and N==3):F.update({(D+A,C+B)for(D,C)in C if 0<=C+B<E})
   elif J and not K and(e>=D-1or d<=0):F.update({(C+A,F+B)for(C,F)in C if 0<=C+A<D and 0<=F+B<E})
   break
 else:
  for H in t(1,5):A,B=H*O[0],H*O[1];F.update({(C+A,F+B)for(C,F)in C if 0<=C+A<D and 0<=F+B<E})
 if len(C)==8and M==3and N==3and len(R)==1:T,p=next(iter(R));P,Q=T+b,p+W;q={(P-1,Q+A)for A in(-1,0,1)}|{(P,Q-1),(P,Q+1)}|{(P+1,Q+A)for A in(-1,0,1)};F|={(A,B)for(A,B)in q if 0<=A<D and 0<=B<E}
 A=[list(A)for A in I]
 for(r,l)in F:A[r][l]=Y
 return A

# [371] e9614598.json
* pattern_expansion
* direction_guessing
* measure_length

In [ ]:
show_examples(load_examples(371)['train'])

In [ ]:
%%writefile task371.py
def p(j,A=enumerate):
 c,E=zip(*((i,j)for i,r in A(j)for j,W in A(r)if W))
 for k,W in((0,0),(-1,0),(1,0),(0,-1),(0,1)):j[sum(c)//2+k][sum(E)//2+W]=3
 return j

# [372] e98196ab.json
* detect_wall
* separate_images
* image_juxtaposition

In [ ]:
show_examples(load_examples(372)['train'])

In [ ]:
%%writefile task372.py
p=lambda g:[[g[i][j]or g[i+6][j]for j in range(11)]for i in range(5)]

# [373] e9afcf9a.json
* pattern_modification

In [ ]:
show_examples(load_examples(373)['train'])

In [ ]:
%%writefile task373.py
p=lambda g:[[[g[i][j],g[1-i][j]][j%2]for j in range(6)]for i in range(2)]

# [374] ea32f347.json
* separate_shapes
* count_tiles
* recoloring
* associate_colors_to_ranks

In [ ]:
show_examples(load_examples(374)['train'])

In [ ]:
%%writefile task374.py
def p(p):
 y=len(p);y=len(p[0]);t=[]
 for a in range(y):
  for d in range(y):
   if p[a][d]==5:
    i=[(a,d)];p[a][d]=0;g=[]
    while i:
     d,n=i.pop();g+=[(d,n)]
     for d,n in((d+1,n),(d-1,n),(d,n+1),(d,n-1)):
      if 0<=d<y and 0<=n<y and p[d][n]==5:p[d][n]=0;i+=[(d,n)]
    t+=g,
 for g,l in zip(sorted(t,key=len),(2,4,1)):
  for d,n in g:p[d][n]=l
 return p

# [375] ea786f4a.json
* pattern_modification
* draw_line_from_point
* diagonals

In [ ]:
show_examples(load_examples(375)['train'])

In [ ]:
%%writefile task375.py
def p(j):
 for A in range(len(j)):j[A][A]=j[-A-1][A]=0
 return j

# [376] eb281b96.json
* image_repetition
* image_reflection

In [ ]:
show_examples(load_examples(376)['train'])

In [ ]:
%%writefile task376.py
p=lambda j:(j+j[-2:0:-1])*2+j[:1]

# [377] eb5a1d5d.json
* summarize

In [ ]:
show_examples(load_examples(377)['train'])

In [ ]:
%%writefile task377.py
def p(D,e=len,i=range):
 n,n=e(D),e(D[0]);R,O=0,[]
 for o in i(e(D)):
  C=[D[o][0]]
  for r in i(e(D[o])-1):
   if D[o][r+1]!=D[o][r]:C+=[D[o][r+1]]
  if e(C)>R:R=e(C);O=C[:]
 D=[O[:]for o in i(e(O))]
 for o in i(e(D)//2):
  for R in i(o,e(D[0])-o-1):D[o][R]=D[o][o];D[-(o+1)][R]=D[-(o+1)][o]
 return D

# [378] ec883f72.json
* pattern_expansion
* draw_line_from_point
* diagonals

In [ ]:
show_examples(load_examples(378)['train'])

In [ ]:
%%writefile task378.py
def f(o,x,s,i,h):
 f=o[x][s]
 if f==0:return
 if not sum(o[x][s+d]==f for d in(1,-1))==sum(o[x+d][s]==f for d in(1,-1))==1:return
 k,g,d,a=2*(o[x+1][s]==f)-1,2*(o[x][s+1]==f)-1,s,x
 if o[x+k][s+g]==f:return
 while 1<=d<h-1and 1<=a<i-1:a-=k;d-=g;o[a][d]=o[x+2*k][s+2*g]
def p(o):
 i,h=len(o),len(o[0])
 for x in range(1,i-1):
  for s in range(1,h-1):f(o,x,s,i,h)
 return o

# [379] ecdecbb3.json
* pattern_modification
* draw_line_from_point

In [ ]:
show_examples(load_examples(379)['train'])

In [ ]:
%%writefile task379.py
def p(a):
 d=None;F=range;t=lambda g:[list(A)for A in zip(*g)];B=a;L=0
 if not any(all(A==8for A in A)for A in a):B,L=t(a),1
 I,M=len(B),len(B[0]);e=[A[:]for A in B];N=[A for A in F(I)if all(A==8for A in B[A])]
 for S in F(I):
  for s in F(M):
   if B[S][s]!=2:continue
   for A in(max([A for A in N if A<S],default=d),min([A for A in N if A>S],default=d)):
    if A is d:continue
    for a in(A-1,A,A+1):
     if 0<=a<I:
      for l in(s-1,s,s+1):
       if 0<=l<M:e[a][l]=2if a==A and l==s else 8
    o=1-2*(A<S)
    for a in F(S,A-o,o):e[a][s]=2
 return t(e)if L else e

# [380] ed36ccf7.json
* image_rotation

In [ ]:
show_examples(load_examples(380)['train'])

In [ ]:
%%writefile task380.py
p=lambda j:[*zip(*j)][::-1]

# [381] ef135b50.json
* draw_line_from_point
* bridges
* connect_the_dots

In [ ]:
show_examples(load_examples(381)['train'])

In [ ]:
%%writefile task381.py
def p(j,A=range):
 c=len(j)
 for E in A(1,c-1):
  k=W=0
  for l in A(c):
   J=j[E][l];k=[k,1][k<1and J>1]
   if k==1and J<1:k=2;W=[W,l][~W]
   if k>1and J>1:
    for a in A(W,l):j[E][a]=9;k=1;W=0
 return j

# [382] f15e1fac.json
* draw_line_from_point
* gravity
* obstacles
* direction_guessing

In [ ]:
show_examples(load_examples(382)['train'])

In [ ]:
%%writefile task382.py
n=range
q=len
def p(f,e=1):
 for i in n(4):
  f=list(map(list,zip(*f[::-1])))
  h,w=q(f),q(f[0])
  if e:
   for i in n(h):
    if f[i][0]==8 and f[0].count(2)+f[-1].count(2)>0:
     e=0
     for c in n(w):
      if f[0][c]==2:e+=1
      if f[-1][c]==2:e-=1
      if 0<=i+e<h:f[i+e][c]=8
     e=0
 return f

# [383] f1cefba8.json
* draw_line_from_point
* pattern_modification

In [ ]:
show_examples(load_examples(383)['train'])

In [ ]:
%%writefile task383.py
def p(I,t=enumerate,p=range):
 from collections import Counter as M;a,b=len(I),len(I[0]);c=M(B for n in I for B in n);R=max(c,key=c.get);K=[(n,B)for n in p(a)for B in p(b)if I[n][B]!=R]
 if not K:return I
 B=min(n for(n,B)in K);G=max(n for(n,B)in K);C=min(n for(B,n)in K);H=max(n for(B,n)in K);L=M(I[n][B]for n in p(B,G+1)for B in p(C,H+1));R={n for n in L if n!=R};D=set()
 for E in p(B,G+1):D.add(I[E][C]);D.add(I[E][H])
 for F in p(C,H+1):D.add(I[B][F]);D.add(I[G][F])
 D.discard(R);n=R-D
 if n:n=max(n,key=lambda c:(L[c],-c))
 else:n=min(R,key=lambda c:(L[c],c))
 if D:e=max(D,key=lambda c:(L[c],-c))
 else:f=[B for B in R if B!=n];e=max(f,key=lambda c:(L[c],-c))if f else n
 N=[sum(1for C in p(C,H+1)if I[B][C]==n)for B in p(B,G+1)];O=[sum(1for B in p(B,G+1)if I[B][C]==n)for C in p(C,H+1)];T=[n for n in N if n];g=[n for n in O if n];h=[];D=[]
 if T:
  P=M(n for n in N if n);U=max(P.items(),key=lambda kv:(kv[1],-kv[0]))[0];V=min(T);l=max(T)
  if V>2and l-V==1:U=V
  h=[n for(n,B)in t(N)if B==U];D=[n for(n,B)in t(N)if B>U]
 j=[];k=[]
 if g:
  P=M(n for n in O if n);W=max(P.items(),key=lambda kv:(kv[1],-kv[0]))[0];X=min(g)
  if X>2and P[X]>1:W=X
  j=[n for(n,B)in t(O)if B==W];k=[n for(n,B)in t(O)if B>W]
 J=[list(n)for n in I]
 for E in p(B,G+1):
  for F in p(C,H+1):J[E][F]=e
 for l in h:
  Q=B+l
  for Z in j:J[Q][C+Z]=n
 for l in D:
  Q=B+l
  for F in p(0,C):J[Q][F]=n
  for F in p(H+1,b):J[Q][F]=n
 for Z in k:
  l=C+Z
  for E in p(0,B):J[E][l]=n
  for E in p(G+1,a):J[E][l]=n
 return J

# [384] f25fbde4.json
* crop
* image_resizing

In [ ]:
show_examples(load_examples(384)['train'])

In [ ]:
%%writefile task384.py
def p(r):a=[max(r)>0 for r in r].index(1);e=len(r)-1-[max(r)>0for r in r][::-1].index(1);t=[r for r,i in enumerate(zip(*r))if max(i)>0];i=t[0];u=t[-1];return[[r for r in r[i:u+1]for j in[0]*2]for r in r[a:e+1]for j in[0]*2]

# [385] f25ffba3.json
* pattern_repetition
* pattern_reflection

In [ ]:
show_examples(load_examples(385)['train'])

In [ ]:
%%writefile task385.py
def p(j,A=enumerate):
 for c,E in A(j):
  for k,W in A(E):
   if c<len(j)//2:j[c][k]=j[-(c+1)][k]
 return j

# [386] f2829549.json
* detect_wall
* separate_images
* take_complement
* pattern_intersection

In [ ]:
show_examples(load_examples(386)['train'])

In [ ]:
%%writefile task386.py
def p(j):
 for A in range(4):
  for c in range(3):
   j[A][c]+=j[A][c+4]
   if j[A][c]>0:j[A][c]=0
   else:j[A][c]=3
 return[R[:3]for R in j]

# [387] f35d900a.json
* pattern_expansion

In [ ]:
show_examples(load_examples(387)['train'])

In [ ]:
%%writefile task387.py
def p(I,n=range):
 L,o=len(I),len(I[0]);D=[list(A)for A in I];C,N=[],set()
 for A in n(L):
  for B in n(o):
   E=I[A][B]
   if E!=0:C.append((A,B,E));N.add(E)
 if not C:return I
 F=sorted(N)
 def S(E):return F[0]if len(F)==1or E==F[1]else F[1]
 for(A,B,E)in C:
  T=S(E)
  for s in(-1,0,1):
   for E in(-1,0,1):
    if s==0and E==0:continue
    I,E=A+s,B+E
    if 0<=I<L and 0<=E<o:D[I][E]=T
 L=min(A for(A,B,B)in C);H=max(A for(A,B,B)in C);J=min(B for(A,B,A)in C);e=max(B for(A,B,A)in C)
 if e>J:
  for B in n(J+1,e):
   if min(B-J,e-B)%2==0:D[L][B]=5;D[H][B]=5
 if H>L:
  for A in n(L+1,H):
   if min(A-L,H-A)%2==0:D[A][J]=5;D[A][e]=5
 return D

# [388] f5b8619d.json
* pattern_expansion
* draw_line_from_point
* image_repetition

In [ ]:
show_examples(load_examples(388)['train'])

In [ ]:
%%writefile task388.py
def p(g):R=range;n=len(g);c={j for i in R(n)for j in R(n)if g[i][j]};m=[[8if g[i][j]==0and j in c else g[i][j]for j in R(n)]for i in R(n)];return[[m[i%n][j%n]for j in R(2*n)]for i in R(2*n)]


# [389] f76d97a5.json
* take_negative
* recoloring
* associate_colors_to_colors

In [ ]:
show_examples(load_examples(389)['train'])

In [ ]:
%%writefile task389.py
def p(r):t=[i for t in r for i in t];t=[t for t in set(t)if t not in[0,5]][0];r=[[t if i==5else 0for i in i]for i in r];return r

# [390] f8a8fe49.json
* pattern_moving
* pattern_reflection

In [ ]:
show_examples(load_examples(390)['train'])

In [ ]:
%%writefile task390.py
def p(g,f=enumerate,a=range):
 t=[t[:]for t in g];M=len(t);S=len(t[0]);s=[n for t in t for(n,C)in f(t)if C==2]
 if not s:return t
 O=min(s);F=max(s);s=[t for(t,n)in f(t)if 2in n];e=min(s);U=max(s);s=[t for(t,n)in f(t)if all(n[t]==2for t in a(O,F+1))];n=bool(s)
 if n:
  J=min(s);K=max(s)
  for C in a(J+1,K):
   s=[n for n in a(O+1,F)if t[C][n]==5]
   if not s:continue
   for n in s:t[C][n]=0
   n=C-J;A=K-C
   if n<=A:
    G=J-n
    if 0<=G<M:
     for n in s:t[G][n]=5
   else:
    G=K+A
    if 0<=G<M:
     for n in s:t[G][n]=5
 else:
  for C in a(e,U+1):
   s=[n for n in a(O+1,F)if t[C][n]==5]
   if not s:continue
   for n in s:
    E=n-O;R=F-n
    if E<=R:s=O-E
    else:s=F+R
    if 0<=s<S:t[C][s]=5
    t[C][n]=0
 return t

# [391] f8b3ba0a.json
* detect_grid
* find_the_intruder
* dominant_color
* count_tiles
* summarize
* order_numbers

In [ ]:
show_examples(load_examples(391)['train'])

In [ ]:
%%writefile task391.py
p=lambda j:[[k]for k,_ in __import__('collections').Counter(i for r in j for i in r).most_common(5)[2:]]

# [392] f8c80d96.json
* pattern_expansion
* background_filling

In [ ]:
show_examples(load_examples(392)['train'])

In [ ]:
%%writefile task392.py
p=lambda n,r=range(10):[[(a:=max(sum(n,[])),5,5)[min(max(abs(f-b),abs(m-s))for b in r for s in r if n[b][s])%(3-(f'{a}, 0, {a}'in str(n)))]for m in r]for f in r]

# [393] f8ff0b80.json
* separate_shapes
* count_tiles
* summarize
* order_numbers

In [ ]:
show_examples(load_examples(393)['train'])

In [ ]:
%%writefile task393.py
p=lambda j:[[k]for k,_ in __import__('collections').Counter(i for r in j for i in r).most_common(4)[1:]]

# [394] f9012d9b.json
* pattern_expansion
* pattern_completion
* crop

In [ ]:
show_examples(load_examples(394)['train'])

In [ ]:
%%writefile task394.py
def p(g):
 s=len(g);b=[(r,c)for r in range(s)for c in range(s)if g[r][c]==0]
 if not b:return[[]]
 R,C=min(r for r,c in b),min(c for r,c in b);H,W=max(r for r,c in b)-R+1,max(c for r,c in b)-C+1
 for m in range(2,5):
  c,v=[],1
  for r in range(m):
   for k in range(m):
    x=None
    for i in range(r,s,m):
     for j in range(k,s,m):
      if g[i][j]!=0:
       if x is None:x=g[i][j]
       elif x!=g[i][j]:v=0;break
     if not v:break
    if not v or x is None:v=0;break
    c.append(x)
   if not v:break
  if v and len(c)==m*m:
   if all(g[r][k]==0 or g[r][k]==c[r%m*m+k%m]for r in range(s)for k in range(s)):
    return[[c[(R+r)%m*m+(C+k)%m]for k in range(W)]for r in range(H)]
 return[[]]

# [395] fafffa47.json
* separate_images
* take_complement
* pattern_intersection

In [ ]:
show_examples(load_examples(395)['train'])

In [ ]:
%%writefile task395.py
def p(g):t,b=g[:3],g[3:];return[[2if t[r][c]==b[r][c]==0else 0for c in range(3)]for r in range(3)]

# [396] fcb5c309.json
* rectangle_guessing
* separate_images
* count_tiles
* take_maximum
* crop
* recoloring

In [ ]:
show_examples(load_examples(396)['train'])

In [ ]:
%%writefile task396.py
def p(j):
	A=range;c,E=len(j),len(j[0]);k={}
	for W in A(c):
		for l in A(E):
			if j[W][l]:k[j[W][l]]=k.get(j[W][l],0)+1
	J,a=max(k,key=k.get),min(k,key=k.get);C,e=0,None
	for K in A(c-2):
		for w in A(E-2):
			for L in A(K+2,c):
				for b in A(w+2,E):
					if all(j[K][A]==J for A in A(w,b+1))and all(j[L][A]==J for A in A(w,b+1))and all(j[A][w]==J for A in A(K,L+1))and all(j[A][b]==J for A in A(K,L+1)):
						d=(L-K+1)*(b-w+1)
						if d>C:C,e=d,(K,w,L,b)
	K,w,L,b=e;return[[a if j[K+L][w+A]==J else j[K+L][w+A]for A in A(b-w+1)]for L in A(L-K+1)]

# [397] fcc82909.json
* pattern_expansion
* separate_images
* count_different_colors

In [ ]:
show_examples(load_examples(397)['train'])

In [ ]:
%%writefile task397.py
def p(i,f=range):
	d,r=len(i),len(i[0]);n=[]
	for m in f(d-1):
		for l in f(r-1):
			e=i[m][l],i[m][l+1],i[m+1][l],i[m+1][l+1]
			if all(e):n+=[(m,l,len(set(e)))]
	for(m,l,a)in n:
		for n in f(a):
			e=m+2+n
			if e<d:i[e][l]=i[e][l+1]=3
	return i

# [398] feca6190.json
* pattern_expansion
* image_expansion
* draw_line_from_point
* diagonals

In [ ]:
show_examples(load_examples(398)['train'])

In [ ]:
%%writefile task398.py
def p(g,L=len,R=range):
	A=R(L([A for A in set(g[0])if A>0])*5);B=[[0for A in A]for B in A];g=g[0];C=0
	for E in A:
		for D in R(5):
			try:B[-(E+1)][D+C]=g[D]
			except:pass
		C+=1
	return B

# [399] ff28f65a.json
* count_shapes
* associate_images_to_numbers

In [ ]:
show_examples(load_examples(399)['train'])

In [ ]:
%%writefile task399.py
def p(i,r=range):
 l=sum(i[d][h]==i[d][h+1]==i[d+1][h]==i[d+1][h+1]==2for d in r(len(i)-1)for h in r(len(i)-1));d=[[0]*3for d in'***']
 for h in r(l):d[h*2//3][h*2%3]=1
 return d

# [400] ff805c23.json
* pattern_expansion
* pattern_completion
* crop

In [ ]:
show_examples(load_examples(400)['train'])

In [ ]:
%%writefile task400.py
def p(D,L=len,o=range):
 i,i,n,E=L(D),L(D[0]),[],[];s=1
 for L in o(i//2+1):
  for t in o(i):
   if D[L][t]==s:D[L][t]=D[-(L+1)][t];n+=[L];E+=[t]
   if D[-(L+1)][t]==s:D[-(L+1)][t]=D[L][t];n+=[i-(L+1)];E+=[t]
 for L in o(i):
  for t in o(i//2+1):
   if D[L][t]==s:D[L][t]=D[L][-(t+1)];n+=[L];E+=[t]
   if D[L][-(t+1)]==s:D[L][-(t+1)]=D[L][t];n+=[L];E+=[i-(t+1)]
 D=D[min(n):max(n)+1];D=[L[min(E):max(E)+1]for L in D];return D

In [ ]:
# ===================================================================
# 步骤 3：挑战者集结 (完整环境复刻)
# ===================================================================
# 【【【使用您提供的环境准备逻辑】】】
print("\n⚔️ 步骤 3/9: 挑战者集结 (完整环境复刻)")

zip_sources = [
    "/kaggle/input/oh-barnacles",
    "/kaggle/input/road-to-400-collaboration",
    "/kaggle/input/big-zippa",
    "/kaggle/input/python-minifier-applied", 
    "/kaggle/input/golf-code-optimization-trials", 
    "/kaggle/input/400-task-with-smart-solution-search-verification",
    "/kaggle/input/solutions-for-all-400-tasks-ensemble",
    "/kaggle/input/neuroips-4-of-some-lessons-learned",
    "/kaggle/input/source-code-for-full-400-solutions",
    "/kaggle/input/code-compression"
]
raw_dir_sources = [
    "/kaggle/input/250917google-code-golf-submission-log/submission",
    "/kaggle/input/neurips-code-golf-2025-optim"
]

EXTRACTED_CHALLENGERS_DIR = "/kaggle/working/extracted_challengers"
if os.path.exists(EXTRACTED_CHALLENGERS_DIR): shutil.rmtree(EXTRACTED_CHALLENGERS_DIR)
os.makedirs(EXTRACTED_CHALLENGERS_DIR, exist_ok=True)
processed_challenger_dirs = {}
submission_idx_counter = 0

print("   - 正在处理 ZIP 文件源...")
for path in zip_sources:
    source_name = os.path.basename(path)
    dest_folder = os.path.join(EXTRACTED_CHALLENGERS_DIR, f"s_{submission_idx_counter}_{source_name}")
    zip_path = os.path.join(path, "submission.zip")
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, "r") as z: z.extractall(dest_folder)
        processed_challenger_dirs[f"s_{submission_idx_counter}_{source_name}"] = dest_folder
        print(f"     - ✅ {source_name} -> {os.path.basename(dest_folder)}")
        submission_idx_counter += 1
    else:
        print(f"     - ⚠️ 在 {source_name} 中未找到 submission.zip")

print("   - 正在处理裸 .py 文件目录源...")
for path in raw_dir_sources:
    source_name_base = os.path.basename(path)
    source_name = os.path.basename(os.path.dirname(path)) if source_name_base == 'submission' else source_name_base
    dest_folder = os.path.join(EXTRACTED_CHALLENGERS_DIR, f"s_{submission_idx_counter}_{source_name}")
    if os.path.isdir(path):
        os.makedirs(dest_folder, exist_ok=True)
        files_copied = sum(1 for f in os.listdir(path) if f.endswith(".py") and shutil.copy(os.path.join(path, f), dest_folder))
        processed_challenger_dirs[f"s_{submission_idx_counter}_{source_name}"] = dest_folder
        print(f"     - ✅ {source_name} -> {os.path.basename(dest_folder)} ({files_copied} files copied)")
        submission_idx_counter += 1
    else:
        print(f"     - ⚠️ 目录不存在: {path}")

print(f"   - 集结完毕！共 {len(processed_challenger_dirs)} 组挑战者准备攻擂。")
print("✅ 步骤 3/9 完成")

In [ ]:
# ===================================================================
# 步骤 4：全局预分析与策略制定
# ===================================================================
print("\n-步骤 4/9: 全局预分析与策略制定")
BASELINE_SOURCE_NAME, BASELINE_DIR = "Embedded_Notebook_Baseline", "/kaggle/working/"
all_sources = {BASELINE_SOURCE_NAME: BASELINE_DIR}; all_sources.update(processed_challenger_dirs)
print(f"   - 识别到1个基线源和 {len(processed_challenger_dirs)} 组挑战者。")
print("   - 正在构建代码语料库并分析字节替换机会...")
corpus = ""
for task_num in tqdm(range(1, 401), desc="扫描代码", leave=False):
    for source_dir in all_sources.values():
        task_path = os.path.join(source_dir, f"task{task_num:03d}.py")
        if os.path.exists(task_path):
            with open(task_path, 'rb') as f:
                corpus += decode_and_clean(f.read(), task_num).decode('latin-1', errors='ignore')
used_chars = set(corpus)
free_chars = [chr(i) for i in range(33, 127) if chr(i) not in used_chars and chr(i) not in ('"', "'", "\\")]
substitutions = {b' in ': free_chars.pop(0).encode('latin-1')} if free_chars else {}
if free_chars: substitutions[b'range('] = free_chars.pop(0).encode('latin-1')
print("   - 全局分析完成！")
if substitutions:
    print("   - 发现字节替换机会:")
    for old, new in substitutions.items(): print(f"     - {repr(old)} -> {repr(new)}")
else: print("   - 未找到合适的字节替换机会。")
print("✅ 步骤 4/9 完成")


# ===================================================================
# 步骤 5：终极对决
# ===================================================================
print("\n🔥 步骤 5/9: 终极对决")
FINAL_SUBMISSION_DIR = "/kaggle/working/final_submission"
if os.path.exists(FINAL_SUBMISSION_DIR): shutil.rmtree(FINAL_SUBMISSION_DIR)
os.makedirs(FINAL_SUBMISSION_DIR, exist_ok=True)
analysis_data, unsolved_tasks, total_score, start_time = [], [], 0, time.time()
temp_verify_path = os.path.join(BASELINE_DIR, 'task.py')
for task_num in tqdm(range(1, 401), desc="进行终极对决", unit="task"):
    task_id, examples = f"task{task_num:03d}.py", load_examples(task_num)
    champion_info = {"c_code": None, "c_len": float('inf'), "o_len": 0, "src": "N/A"}
    for source_name, source_dir in all_sources.items():
        task_path = os.path.join(source_dir, task_id)
        if not os.path.exists(task_path): continue
        with open(task_path, 'rb') as f: original_code_bytes = decode_and_clean(f.read(), task_num)
        with open(temp_verify_path, 'wb') as temp_f: temp_f.write(original_code_bytes)
        try:
            _, agi_w, _, agen_w = simple_verify_program(task_num, examples)
            is_valid = (agi_w + agen_w == 0)
        except Exception: is_valid = False
        if not is_valid: continue
        compressed_code = zip_src_uber_extreme(original_code_bytes, substitutions)
        if len(compressed_code) < champion_info["c_len"]:
            champion_info = {"c_code": compressed_code, "c_len": len(compressed_code), "o_len": len(original_code_bytes), "src": source_name}
    if champion_info["c_code"] is not None:
        with open(os.path.join(FINAL_SUBMISSION_DIR, task_id), 'wb') as f: f.write(champion_info["c_code"])
        score = max(1, 2500 - champion_info["c_len"])
        total_score += score
        analysis_data.append({"task": task_num, "best_source": champion_info["src"], "original_len": champion_info["o_len"], "final_len": champion_info["c_len"], "score": score, "improvement": champion_info["o_len"] - champion_info["c_len"], "compression_ratio": champion_info["c_len"] / (champion_info["o_len"] or 1)})
    else:
        unsolved_tasks.append(task_num); total_score += 0.001
df_analysis = pd.DataFrame(analysis_data)
print(f"   - 对决结束！耗时: {time.time() - start_time:.2f} 秒。 ⭐ 预计总分: {total_score:,.3f}")
if unsolved_tasks: print(f"   - ⚠️ {len(unsolved_tasks)} 个任务未找到任何有效解决方案: {unsolved_tasks}")
print("✅ 步骤 5/9 完成")

In [ ]:
# ===================================================================
# 步骤 6-9：打包、清理与分析
# ===================================================================
print("\n📦 步骤 6/9: 创建最终提交文件")
submission_zip_path = "/kaggle/working/submission.zip"; zip_size=0
with zipfile.ZipFile(submission_zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for tf in sorted(os.listdir(FINAL_SUBMISSION_DIR)): zf.write(os.path.join(FINAL_SUBMISSION_DIR, tf), arcname=tf)
if os.path.exists(submission_zip_path): zip_size = os.path.getsize(submission_zip_path)
print(f"   - 提交文件创建成功！大小: {zip_size/1024:.2f} KB")
print("✅ 步骤 6/9 完成")

In [ ]:
# ===================================================================
# 步骤 7：清理中间文件 (清理)
# ===================================================================
print("\n🧹 步骤 7/9: 清理中间文件")
if 'EXTRACTED_CHALLENGERS_DIR' in locals() and os.path.exists(EXTRACTED_CHALLENGERS_DIR): shutil.rmtree(EXTRACTED_CHALLENGERS_DIR)
if os.path.exists(FINAL_SUBMISSION_DIR): shutil.rmtree(FINAL_SUBMISSION_DIR)
if os.path.exists(temp_verify_path): os.remove(temp_verify_path)
print("   - 已清理所有临时目录和文件。")
print("✅ 步骤 7/9 完成")

In [ ]:
# ===================================================================
# 步骤 8：深度洞察 (复盘)
# ===================================================================
print("\n📊 步骤 8/9: 深度洞察 - 全面可视化分析")

# --- 8.1 执行摘要 ---
print("\n🎯 执行摘要")
print("─" * 60)
source_wins = df_analysis['best_source'].value_counts()
compression_beneficial_count = (df_analysis['improvement'] > 0).sum()
total_bytes_saved = df_analysis['improvement'].sum()
baseline_wins = source_wins.get(BASELINE_SOURCE_NAME, 0)
print(f"  • 预计总分: {total_score:,.3f} | 平均分/任务: {total_score/400:.2f}")
print(f"  • 基线方案胜出率: {(baseline_wins/400)*100:.1f}% ({baseline_wins}/400 个任务)")
print(f"  • 压缩策略在 {(compression_beneficial_count/400)*100:.1f}% 的任务中有效")
print(f"  • 通过压缩总共节省: {total_bytes_saved:,.0f} 字节 (相对于最优原始解)")
if len(source_wins) > 1:
    challenger_wins_series = source_wins.drop(BASELINE_SOURCE_NAME, errors='ignore')
    if not challenger_wins_series.empty:
        print(f"  • 贡献最大的挑战者: {challenger_wins_series.index[0]} (胜出 {challenger_wins_series.iloc[0]} 次)")

# --- 8.2 尺寸分布分析 (6合1图) ---
print("\n📊 正在生成：尺寸分布分析...")
fig, axes = plt.subplots(2, 3, figsize=(20, 12)); fig.suptitle('代码尺寸分布深度分析', fontsize=20, fontweight='bold')
sns.histplot(df_analysis['original_len'], bins=50, ax=axes[0,0], color='#3498db', kde=True).set(title='最优原始解尺寸分布', xlabel='字节数')
sns.histplot(df_analysis['final_len'], bins=50, ax=axes[0,1], color='#2ecc71', kde=True).set(title='最终压缩后尺寸分布', xlabel='字节数')
sns.violinplot(data=df_analysis[['original_len', 'final_len']], ax=axes[0,2]).set_title('尺寸缩减阶段')
sns.histplot(df_analysis['score'], bins=50, ax=axes[1,0], color='#f39c12', kde=True).set(title='分数分布', xlabel='分数')
sns.histplot(df_analysis['compression_ratio'], bins=40, ax=axes[1,1], color='#e74c3c', kde=True).set(title='压缩率 (最终/原始)', xlabel='比率')
sns.histplot(df_analysis[df_analysis['improvement'] > 0]['improvement'], bins=40, ax=axes[1,2], color='#9b59b6', kde=True).set(title='节约字节数分布 (当压缩有效时)', xlabel='字节数')
plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()

# --- 8.3 来源表现对比 (4合1图) ---
print("\n📊 正在生成：来源表现对比分析...")
fig, axes = plt.subplots(2, 2, figsize=(20, 15)); fig.suptitle('各来源最终胜出表现对比', fontsize=20, fontweight='bold')
colors = ['#FFD700' if BASELINE_SOURCE_NAME in x else '#87CEEB' for x in source_wins.index]
sns.barplot(x=source_wins.values, y=source_wins.index, ax=axes[0,0], palette=colors).set(title='各来源最终胜出次数 (基线为金色)', xlabel='任务数量')
challenger_wins = source_wins.drop(BASELINE_SOURCE_NAME, errors='ignore')
if not challenger_wins.empty:
    axes[0,1].pie(challenger_wins.values, labels=challenger_wins.index, autopct='%1.1f%%', startangle=90, colors=plt.cm.Set3.colors)
    axes[0,1].set_title('各挑战者胜出次数分布')
cumulative_wins = {source: np.cumsum(df_analysis['best_source'] == source) for source in source_wins.index}
for source, wins in cumulative_wins.items():
    axes[1,0].plot(df_analysis['task'], wins, label=source, lw=3 if BASELINE_SOURCE_NAME in source else 2, linestyle='-' if BASELINE_SOURCE_NAME in source else '--')
axes[1,0].legend(); axes[1,0].set(title='各来源累积胜出数 (基线为实线)', xlabel='任务编号', ylabel='累积胜出数')
task_ranges = [(i*50+1, (i+1)*50) for i in range(8)]
heatmap_data = pd.DataFrame(index=source_wins.index, columns=[f'{r[0]}-{r[1]}' for r in task_ranges], data=0)
for _, row in df_analysis.iterrows():
    if row['task'] > 400: continue
    range_idx = (row['task'] - 1) // 50
    heatmap_data.loc[row['best_source'], heatmap_data.columns[range_idx]] += 1
sns.heatmap(heatmap_data, ax=axes[1,1], cmap='YlOrRd', annot=True, fmt='d').set_title('来源在不同任务区间的主导性')
plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()

# --- 8.4 任务趋势分析 (4合1图) ---
print("\n📊 正在生成：任务趋势分析...")
fig, axes = plt.subplots(2, 2, figsize=(20, 12)); fig.suptitle('任务趋势与难度演变', fontsize=20, fontweight='bold')
window = 20
axes[0,0].plot(df_analysis['task'], df_analysis['final_len'], alpha=0.3, label='任务尺寸'); axes[0,0].plot(df_analysis['task'], df_analysis['final_len'].rolling(window, center=True).mean(), 'r-', lw=2, label=f'{window}任务滚动平均')
axes[0,0].set(title='最终代码尺寸随任务编号的变化', xlabel='任务编号', ylabel='字节数'); axes[0,0].legend()
axes[0,1].plot(df_analysis['task'], df_analysis['score'], alpha=0.3, label='任务得分', color='green'); axes[0,1].plot(df_analysis['task'], df_analysis['score'].rolling(window, center=True).mean(), 'b-', lw=2, label=f'{window}任务滚动平均')
axes[0,1].set(title='分数随任务编号的变化', xlabel='任务编号', ylabel='分数'); axes[0,1].legend()
compression_benefit_rate = (df_analysis['improvement'] > 0).rolling(50, center=True).mean() * 100
axes[1,0].plot(df_analysis['task'], compression_benefit_rate, lw=2, color='#e67e22'); axes[1,0].fill_between(df_analysis['task'], compression_benefit_rate, alpha=0.3, color='#e67e22')
axes[1,0].axhline(50, color='black', linestyle='--', alpha=0.5); axes[1,0].set(title='压缩有效率 (50任务滚动窗口)', xlabel='任务编号', ylabel='有效率 (%)')
df_analysis['difficulty'] = pd.qcut(df_analysis['final_len'], 4, labels=['简单', '中等', '困难', '专家'], duplicates='drop')
sns.countplot(x='difficulty', data=df_analysis, ax=axes[1,1], palette='Set2').set_title('任务难度分布 (基于最终尺寸)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()

# --- 8.5 交互式仪表盘 ---
print("\n📊 正在生成：交互式分析仪表盘...")
fig = make_subplots(rows=2, cols=3, subplot_titles=('尺寸 vs 分数', '最终胜出者来源', '压缩效益', '最终尺寸分布', '分数分布', '难度分布'), specs=[[{"type": "scatter"}, {"type": "bar"}, {"type": "pie"}], [{"type": "histogram"}, {"type": "histogram"}, {"type": "pie"}]])
fig.add_trace(go.Scatter(x=df_analysis['final_len'], y=df_analysis['score'], mode='markers', text=df_analysis['best_source'], marker=dict(color=df_analysis['task'], colorscale='Viridis', showscale=True, colorbar_title='Task #')), row=1, col=1)
fig.add_trace(go.Bar(x=source_wins.index, y=source_wins.values, marker_color=['gold' if BASELINE_SOURCE_NAME in x else 'lightblue' for x in source_wins.index]), row=1, col=2)
fig.add_trace(go.Pie(labels=['压缩有效', '压缩无效'], values=df_analysis['improvement'].gt(0).value_counts().values), row=1, col=3)
fig.add_trace(go.Histogram(x=df_analysis['final_len'], name='Final Size', marker_color='lightblue'), row=2, col=1)
fig.add_trace(go.Histogram(x=df_analysis['score'], name='Score', marker_color='lightgreen'), row=2, col=2)
difficulty_counts = df_analysis['difficulty'].value_counts()
if not difficulty_counts.empty: fig.add_trace(go.Pie(labels=difficulty_counts.index, values=difficulty_counts.values), row=2, col=3)
fig.update_layout(height=800, showlegend=False, title_text="代码高尔夫交互式分析仪表盘"); fig.write_html("/kaggle/working/interactive_dashboard.html")
print("   - ✅ 交互式仪表盘已保存至 'interactive_dashboard.html'")
fig.show()

print("\n" + "="*80); print("📈 分析完成 - 最终提交文件已准备就绪！"); print("✅ 步骤 8/9 完成\n")

In [ ]:
# ===================================================================
# 步骤 9：洞察与行动建议 (新增)
# ===================================================================
print("\n💡 步骤 9/9: 洞察与行动建议")
print("─" * 60)
print("   根据本次运行结果，建议您手动关注以下任务以寻求进一步突破：\n")

# 9.1 最终尺寸最大的任务
print("   - 📏 【尺寸最大】这些任务的代码最长，是优化的首要目标：")
top_longest = df_analysis.sort_values('final_len', ascending=False).head(10)
for _, row in top_longest.iterrows():
    print(f"     - Task {int(row['task']):03d}: {int(row['final_len'])} bytes (来自: {row['best_source']})")

# 9.2 压缩率最差的任务
print("\n   - 🧱 【压缩率最差】这些任务的代码可能结构混乱，重写可能带来巨大收益：")
top_worst_ratio = df_analysis[df_analysis['improvement'] > 0].sort_values('compression_ratio', ascending=False).head(10)
for _, row in top_worst_ratio.iterrows():
    print(f"     - Task {int(row['task']):03d}: 压缩率 {row['compression_ratio']:.2f} (从 {int(row['original_len'])} -> {int(row['final_len'])} bytes)")

# 9.3 被优化最狠的任务
print("\n   - ✨ 【优化潜力最大】研究这些任务的胜者源码，可以学到高超的压缩技巧：")
top_improvement = df_analysis.sort_values('improvement', ascending=False).head(10)
for _, row in top_improvement.iterrows():
    print(f"     - Task {int(row['task']):03d}: 节省 {int(row['improvement'])} bytes (胜者: {row['best_source']})")

print("\n✅ 步骤 9/9 完成")
print("\n🎉🎉🎉 所有流程执行完毕！🎉🎉🎉")